<a href="https://colab.research.google.com/github/osmarbraz/exemplos_BERT/blob/main/ExemplosWordEmbeddingContextualBERT_pt_br_sentenca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Exemplo de Word Embeddings(pt-br) Contextual usando BERT Transformers by HuggingFace

## **A execução pode ser feita através do menu Ambiente de Execução opção Executar tudo.**

Exemplos de uso de **Word Embeddings Contextuais** para **desambiguação** de documentos originais e permutados utilizando suas sentenças. No final do notebook estão os exemplos com os documentos:

*   documento original e permutado

**Link biblioteca Huggingface:**
https://github.com/huggingface/transformers


**Artigo original BERT Jacob Devlin:**
https://arxiv.org/pdf/1506.06724.pdf

## 0 - Preparação do ambiente
Preparação do ambiente para execução do exemplo.

###Tratamento de logs

Método para tratamento dos logs.

In [ ]:
# Biblioteca de logging
import logging

# Formatando a mensagem de logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

### Identificando o ambiente Colab

Cria uma variável para identificar que o notebook está sendo executado no Google Colaboratory.

In [ ]:
# Se estiver executando no Google Colaboratory
import sys

# Retorna true ou false se estiver no Google Colaboratory
IN_COLAB = 'google.colab' in sys.modules

## 1 - Instalação BERT da Hugging Face

Instala a interface pytorch para o BERT by Hugging Face. 

In [ ]:
# Instala a última versão da biblioteca
#!pip install transformers

# Instala uma versão específica da biblioteca
!pip install -U transformers==4.5.1

     |████████████████████████████████| 2.1MB 5.2MB/s 
     |████████████████████████████████| 3.3MB 39.4MB/s 
     |████████████████████████████████| 901kB 39.4MB/s 


## 2 - Download do arquivo do PyTorch Checkpoint

Lista de modelos da comunidade:
* https://huggingface.co/models

Português(https://github.com/neuralmind-ai/portuguese-bert):  
* **'neuralmind/bert-base-portuguese-cased'**
* **'neuralmind/bert-large-portuguese-cased'**

In [ ]:
# Importando as bibliotecas
import os

# Variável para setar o arquivo
URL_MODELO = None

# Comente uma das urls para carregar modelos de tamanhos diferentes(base/large)
# URL_MODELO do arquivo do modelo tensorflow
# arquivo menor(base) 1.1 Gbytes
URL_MODELO = "https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-base-portuguese-cased/bert-base-portuguese-cased_pytorch_checkpoint.zip"

# arquivo grande(large) 3.5 Gbytes
#URL_MODELO = "https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-large-portuguese-cased/bert-large-portuguese-cased_pytorch_checkpoint.zip"

# Se a variável foi setada
if URL_MODELO:

    # Diretório descompactação
    DIRETORIO_MODELO = '/content/modelo'

    # Recupera o nome do arquivo do modelo da URL_MODELO
    arquivo = URL_MODELO.split("/")[-1]

    # Nome do arquivo do vocabulário
    arquivo_vocab = "vocab.txt"

    # Caminho do arquivo na URL_MODELO
    caminho = URL_MODELO[0:len(URL_MODELO)-len(arquivo)]

    # Verifica se a pasta de descompactação existe na pasta corrente
    if os.path.exists(DIRETORIO_MODELO):
      print("Apagando diretório existente do modelo!")
      # Apaga a pasta e os arquivos existentes
      !rm -rf $DIRETORIO_MODELO      
    
    # Baixa o arquivo do modelo
    !wget $URL_MODELO
    # Descompacta o arquivo na pasta de descompactação
    !unzip -o $arquivo -d $DIRETORIO_MODELO

    # Baixa o arquivo do vocabulário
    # O vocabulário não está no arquivo compactado acima, mesma url mas arquivo diferente
    URL_MODELO_VOCAB = caminho + arquivo_vocab
    !wget $URL_MODELO_VOCAB
    
    # Coloca o arquivo do vocabulário no diretório de descompactação
    !mv $arquivo_vocab $DIRETORIO_MODELO
            
    # Move o arquivo para pasta de descompactação
    !mv $arquivo $DIRETORIO_MODELO
       
    print('Pasta do ' + DIRETORIO_MODELO + ' pronta!')
    
    # Lista a pasta corrente
    !ls -la $DIRETORIO_MODELO
else:
    print('Variável URL_MODELO não setada!')

--2021-05-28 09:15:49--  https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-base-portuguese-cased/bert-base-portuguese-cased_pytorch_checkpoint.zip
Resolving neuralmind-ai.s3.us-east-2.amazonaws.com (neuralmind-ai.s3.us-east-2.amazonaws.com)... 52.219.96.184
Connecting to neuralmind-ai.s3.us-east-2.amazonaws.com (neuralmind-ai.s3.us-east-2.amazonaws.com)|52.219.96.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 406220891 (387M) [application/zip]
Saving to: ‘bert-base-portuguese-cased_pytorch_checkpoint.zip’

bert-base-portugues 100%[===================>] 387.40M  87.7MB/s    in 4.6s    

2021-05-28 09:15:54 (83.5 MB/s) - ‘bert-base-portuguese-cased_pytorch_checkpoint.zip’ saved [406220891/406220891]

Archive:  bert-base-portuguese-cased_pytorch_checkpoint.zip
  inflating: /content/modelo/config.json  
  inflating: /content/modelo/pytorch_model.bin  
--2021-05-28 09:15:59--  https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-base-portuguese-ca

## 3 - Carregando o Tokenizador BERT

O tokenizador utiliza WordPiece, veja em [artigo original](https://arxiv.org/pdf/1609.08144.pdf).

Carregando o tokenizador da pasta '/content/modelo/' do diretório padrão se variável `URL_MODELO` setada.

**Caso contrário carrega da comunidade**

Por default(`do_lower_case=True`) todas as letras são colocadas para minúsculas. Para ignorar a conversão para minúsculo use o parâmetro `do_lower_case=False`. Esta opção também considera as letras acentuadas(ãçéí...), que são necessárias a língua portuguesa.

O parâmetro `do_lower_case` interfere na quantidade tokens a ser gerado apartir de um texto. Quando igual a `False` reduz a quantidade de tokens gerados.

In [ ]:
# Importando as bibliotecas do tokenizador
from transformers import BertTokenizer

# Se a variável URL_MODELO foi setada
if DIRETORIO_MODELO:
    # Carregando o Tokenizador
    print('Carrgando o tokenizador BERT do diretório ' + DIRETORIO_MODELO + '...')

    tokenizer = BertTokenizer.from_pretrained(DIRETORIO_MODELO, 
                                              do_lower_case=False)    
else:
    # Carregando o Tokenizador da comunidade
    print('Carregando o tokenizador da comunidade...')
    
    #tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=False)
    tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-large-portuguese-cased', 
                                              do_lower_case=False)

Carrgando o tokenizador BERT do diretório /content/modelo...


## 4 - Carregando o Modelo BERT(BertModel)

Se a variável `URL_MODELO` estiver setada carrega o modelo do diretório `content/modelo`.

Caso contrário carrega da comunidade.

Carregando o modelo da pasta '/content/modelo/' do diretório padrão.

A implementação do huggingface pytorch inclui um conjunto de interfaces projetadas para uma variedade de tarefas de PNL. Embora essas interfaces sejam todas construídas sobre um modelo treinado de BERT, cada uma possui diferentes camadas superiores e tipos de saída projetados para acomodar suas tarefas específicas de PNL.

A documentação para estas pode ser encontrada em [aqui](https://huggingface.co/transformers/v2.2.0/model_doc/bert.html).

Por default o modelo está em modo avaliação ou seja `model.eval()`.

-----------------------

Durante a avaliação do modelo, este retorna um número de diferentes objetos com base em como é configurado na chamada do método `from_pretrained`. 

Quando definimos `output_hidden_states = True` na chamada do método `from_pretrained`, retorno do modelo possui no terceiro item os estados ocultos(**hidden_states**) de todas as camadas.  Veja a documentação para mais detalhes: https://huggingface.co/transformers/model_doc/bert.html#bertmodel

Quando **`output_hidden_states = True`** model retorna:
- outputs[0] = last_hidden_state;
- outputs[1] = pooler_output; 
- outputs[2] = hidden_states.

Quando **`output_hidden_states = False`** ou não especificado model retorna:
- outputs[0] = last_hidden_state;
- outputs[1] = pooler_output.


**ATENÇÃO**: O parâmetro ´**output_hidden_states = True**´ habilita gerar as camadas ocultas do modelo. Caso contrário somente a última camada é mantida. Este parâmetro otimiza a memória mas não os resultados.


In [ ]:
# Importando as bibliotecas do Modelo
from transformers import BertModel

# Se a variável URL_MODELO1 foi setada
if URL_MODELO:
    # Carregando o Tokenizador
    print('Carregando o modelo BERT do diretório ' + DIRETORIO_MODELO + '...')

    model = BertModel.from_pretrained(DIRETORIO_MODELO, 
                                      output_attentions = True,
                                      output_hidden_states = True)    
else:
    # Carregando o Tokenizador da comunidade
    print('Carregando o modelo BERT da comunidade ...')

    model = BertModel.from_pretrained('neuralmind/bert-large-portuguese-cased', 
                                      output_attentions = True,
                                      output_hidden_states = True)

Carregando o modelo BERT do diretório /content/modelo...


## 5 - Funções auxiliares

### Imports

In [ ]:
# Import das bibliotecas
import numpy as np
import torch

import matplotlib.pyplot as plt
%matplotlib inline

### getEmbeddingsCamadas

Funções que recuperam os embeddings das camadas:
- Primeira camada;
- Penúltima camada;
- Ùltima camada;
- Soma das 4 últimas camadas;
- Concatenação das 4 últimas camadas;
- Soma de todas as camadas.

In [ ]:
def getEmbeddingPrimeiraCamada(output):
  # outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
  # hidden_states é uma lista python, e cada elemento um tensor pytorch no formado <lote> x <qtde_tokens> x <768 ou 1024>.
      
  # Retorna todas a primeira(-1) camada
  # Entrada: List das camadas(13 ou 25) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  resultado = output[2][0]
  # Saída: (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  
  return resultado

def getEmbeddingPenultimaCamada(output):
  # outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
  # hidden_states é uma lista python, e cada elemento um tensor pytorch no formado <lote> x <qtde_tokens> x <768 ou 1024>.
      
  # Retorna todas a primeira(-1) camada
  # Entrada: List das camadas(13 ou 25) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  resultado = output[2][-2]
  # Saída: (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  
  return resultado

def getEmbeddingUltimaCamada(output):
  # outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
  # hidden_states é uma lista python, e cada elemento um tensor pytorch no formado <lote> x <qtde_tokens> x <768 ou 1024>.
     
  # Retorna todas a primeira(-1) camada
  # Entrada: List das camadas(13 ou 25) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  resultado = output[2][-1]
  # Saída: (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  
  return resultado    

def getEmbeddingSoma4UltimasCamadas(output):
  # outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
  # hidden_states é uma lista python, e cada elemento um tensor pytorch no formado <lote> x <qtde_tokens> x <768 ou 1024>.
      
  # Retorna todas a primeira(-1) camada
  # Entrada: List das camadas(13 ou 25) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  embeddingCamadas = output[2][-4:]
  # Saída: List das camadas(4) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  

  # Usa o método `stack` para criar uma nova dimensão no tensor 
  # com a concateção dos tensores dos embeddings.        
  #Entrada: List das camadas(4) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  resultadoStack = torch.stack(embeddingCamadas, dim=0)
  # Saída: <4> x <1(lote)> x <qtde_tokens> x <768 ou 1024>
  
  # Realiza a soma dos embeddings de todos os tokens para as camadas
  # Entrada: <4> x <1(lote)> x <qtde_tokens> x <768 ou 1024>
  resultado = torch.sum(resultadoStack, dim=0)
  # Saida: <1(lote)> x <qtde_tokens> x <768 ou 1024>
  
  return resultado

def getEmbeddingConcat4UltimasCamadas(output):  
  # outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
  # hidden_states é uma lista python, e cada elemento um tensor pytorch no formado <lote> x <qtde_tokens> x <768 ou 1024>.
      
  # Cria uma lista com os tensores a serem concatenados
  # Entrada: List das camadas(13 ou 25) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  # Lista com os tensores a serem concatenados
  listaConcat = []
  # Percorre os 4 últimos
  for i in [-1,-2,-3,-4]:
      # Concatena da lista
      listaConcat.append(output[2][i])
  # Saída: Entrada: List das camadas(4) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  
  # Realiza a concatenação dos embeddings de todos as camadas
  # Saída: Entrada: List das camadas(4) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  resultado = torch.cat(listaConcat, dim=-1)
  # Saída: Entrada: (<1(lote)> x <qtde_tokens> <3072 ou 4096>)  
    
  return resultado   

def getEmbeddingSomaTodasAsCamada(output):
  # outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
  # hidden_states é uma lista python, e cada elemento um tensor pytorch no formado <lote> x <qtde_tokens> x <768 ou 1024>.
   
  # Retorna todas as camadas descontando a primeira(0)
  # Entrada: List das camadas(13 ou 25) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  embeddingCamadas = output[2][1:]
  # Saída: List das camadas(12 ou 24) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  
  # Usa o método `stack` para criar uma nova dimensão no tensor 
  # com a concateção dos tensores dos embeddings.        
  #Entrada: List das camadas(12 ou 24) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  resultadoStack = torch.stack(embeddingCamadas, dim=0)
  # Saída: <12 ou 24> x <1(lote)> x <qtde_tokens> x <768 ou 1024>
    
  # Realiza a soma dos embeddings de todos os tokens para as camadas
  # Entrada: <12 ou 24> x <1(lote)> x <qtde_tokens> x <768 ou 1024>
  resultado = torch.sum(resultadoStack, dim=0)
  # Saida: <1(lote)> x <qtde_tokens> x <768 ou 1024>
    
  return resultado

### getEmbeddingsVisual

Função para gerar as coordenadas de plotagem a partir das sentenças de embeddings.

Existe uma função para os tipos de camadas utilizadas:
- Ùltima camada;
- Soma das 4 últimas camadas;
- Concatenação das 4 últimas camadas;
- Soma de todas as camadas.

In [ ]:
def getEmbeddingsVisualUltimaCamada(texto, modelo, tokenizador):
    
    # Adiciona os tokens especiais
    texto_marcado = "[CLS] " + texto + " [SEP]"

    # Divide a sentença em tokens
    texto_tokenizado = tokenizador.tokenize(texto_marcado)

    # Mapeia as strings dos tokens em seus índices do vocabuário    
    tokens_indexados = tokenizador.convert_tokens_to_ids(texto_tokenizado)
    
    # Marca cada um dos tokens como pertencentes à sentença "1".
    mascara_atencao = [1] * len(texto_tokenizado)

    # Converte a entrada em tensores
    tokens_tensores = torch.as_tensor([tokens_indexados])
    mascara_atencao_tensores = torch.as_tensor([mascara_atencao])
    
    # Prediz os atributos dos estados ocultos para cada camada
    with torch.no_grad():        
        # Retorno de model quando ´output_hidden_states=True´ é setado:  
        #outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
        outputs = modelo(tokens_tensores, mascara_atencao_tensores)

    # Camada embedding    
    camada = getEmbeddingUltimaCamada(outputs)

    # Remove a dimensão 1, o lote "batches".
    token_embeddings = torch.squeeze(camada, dim=0)

    # Recupera os embeddings dos tokens como um vetor
    embeddings = token_embeddings.numpy()

    # Converte para um array
    W = np.array(embeddings)
    # Transforma em um array
    B = np.array([embeddings[0], embeddings[-1]])
    # Invertee B.T
    Bi = np.linalg.pinv(B.T)

    #Projeta a palavra no espaço
    Wp = np.matmul(Bi,W.T)

    return Wp, texto_tokenizado

In [ ]:
def getEmbeddingsVisualSoma4UltimasCamadas(texto, modelo, tokenizador):
    
    # Adiciona os tokens especiais
    texto_marcado = "[CLS] " + texto + " [SEP]"

    # Divide a sentença em tokens
    texto_tokenizado = tokenizador.tokenize(texto_marcado)

    # Mapeia as strings dos tokens em seus índices do vocabuário    
    tokens_indexados = tokenizador.convert_tokens_to_ids(texto_tokenizado)
    
    # Marca cada um dos tokens como pertencentes à sentença "1".
    mascara_atencao = [1] * len(texto_tokenizado)

    # Converte a entrada em tensores
    tokens_tensores = torch.as_tensor([tokens_indexados])
    mascara_atencao_tensores = torch.as_tensor([mascara_atencao])
    
    # Prediz os atributos dos estados ocultos para cada camada
    with torch.no_grad():        
        # Retorno de model quando ´output_hidden_states=True´ é setado:  
        #outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
        outputs = modelo(tokens_tensores, mascara_atencao_tensores)

    # Camada embedding    
    camada = getEmbeddingSoma4UltimasCamadas(outputs)

    # Remove a dimensão 1, o lote "batches".
    token_embeddings = torch.squeeze(camada, dim=0)

    # Recupera os embeddings dos tokens como um vetor
    embeddings = token_embeddings.numpy()

    # Converte para um array
    W = np.array(embeddings)
    # Transforma em um array
    B = np.array([embeddings[0], embeddings[-1]])
    # Invertee B.T
    Bi = np.linalg.pinv(B.T)

    #Projeta a palavra no espaço
    Wp = np.matmul(Bi,W.T)

    return Wp, texto_tokenizado

In [ ]:
def getEmbeddingsVisualConcat4UltimasCamadas(texto, modelo, tokenizador):
    
    # Adiciona os tokens especiais
    texto_marcado = "[CLS] " + texto + " [SEP]"

    # Divide a sentença em tokens
    texto_tokenizado = tokenizador.tokenize(texto_marcado)

    # Mapeia as strings dos tokens em seus índices do vocabuário    
    tokens_indexados = tokenizador.convert_tokens_to_ids(texto_tokenizado)
    
    # Marca cada um dos tokens como pertencentes à sentença "1".
    mascara_atencao = [1] * len(texto_tokenizado)

    # Converte a entrada em tensores
    tokens_tensores = torch.as_tensor([tokens_indexados])
    mascara_atencao_tensores = torch.as_tensor([mascara_atencao])
    
    # Prediz os atributos dos estados ocultos para cada camada
    with torch.no_grad():        
        # Retorno de model quando ´output_hidden_states=True´ é setado:  
        #outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
        outputs = modelo(tokens_tensores, mascara_atencao_tensores)

    # Camada embedding    
    camada = getEmbeddingConcat4UltimasCamadas(outputs)

    # Remove a dimensão 1, o lote "batches".
    token_embeddings = torch.squeeze(camada, dim=0)

    # Recupera os embeddings dos tokens como um vetor
    embeddings = token_embeddings.numpy()

    # Converte para um array
    W = np.array(embeddings)
    # Transforma em um array
    B = np.array([embeddings[0], embeddings[-1]])
    # Invertee B.T
    Bi = np.linalg.pinv(B.T)

    #Projeta a palavra no espaço
    Wp = np.matmul(Bi,W.T)

    return Wp, texto_tokenizado

In [ ]:
def getEmbeddingsVisualSomaTodasAsCamadas(texto, modelo, tokenizador):
    
    # Adiciona os tokens especiais
    texto_marcado = "[CLS] " + texto + " [SEP]"

    # Divide a sentença em tokens
    texto_tokenizado = tokenizador.tokenize(texto_marcado)

    # Mapeia as strings dos tokens em seus índices do vocabuário    
    tokens_indexados = tokenizador.convert_tokens_to_ids(texto_tokenizado)
    
    # Marca cada um dos tokens como pertencentes à sentença "1".
    mascara_atencao = [1] * len(texto_tokenizado)

    # Converte a entrada em tensores
    tokens_tensores = torch.as_tensor([tokens_indexados])
    mascara_atencao_tensores = torch.as_tensor([mascara_atencao])
    
    # Prediz os atributos dos estados ocultos para cada camada
    with torch.no_grad():        
        # Retorno de model quando ´output_hidden_states=True´ é setado:  
        #outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
        outputs = modelo(tokens_tensores, mascara_atencao_tensores)

    # Camada embedding    
    camada = getEmbeddingSomaTodasAsCamada(outputs)

    # Remove a dimensão 1, o lote "batches".
    token_embeddings = torch.squeeze(camada, dim=0)

    # Recupera os embeddings dos tokens como um vetor
    embeddings = token_embeddings.numpy()

    # Converte para um array
    W = np.array(embeddings)
    # Transforma em um array
    B = np.array([embeddings[0], embeddings[-1]])
    # Invertee B.T
    Bi = np.linalg.pinv(B.T)

    #Projeta a palavra no espaço
    Wp = np.matmul(Bi,W.T)

    return Wp, texto_tokenizado

### getDocumentoTokenizado 
Retorna o documento tokenizado

In [ ]:
def getDocumentoTokenizado(documento, tokenizador):

    # Adiciona os tokens especiais.
    documentoMarcado = "[CLS] " + documento + " [SEP]"

    # Documento tokenizado
    documentoTokenizado = tokenizador.tokenize(documentoMarcado)

    return documentoTokenizado

### encontrarIndiceSubLista 

Retorna os índices de início e fim da sublista na lista

In [ ]:
# Localiza os índices de início e fim de uma sublista em uma lista
def encontrarIndiceSubLista(lista, sublista):
    # Recupera o tamanho da lista 
    h = len(lista)
    # Recupera o tamanho da sublista
    n = len(sublista)    
    skip = {sublista[i]: n - i - 1 for i in range(n - 1)}
    i = n - 1
    while i < h:
        for j in range(n):
            if lista[i - j] != sublista[-j - 1]:
                i += skip.get(lista[i], n)
                break
        else:
            indiceInicio = i - n + 1
            indiceFim = indiceInicio + len(sublista)-1
            return indiceInicio, indiceFim
    return -1, -1

### getEmbeddingSentencaDocumentoEmbedding

Retorna os embeddings de uma sentença apartir dos embeddings do documento.

In [ ]:
def getEmbeddingSentencaDocumentoEmbedding(embeddingDocumento, documento, sentenca, tokenizador):
  # Tokeniza o documento
  documentoTokenizado = getDocumentoTokenizado(documento, tokenizador)  
  #print(documentoTokenizado)

  # Tokeniza a sentença
  sentençaTokenizada = getDocumentoTokenizado(sentenca, tokenizador)
  
  # Remove os tokens de início e fim da sentença
  sentençaTokenizada.remove('[CLS]')
  sentençaTokenizada.remove('[SEP]')  
  #print(sentençaTokenizada)
  #print(len(sentençaTokenizada))
  
  # Localiza os índices dos tokens da sentença no documento
  inicio, fim = encontrarIndiceSubLista(documentoTokenizado,sentençaTokenizada)
  #print("Sentença inicia em:", inicio, "até", fim) 
 
  # Recupera os embeddings dos tokens da sentença a partir dos embeddings do documento
  embeddingSentenca = embeddingDocumento[inicio:fim+1]
  #print("embeddingSentenca=", embeddingSentenca.shape)
  
  # Retorna o embedding da sentença no documento
  return embeddingSentenca

## 6 - Exemplo sentenças de documento original e permutado utilizando embedding da última camada do BERT usando a estratégia MEAN

### Documento Original

In [ ]:
# Define um documento com 4 sentenças
documento_original = ["Bom Dia, professor.",
             "Qual o conteúdo da prova?",              
             "Vai cair tudo na prova?",
             "Aguardo uma resposta, João."]

# Concatena as sentenças do documento em uma string
stringDocumentoPermutado = ' '.join(documento_original)

# Adiciona os tokens especiais
documento_marcado_original = "[CLS] " + stringDocumentoPermutado + " [SEP]"

# Divide a sentença em tokens
documento_tokenizado_original = tokenizer.tokenize(documento_marcado_original)

# Mapeia os tokens em seus índices do vocabulário
documento_tokens_indexados_original = tokenizer.convert_tokens_to_ids(documento_tokenizado_original)

# Mostra os tokens com seus índices
i = 0
for tup in zip(documento_tokenizado_original, documento_tokens_indexados_original):
    print('{:>3} {:<12} {:>6,}'.format(i, tup[0], tup[1]))
    i = i + 1

  0 [CLS]           101
  1 Bom           8,399
  2 Dia           3,616
  3 ,               117
  4 professor     2,917
  5 .               119
  6 Qual         13,082
  7 o               146
  8 conteúdo      5,015
  9 da              180
 10 prova         2,310
 11 ?               136
 12 Vai          20,805
 13 cair          9,322
 14 tudo          2,745
 15 na              229
 16 prova         2,310
 17 ?               136
 18 Agu           8,125
 19 ##ardo        2,222
 20 uma             230
 21 resposta      4,299
 22 ,               117
 23 João          1,453
 24 .               119
 25 [SEP]           102


Máscara de atenção das palavras

In [ ]:
# Marca cada um dos tokens como pertencentes à sentença "1".
mascara_atencao_original = [1] * len(documento_tokenizado_original)

print (mascara_atencao_original)
print (len(mascara_atencao_original))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
26


Convertendo as listas em tensores

In [ ]:
# Importa a bibliteca
import torch

# Converte as entradas de listas para tensores do torch
tokens_tensores_original = torch.as_tensor([documento_tokens_indexados_original])
mascara_atencao_tensores_original = torch.as_tensor([mascara_atencao_original])

Gera os embeddings para o documento original. Guarda somente a última camada da rede em `outputs`.


In [ ]:
# Prediz os atributos dos estados ocultos para cada camada
with torch.no_grad():
    # output[0] contém last_hidden_states
    outputs = model(tokens_tensores_original, mascara_atencao_tensores_original)

Recupera a saída da última camada

In [ ]:
# Recupera a última e única camada da saída
last_hidden_states = outputs[0]

print ("O vetor da última camada oculta tem o formato:", last_hidden_states.size())

O vetor da última camada oculta tem o formato: torch.Size([1, 26, 768])


Vamos nos livrar da dimensão lotes "batches", pois não precisamos dela.

In [ ]:
# Remove a dimensão 1, o lote "batches".
#O método squeeze remove a primeira dimensão(0) pois possui tamanho 1
embeddingDocumentoOriginal = torch.squeeze(last_hidden_states, dim=0)

print ("O vetor de tokens de embedding do documento original tem o formato:", embeddingDocumentoOriginal.size())

O vetor de tokens de embedding do documento original tem o formato: torch.Size([26, 768])


Confirmando vetores dependentes do contexto


In [ ]:
for i, token_str in enumerate(documento_tokenizado_original):
  print (i, token_str)

0 [CLS]
1 Bom
2 Dia
3 ,
4 professor
5 .
6 Qual
7 o
8 conteúdo
9 da
10 prova
11 ?
12 Vai
13 cair
14 tudo
15 na
16 prova
17 ?
18 Agu
19 ##ardo
20 uma
21 resposta
22 ,
23 João
24 .
25 [SEP]


Exibe os embenddings das sentenças

In [ ]:
# Índice das sentenças a serem comparadas
sentenca1Original = documento_original[0]
sentenca2Original = documento_original[1]
sentenca3Original = documento_original[2]
sentenca4Original = documento_original[3]

embeddingSentenca1Original = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, sentenca1Original, tokenizer)
embeddingSentenca2Original = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, sentenca2Original, tokenizer)
embeddingSentenca3Original = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, sentenca3Original, tokenizer)
embeddingSentenca4Original = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, sentenca4Original, tokenizer)

print('Os primeiros 4 valores de cada sentença do documento original.')

print('\nSentença 1:', sentenca1Original,'-', str(embeddingSentenca1Original[:4]))
print('Soma embedding Sentença1:', sentenca1Original,'-', str(torch.sum(embeddingSentenca1Original[:4])))

print('\nSentença 2:', sentenca2Original,'-', str(embeddingSentenca2Original[:4]))
print('Soma embedding Sentença2:', sentenca2Original,'-', str(torch.sum(embeddingSentenca2Original[:4])))

print('\nSentença 3:', sentenca3Original,'-', str(embeddingSentenca3Original[:4]))
print('Soma embedding Sentença3:', sentenca3Original,'-', str(torch.sum(embeddingSentenca3Original[:4])))

print('\nSentença 4:', sentenca4Original,'-', str(embeddingSentenca4Original[:4]))
print('Soma embedding Sentença4:', sentenca4Original,'-', str(torch.sum(embeddingSentenca4Original[:4])))

Os primeiros 4 valores de cada sentença do documento original.

Sentença 1: Bom Dia, professor. - tensor([[-0.0680, -0.4615,  0.3552,  ..., -0.3943, -0.1818, -0.4821],
        [-0.1000, -0.0630,  0.0840,  ..., -0.6630,  0.1641, -0.8297],
        [-0.3165,  0.4208,  0.2178,  ..., -0.4981,  0.1935, -0.3366],
        [ 0.1248,  0.2383,  0.8987,  ..., -0.4940, -0.4578, -0.0353]])
Soma embedding Sentença1: Bom Dia, professor. - tensor(-13.8934)

Sentença 2: Qual o conteúdo da prova? - tensor([[-0.5894, -0.4310,  0.1449,  ...,  0.1601, -0.2918, -0.5303],
        [ 0.1349, -0.2476,  0.4605,  ..., -0.3036, -0.6972,  0.2135],
        [ 0.4359, -0.6972,  0.4066,  ...,  0.0177, -0.5852, -0.0615],
        [ 0.0544,  0.1606,  0.4150,  ..., -0.3822, -0.1052, -0.0296]])
Soma embedding Sentença2: Qual o conteúdo da prova? - tensor(-13.9466)

Sentença 3: Vai cair tudo na prova? - tensor([[-0.3652, -0.5015, -0.1626,  ...,  0.0161, -0.5967,  0.1675],
        [ 0.1421,  0.1797, -0.0014,  ..., -0.5730, -0.

Examinando os embeddings do documento original



In [ ]:
# Índice das sentenças a serem comparadas
sentenca1Original = documento_original[0]
sentenca2Original = documento_original[1]
sentenca3Original = documento_original[2]
sentenca4Original = documento_original[3]

print("Documento Original:", documento_original)

# Localiza os índices dos tokens da sentença no documento
sentenca1TokenizadaOriginal = tokenizer.tokenize(sentenca1Original)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_original,sentenca1TokenizadaOriginal)
embeddingSentenca1Original = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, sentenca1Original, tokenizer)
print('\nSentença 1 Original=\'', sentenca1Original, '\'')
print('    Sentença tokenizada:', sentenca1TokenizadaOriginal)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca1Original.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca1Original))

# Localiza os índices dos tokens da sentença no documento
sentenca2TokenizadaOriginal = tokenizer.tokenize(sentenca2Original)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_original,sentenca2TokenizadaOriginal)
embeddingSentenca2Original = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, sentenca2Original, tokenizer)
print('\nSentença 2 Original=\'', sentenca2Original, '\'')
print('    Sentença tokenizada:', sentenca2TokenizadaOriginal)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca2Original.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca2Original))

# Localiza os índices dos tokens da sentença no documento
sentenca3TokenizadaOriginal = tokenizer.tokenize(sentenca3Original)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_original,sentenca3TokenizadaOriginal)
embeddingSentenca3Original = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, sentenca3Original, tokenizer)
print('\nSentença 3 Original=\'', sentenca3Original, '\'')
print('    Sentença tokenizada:', sentenca3TokenizadaOriginal)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca3Original.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca3Original))

# Localiza os índices dos tokens da sentença no documento
sentenca4TokenizadaOriginal = tokenizer.tokenize(sentenca4Original)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_original,sentenca4TokenizadaOriginal)
embeddingSentenca4Original = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, sentenca4Original, tokenizer)
print('\nSentença 4 Original=\'', sentenca4Original, '\'')
print('    Sentença tokenizada:', sentenca4TokenizadaOriginal)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca4Original.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca4Original))


Documento Original: ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']

Sentença 1 Original=' Bom Dia, professor. '
    Sentença tokenizada: ['Bom', 'Dia', ',', 'professor', '.']
    => inicio em 1 e término em 5
    Formato modelo : torch.Size([5, 768])
    Soma embeddings:  -15.02

Sentença 2 Original=' Qual o conteúdo da prova? '
    Sentença tokenizada: ['Qual', 'o', 'conteúdo', 'da', 'prova', '?']
    => inicio em 6 e término em 11
    Formato modelo : torch.Size([6, 768])
    Soma embeddings:  -17.08

Sentença 3 Original=' Vai cair tudo na prova? '
    Sentença tokenizada: ['Vai', 'cair', 'tudo', 'na', 'prova', '?']
    => inicio em 12 e término em 17
    Formato modelo : torch.Size([6, 768])
    Soma embeddings:  -10.65

Sentença 4 Original=' Aguardo uma resposta, João. '
    Sentença tokenizada: ['Agu', '##ardo', 'uma', 'resposta', ',', 'João', '.']
    => inicio em 18 e término em 24
    Formato modelo : torch.Size([7

### Documento Permutado

In [ ]:
# Define um documento com a permutação das sentenças do documento original
documento_permutado = [documento_original[3],   # "Aguardo uma resposta, João.",
             documento_original[1],             # "Qual o conteúdo da prova?",              
             documento_original[0],             # "Vai cair tudo na prova?",
             documento_original[2]]             # "Bom Dia, professor."]     

# Use o documento permutado igual ao original para testar se as medidas estão corretas
#documento_permutado = documento_original

# Concatena as sentenças do documento em uma string
stringDocumentoPermutado = ' '.join(documento_permutado)

# Adiciona os tokens especiais
documento_marcado_permutado = "[CLS] " + stringDocumentoPermutado + " [SEP]"

# Divide a sentença em tokens
documento_tokenizado_permutado = tokenizer.tokenize(documento_marcado_permutado)

# Mapeia os tokens em seus índices do vocabulário
documento_tokens_indexados_permutado = tokenizer.convert_tokens_to_ids(documento_tokenizado_permutado)

# Mostra os tokens com seus índices
i = 0
for tup in zip(documento_tokenizado_permutado, documento_tokens_indexados_permutado):
    print('{:>3} {:<12} {:>6,}'.format(i, tup[0], tup[1]))
    i = i + 1

  0 [CLS]           101
  1 Agu           8,125
  2 ##ardo        2,222
  3 uma             230
  4 resposta      4,299
  5 ,               117
  6 João          1,453
  7 .               119
  8 Qual         13,082
  9 o               146
 10 conteúdo      5,015
 11 da              180
 12 prova         2,310
 13 ?               136
 14 Bom           8,399
 15 Dia           3,616
 16 ,               117
 17 professor     2,917
 18 .               119
 19 Vai          20,805
 20 cair          9,322
 21 tudo          2,745
 22 na              229
 23 prova         2,310
 24 ?               136
 25 [SEP]           102


In [ ]:
# Marca cada um dos tokens como pertencentes à sentença "1".
mascara_atencao_permutado = [1] * len(documento_tokenizado_permutado)

print (mascara_atencao_permutado)
print (len(mascara_atencao_permutado))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
26


Convertendo as listas em tensores

In [ ]:
# Importa a bibliteca
import torch

# Converte as entradas de listas para tensores do torch
tokens_tensores_permutado = torch.as_tensor([documento_tokens_indexados_permutado])
mascara_atencao_tensores_permutado = torch.as_tensor([mascara_atencao_permutado])

Gera os embeddings para o documento original. Guarda somente a última camada da rede em `outputs`.


In [ ]:
# Prediz os atributos dos estados ocultos para cada camada
with torch.no_grad():
    # output[0] contém last_hidden_states
    outputs = model(tokens_tensores_permutado, mascara_atencao_tensores_permutado)

Recupera a saída da última camada

In [ ]:
# Recupera a última e única camada da saída
last_hidden_states = outputs[0]

print ("O vetor da última camada oculta tem o formato:", last_hidden_states.size())

O vetor da última camada oculta tem o formato: torch.Size([1, 26, 768])


Vamos nos livrar da dimensão lotes "batches", pois não precisamos dela.

In [ ]:
# Remove a dimensão 1, o lote "batches".
#O método squeeze remove a primeira dimensão(0) pois possui tamanho 1
embeddingDocumentoPermutado = torch.squeeze(last_hidden_states, dim=0)

print ("O vetor de tokens de embedding do documento permutado tem o formato:", embeddingDocumentoPermutado.size())

O vetor de tokens de embedding do documento permutado tem o formato: torch.Size([26, 768])


Exibe os embenddings das sentenças

In [ ]:
# Índice das sentenças a serem comparadas
sentenca1Permutado = documento_permutado[0]
sentenca2Permutado = documento_permutado[1]
sentenca3Permutado = documento_permutado[2]
sentenca4Permutado = documento_permutado[3]

embeddingSentenca1Permutado = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoPermutado, stringDocumentoPermutado, sentenca1Permutado, tokenizer)
embeddingSentenca2Permutado = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoPermutado, stringDocumentoPermutado, sentenca2Permutado, tokenizer)
embeddingSentenca3Permutado = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoPermutado, stringDocumentoPermutado, sentenca3Permutado, tokenizer)
embeddingSentenca4Permutado = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoPermutado, stringDocumentoPermutado, sentenca4Permutado, tokenizer)

print('Os primeiros 4 valores de cada sentença do documento permutado.')

print('\nSentença 1:', sentenca1Permutado,'-', str(embeddingSentenca1Permutado[:4]))
print('Soma embedding Sentença1:', sentenca1Original,'-', str(torch.sum(embeddingSentenca1Original[:4])))

print('\nSentença 2:', sentenca2Permutado,'-', str(embeddingSentenca2Permutado[:4]))
print('Soma embedding Sentença2:', sentenca2Permutado,'-', str(torch.sum(embeddingSentenca2Permutado[:4])))

print('\nSentença 3:', sentenca3Permutado,'-', str(embeddingSentenca3Permutado[:4]))
print('Soma embedding Sentença3:', sentenca3Permutado,'-', str(torch.sum(embeddingSentenca3Original[:4])))

print('\nSentença 4:', sentenca4Permutado,'-', str(embeddingSentenca4Permutado[:4]))
print('Soma embedding Sentença4:', sentenca4Permutado,'-', str(torch.sum(embeddingSentenca4Permutado[:4])))

Os primeiros 4 valores de cada sentença do documento permutado.

Sentença 1: Aguardo uma resposta, João. - tensor([[-0.0052, -0.4706,  0.6481,  ..., -0.4811,  0.2881, -0.4057],
        [-0.1516, -0.2450,  0.2693,  ..., -0.5534, -0.3164, -0.3582],
        [-0.7358, -0.0988,  0.5145,  ..., -0.1814, -0.3193, -0.3159],
        [-0.1278,  0.3571,  0.5481,  ..., -0.0720, -0.0478, -0.4931]])
Soma embedding Sentença1: Bom Dia, professor. - tensor(-13.8934)

Sentença 2: Qual o conteúdo da prova? - tensor([[-0.4295, -0.3426,  0.1005,  ...,  0.1568, -0.3225, -0.3857],
        [ 0.1047, -0.1946,  0.4177,  ..., -0.3388, -0.7529,  0.1693],
        [ 0.4377, -0.5952,  0.5448,  ...,  0.0463, -0.5607, -0.1783],
        [ 0.0898,  0.1063,  0.4610,  ..., -0.3313, -0.1764,  0.0920]])
Soma embedding Sentença2: Qual o conteúdo da prova? - tensor(-13.8054)

Sentença 3: Bom Dia, professor. - tensor([[-0.0458, -0.4268,  0.2399,  ..., -0.4504, -0.2225, -0.6343],
        [ 0.0650, -0.1385,  0.0230,  ..., -0.7129

In [ ]:
# Índice das sentenças a serem comparadas
sentenca1Permutado = documento_permutado[0]
sentenca2Permutado = documento_permutado[1]
sentenca3Permutado = documento_permutado[2]
sentenca4Permutado = documento_permutado[3]

print("Documento Permutado:", documento_permutado)

# Localiza os índices dos tokens da sentença no documento
sentenca1TokenizadaPermutado = tokenizer.tokenize(sentenca1Permutado)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_permutado,sentenca1TokenizadaPermutado)
embeddingSentenca1Permutado = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoPermutado, stringDocumentoPermutado, sentenca1Permutado, tokenizer)
print('\nSentença 1 Permutada=\'', sentenca1Permutado, '\'')
print('    Sentença tokenizada:', sentenca1TokenizadaPermutado)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca1Permutado.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca1Permutado))

# Localiza os índices dos tokens da sentença no documento
sentenca2TokenizadaPermutado = tokenizer.tokenize(sentenca2Permutado)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_permutado,sentenca2TokenizadaPermutado)
embeddingSentenca2Permutado = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoPermutado, stringDocumentoPermutado, sentenca2Permutado, tokenizer)
print('\nSentença 2 Permutada=\'', sentenca2Permutado, '\'')
print('    Sentença tokenizada:', sentenca2TokenizadaPermutado)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca2Permutado.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca2Permutado))

# Localiza os índices dos tokens da sentença no documento
sentenca3TokenizadaPermutado = tokenizer.tokenize(sentenca3Permutado)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_permutado,sentenca3TokenizadaPermutado)
embeddingSentenca3Permutado = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoPermutado, stringDocumentoPermutado, sentenca3Permutado, tokenizer)
print('\nSentença 3 Permutada=\'', sentenca3Permutado, '\'')
print('    Sentença tokenizada:', sentenca3TokenizadaPermutado)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca3Permutado.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca3Permutado))

# Localiza os índices dos tokens da sentença no documento
sentenca4TokenizadaPermutado = tokenizer.tokenize(sentenca4Permutado)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_permutado,sentenca4TokenizadaPermutado)
embeddingSentenca4Permutado = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoPermutado, stringDocumentoPermutado, sentenca4Permutado, tokenizer)
print('\nSentença 4 Permutada=\'', sentenca4Permutado, '\'')
print('    Sentença tokenizada:', sentenca4TokenizadaPermutado)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca4Permutado.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca4Permutado))


Documento Permutado: ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']

Sentença 1 Permutada=' Aguardo uma resposta, João. '
    Sentença tokenizada: ['Agu', '##ardo', 'uma', 'resposta', ',', 'João', '.']
    => inicio em 1 e término em 7
    Formato modelo : torch.Size([7, 768])
    Soma embeddings:  -14.89

Sentença 2 Permutada=' Qual o conteúdo da prova? '
    Sentença tokenizada: ['Qual', 'o', 'conteúdo', 'da', 'prova', '?']
    => inicio em 8 e término em 13
    Formato modelo : torch.Size([6, 768])
    Soma embeddings:  -17.25

Sentença 3 Permutada=' Bom Dia, professor. '
    Sentença tokenizada: ['Bom', 'Dia', ',', 'professor', '.']
    => inicio em 14 e término em 18
    Formato modelo : torch.Size([5, 768])
    Soma embeddings:  -15.11

Sentença 4 Permutada=' Vai cair tudo na prova? '
    Sentença tokenizada: ['Vai', 'cair', 'tudo', 'na', 'prova', '?']
    => inicio em 19 e término em 24
    Formato modelo : torch.Si

### Examinando as sentenças

A mesma sentença apresenta embeddings com valores diferentes, pois se encontram em locais diferentes do documento. A soma de todos os embeddings demonstra isto.

In [ ]:
print('\nSentença 4 Original=\'', sentenca4Original, '\'')
print('    Sentença tokenizada:', sentenca4TokenizadaOriginal)
print('    Formato modelo :', embeddingSentenca4Original.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca4Original))
print('    Os 4 primeiros embeddings:', str(embeddingSentenca4Original[:4]))

print('\nSentença 1 Permutada=\'', sentenca1Permutado, '\'')
print('    Sentença tokenizada:', sentenca1TokenizadaPermutado)
print('    Formato modelo :', embeddingSentenca1Permutado.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca1Permutado))
print('    Os 4 primeiros embeddings:', str(embeddingSentenca1Permutado[:4]))


Sentença 4 Original=' Aguardo uma resposta, João. '
    Sentença tokenizada: ['Agu', '##ardo', 'uma', 'resposta', ',', 'João', '.']
    Formato modelo : torch.Size([7, 768])
    Soma embeddings:  -12.13
    Os 4 primeiros embeddings: tensor([[-0.0835, -0.4042,  0.4330,  ..., -0.3271,  0.2262, -0.4599],
        [-0.2782, -0.1630,  0.0997,  ..., -0.4231, -0.3089, -0.5329],
        [-0.5474, -0.0118,  0.3950,  ..., -0.0949, -0.3534, -0.3717],
        [ 0.0146,  0.2901,  0.3920,  ...,  0.1161,  0.0407, -0.6904]])

Sentença 1 Permutada=' Aguardo uma resposta, João. '
    Sentença tokenizada: ['Agu', '##ardo', 'uma', 'resposta', ',', 'João', '.']
    Formato modelo : torch.Size([7, 768])
    Soma embeddings:  -14.89
    Os 4 primeiros embeddings: tensor([[-0.0052, -0.4706,  0.6481,  ..., -0.4811,  0.2881, -0.4057],
        [-0.1516, -0.2450,  0.2693,  ..., -0.5534, -0.3164, -0.3582],
        [-0.7358, -0.0988,  0.5145,  ..., -0.1814, -0.3193, -0.3159],
        [-0.1278,  0.3571,  0.5481,  .

### Subtração entre os embeddings das sentenças

#### Calcula a média aritmética da subtração entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento original. 



In [ ]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaSsub = 0

# Percorre as sentenças do documento
for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSi=", mediaEmbeddingSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSj=", mediaEmbeddingSj.shape)
  
    # Subtração entre os embeddings de Si e Sj
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)  
    Ssub = torch.sub(mediaEmbeddingSi, mediaEmbeddingSj)
    # Saída: <768 ou 1024>  
    #print("Ssub=", Ssub.shape)

    # Calcula a média dos embeddings do subtração para realizar a soma
    somaSsub = somaSsub + torch.mean(Ssub)

DsubOriginal = float(somaSsub)/float(len(documento_original)-1)
print("Ssub Original:", DsubOriginal)
    


Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Ssub Original: 0.0007491791620850563


#### Calcula a média aritmética da subtração entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento permutado. 

In [ ]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaSsub = 0

# Percorre as sentenças do documento
for i in range(np-1):
    # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSi=", mediaEmbeddingSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSj=", mediaEmbeddingSj.shape)
  
    # Subtração entre os embeddings de Si e Sj
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Ssub = torch.sub(mediaEmbeddingSi, mediaEmbeddingSj)
    # Saída: <768 ou 1024>  
    #print("Ssub=", Ssub.shape)

    # Calcula a média dos embeddings do subtração para realizar a soma
    somaSsub = somaSsub + torch.mean(Ssub)

DsubPermutado = float(somaSsub)/float(np-1)
print("Ssub permutado:", DsubPermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']
Quantidade de sentenças: 4
Ssub permutado: -0.0006407623489697775


#### Compara as médias da subtração dos embeddings das sentenças do documento original e permutado.

Características das medidas:
- Documentos com sentenças iguais resulta uma medida igual a 0.
- Documentos com sentenças diferenntes resulta uma medida maior que 0.
- Documento com sentenças muito diferentes apresentam valores maiores que 0.
- Documentos iguais resultam em medidas iguais. 
- É uma medida de diferença.


In [ ]:
print("Dsub Original :", DsubOriginal)
print("Dsub Permutado:", DsubPermutado)

if (DsubOriginal < DsubPermutado):
    print("Documento original tem menor subtração entre as sentenças!")
else:
    print("Documento Permutado tem menor subtração entre as sentenças!")

Dsub Original : 0.0007491791620850563
Dsub Permutado: -0.0006407623489697775
Documento Permutado tem menor subtração entre as sentenças!


### Subtração absoluta entre os embeddings das sentenças

#### Calcula a média aritmética da subtração absoluta entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento original. 

In [ ]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaSsubabs = 0

# Percorre as sentenças do documento
for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSi=", mediaEmbeddingSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSj=", mediaEmbeddingSj.shape)
  
    # Subtração entre os embeddings de Si e Sj
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Ssubabs = abs(torch.sub(mediaEmbeddingSi, mediaEmbeddingSj))
    # Saída: <768 ou 1024>  
    #print("Ssubabs=", Ssubabs.shape)

    # Calcula a média dos embeddings do subtração para realizar a soma
    somaSsubabs = somaSsubabs + torch.mean(Ssubabs)

DsubabsOriginal = float(somaSsubabs)/float(n-1)
print("Dsubabs Original:", DsubabsOriginal)
    


Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Dsubabs Original: 0.17778096596399942


#### Calcula a média aritmética da subtração absoluta entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento permutado. 

In [ ]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaSsubabs = 0

# Percorre as sentenças do documento
for i in range(np-1):
    # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSi=", mediaEmbeddingSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSj=", mediaEmbeddingSj.shape)
  
    # Subtração entre os embeddings de Si e Sj
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Ssubabs = abs(torch.sub(mediaEmbeddingSi, mediaEmbeddingSj))
    # Saída: <768 ou 1024>  
    #print("Ssubabs=", Ssubabs.shape)

    # Calcula a média dos embeddings do subtração para realizar a soma
    somaSsubabs = somaSsubabs + torch.mean(Ssubabs)

DsubabsPermutado = float(somaSsubabs)/float(np-1)
print("Dsubabs permutado:", DsubabsPermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']
Quantidade de sentenças: 4
Dsubabs permutado: 0.17206366856892905


#### Compara as médias da subtração absoluta dos embeddings das sentenças do documento original e permutado

Características das medidas:
- Documentos com sentenças iguais resulta uma medida igual a 0.
- Documentos com sentenças diferenntes resulta uma medida maior que 0.
- Documento com sentenças muito diferentes apresentam valores maiores que 0.
- Documentos iguais resultam em medidas iguais. 
- É uma medida de diferença.

In [ ]:
print("Dsubabs Original :", DsubabsOriginal)
print("Dsubabs Permutado:", DsubabsPermutado)

if (DsubabsOriginal < DsubabsPermutado):
    print("Documento original tem menor subtração absoluta entre as sentenças!")
else:
    print("Documento Permutado tem menor subtração absoluta entre as sentenças!")

Dsubabs Original : 0.17778096596399942
Dsubabs Permutado: 0.17206366856892905
Documento Permutado tem menor subtração absoluta entre as sentenças!


### Produto entre os embeddings das sentenças

#### Calcula a média aritmética do produto das matrizes dos tokens das sentenças do documento original. 

In [ ]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaSprod = 0

# Percorre as sentenças do documento
for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Para a multiplicação pode ocorrer número de colunas(qtde_tokens) de Si tem que ser igual ao número de linhas(embeddings) de Sj.
    # Permute realiza a troca da dimensão 1(qtde_embeddings) pela 0(qtde_tokesn) na sentença Sj para que seja possível realizar o produto. 
    # Ocorre a troca da qtde_tokens pela quantidade de embeddings.
    # Entrada: (<qtde_tokensSi> x <768 ou 1024>) x (<qtde_tokensSj> x <768 ou 1024>)
    # Permute: <qtde_tokensSi> x <768 ou 1024>) x (<768 ou 1024> x <qtde_tokensSj>)
    Sprod = torch.matmul(embeddingSi, embeddingSj.permute(1,0))    
    # Saída: <qtde_tokensSi> x  <qtde_tokensSj    #print("Sprod=", Sprod.shape)
    
    # Calcula a média dos embeddings do produto para realizar a soma
    somaSprod = somaSprod + torch.mean(Sprod)

DprodOriginal = float(somaSprod)/float(n-1)
print("Dprod Original:", DprodOriginal)
    


Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Dprod Original: 43.7073974609375


#### Calcula a média aritmética do produto das matrizes dos tokens das sentenças do documento permutado. 

In [ ]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaSprod = 0

# Percorre as sentenças do documento
for i in range(np-1):
    # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)
 
    # Produto entre os embeddings de Si e Sj    
    # Para a multiplicação pode ocorrer número de colunas(qtde_tokens) de Si tem que ser igual ao número de linhas(embeddings) de Sj.
    # Permute realiza a troca da dimensão 1(qtde_embeddings) pela 0(qtde_tokesn) na sentença Sj para que seja possível realizar o produto. 
    # Ocorre a troca da qtde_tokens pela quantidade de embeddings.
    # Entrada: (<qtde_tokensSi> x <768 ou 1024>) x (<qtde_tokensSj> x <768 ou 1024>)
    # Permute: <qtde_tokensSi> x <768 ou 1024>) x (<768 ou 1024> x <qtde_tokensSj>)
    Sprod = torch.matmul(embeddingSi, embeddingSj.permute(1,0))    
    # Saída: <qtde_tokensSi> x  <qtde_tokensSj>
    #print("Sprod=", Sprod.shape)
        
    # Calcula a média dos embeddings do produto para realizar a soma
    somaSprod = somaSprod + torch.mean(Sprod)

DprodPermutado = float(somaSprod)/float(np-1)
print("Dprod permutado:", DprodPermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']
Quantidade de sentenças: 4
Dprod permutado: 43.55047607421875


#### Compara as médias do produto dos embeddings das sentenças do documento original e permutado



In [ ]:
print("Dprod Original :", DprodOriginal)
print("Dprod Permutado:", DprodPermutado)

if (DprodOriginal < DprodPermutado):
    print("Documento original tem maior similaridade com o produto entre as sentenças!")
else:
    print("Documento Permutado tem menor similaridade com o produto entre as sentenças!")

Dprod Original : 43.7073974609375
Dprod Permutado: 43.55047607421875
Documento Permutado tem menor similaridade com o produto entre as sentenças!


### Produto Absoluto entre os embeddings das sentenças

#### Calcula a média aritmética do produto das matrizes entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento original. 

In [ ]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaSprodabs = 0

# Percorre as sentenças do documento
for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)
    
    # Produto entre os embeddings de Si e Sj
    # Entrada: <qtde_tokens> x <768 ou 1024>
    # Para a multiplicação pode ocorrer número de colunas(qtde_tokens) de Si tem que ser igual ao número de linhas(embeddings) de Sj.
    # Permute realiza a troca da dimensão 1(qtde_embeddings) pela 0(qtde_tokesn) na sentença Sj para que seja possível realizar o produto. 
    # Ocorre a troca da qtde_tokens pela quantidade de embeddings.
    # Entrada: (<qtde_tokensSi> x <768 ou 1024>) x (<qtde_tokensSj> x <768 ou 1024>)
    # Permute: <qtde_tokensSi> x <768 ou 1024>) x (<768 ou 1024> x <qtde_tokensSj>)
    Sprodabs = abs(torch.matmul(embeddingSi, embeddingSj.permute(1,0)))   
    # Saída: <qtde_tokensSi> x  <qtde_tokensSj>
    #print("Sprodabs=", Sprodabs.shape)

    # Calcula a média dos embeddings do produto para realizar a soma
    somaSprodabs = somaSprodabs + torch.mean(Sprodabs)

DprodabsOriginal = float(somaSprodabs)/float(n-1)
print("Dprodabs Original:", DprodabsOriginal)

Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Dprodabs Original: 43.7073974609375


#### Calcula a média aritmética do produto das matrizes entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento permutado. 

In [ ]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaSprodabs = 0

# Percorre as sentenças do documento
for i in range(np-1):
    # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)
 
    # Produto entre os embeddings de Si e Sj
    # Entrada: <qtde_tokens> x <768 ou 1024>
    # Para a multiplicação pode ocorrer número de colunas(qtde_tokens) de Si tem que ser igual ao número de linhas(embeddings) de Sj.
    # Permute realiza a troca da dimensão 1(qtde_embeddings) pela 0(qtde_tokesn) na sentença Sj para que seja possível realizar o produto. 
    # Ocorre a troca da qtde_tokens pela quantidade de embeddings.
    # Entrada: (<qtde_tokensSi> x <768 ou 1024>) x (<qtde_tokensSj> x <768 ou 1024>)
    # Permute: <qtde_tokensSi> x <768 ou 1024>) x (<768 ou 1024> x <qtde_tokensSj>)
    Sprodabs = abs(torch.matmul(embeddingSi, embeddingSj.permute(1,0)))    
    # Saída: <qtde_tokensSi> x  <qtde_tokensSj>
    #print("Sprodabs=", Sprodabs.shape)

    # Calcula a média dos embeddings do produto para realizar a soma
    somaSprodabs = somaSprodabs + torch.mean(Sprodabs)

DprodabsPermutado = float(somaSprodabs)/float(np-1)
print("Dprodabs permutado:", DprodabsPermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']
Quantidade de sentenças: 4
Dprodabs permutado: 43.55047607421875


#### Compara as médias do produto absoluto dos embeddings das sentenças do documento original e permutado


In [ ]:
print("Dprodabs Original :", DprodabsOriginal)
print("Dprodabs Permutado:", DprodabsPermutado)

if (DprodabsOriginal < DprodabsPermutado):
    print("Documento original tem maior similaridade com o produto absoluto entre as sentenças!")
else:
    print("Documento Permutado tem menor similaridade com o produto absoluto entre as sentenças!")

Dprodabs Original : 43.7073974609375
Dprodabs Permutado: 43.55047607421875
Documento Permutado tem menor similaridade com o produto absoluto entre as sentenças!


### Produto Escalar entre os embeddings das sentenças

#### Calcula a média aritmética do produto escalar entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento orginal. 



In [ ]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaSprode = 0

# Percorre as sentenças do documento
for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSi=", mediaEmbeddingSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSj=", mediaEmbeddingSj.shape)

    # Produto escalar entre os embeddings de Si e Sj, pois Si e Sj possui uma única dimensão
    # https://pytorch.org/docs/master/generated/torch.matmul.html#torch.matmul
    # Entrada: <768 ou 1024> * <768 ou 1024>
    Sprode = torch.matmul(mediaEmbeddingSi, mediaEmbeddingSj)
    # Saída: Um número real 
    #print("Sprode=", Sprode.shape)
        
    somaSprode = somaSprode + Sprode

DprodeOriginal = float(somaSprode)/float(n-1)
print("Dprode Original:", DprodeOriginal)

Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Dprode Original: 43.7073974609375


#### Calcula a média aritmética do produto escalar entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento permutado. 

In [ ]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaSprode = 0

# Percorre as sentenças do documento
for i in range(np-1):
    # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSi=", mediaEmbeddingSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSj=", mediaEmbeddingSj.shape)
  
    # Produto escalar entre os embeddings de Si e Sj, pois Si e Sj possui uma única dimensão
    # https://pytorch.org/docs/master/generated/torch.matmul.html#torch.matmul
    # Entrada: <768 ou 1024> * <768 ou 1024>
    Sprode = torch.matmul(mediaEmbeddingSi, mediaEmbeddingSj)    
    # Saída: Um número real 
    #print("Sprode=", Sprode)
        
    somaSprode = somaSprode + Sprode

DprodePermutado = float(somaSprode)/float(np-1)
print("Dprod permutado:", DprodePermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']
Quantidade de sentenças: 4
Dprod permutado: 43.55047607421875


#### Compara as médias do produto escalar dos embeddings das sentenças do documento original e permutado

Dprod Original : 230.61181640625
Dprod Permutado: 227.75482177734375
Documento original tem maior similaridade com o produto entre as sentenças!

In [ ]:
print("Dprode Original :", DprodeOriginal)
print("Dprode Permutado:", DprodePermutado)

if (DprodeOriginal < DprodePermutado):
    print("Documento original tem maior similaridade com o produto entre as sentenças!")
else:
    print("Documento Permutado tem menor similaridade com o produto entre as sentenças!")

Dprode Original : 43.7073974609375
Dprode Permutado: 43.55047607421875
Documento Permutado tem menor similaridade com o produto entre as sentenças!


### Produto Escalar Absoluto entre os embeddings das sentenças

#### Calcula a média aritmética do produto escalar absoluto entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento original. 

In [ ]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaSprodeabs = 0

# Percorre as sentenças do documento
for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSi=", mediaEmbeddingSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSj=", mediaEmbeddingSj.shape)
  
    # Produto escalar entre os embeddings de Si e Sj, pois Si e Sj possui uma única dimensão
    # https://pytorch.org/docs/master/generated/torch.matmul.html#torch.matmul
    # Entrada: <768 ou 1024>
    Sprodeabs = torch.matmul(mediaEmbeddingSi, mediaEmbeddingSj)
    # Saída: Um número real 
    #print("Sprodeabs=", Sprodeabs.shape)

    somaSprodeabs = somaSprodeabs + Sprodeabs

DprodeabsOriginal = float(somaSprodeabs)/float(n-1)
print("Dprodabs Original:", DprodeabsOriginal)

Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Dprodabs Original: 43.7073974609375


#### Calcula a média aritmética do produto escalar absoluto entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento permutado. 

In [ ]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaSprodeabs = 0

# Percorre as sentenças do documento
for i in range(np-1):
    # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)
    
    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSi=", mediaEmbeddingSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSj=", mediaEmbeddingSj.shape)
    
    # Produto escalar entre os embeddings de Si e Sj, pois Si e Sj possui uma única dimensão
    # https://pytorch.org/docs/master/generated/torch.matmul.html#torch.matmul
    # Entrada: <768 ou 1024>
    Sprodeabs = torch.matmul(mediaEmbeddingSi, mediaEmbeddingSj)
    # Saída: Um número real 
    #print("Sprodeabs=", Sprodeabs.shape)
    
    somaSprodeabs = somaSprodeabs + Sprodeabs

DprodeabsPermutado = float(somaSprodeabs)/float(np-1)
print("Dprodabs permutado:", DprodeabsPermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']
Quantidade de sentenças: 4
Dprodabs permutado: 43.55047607421875


#### Compara as médias do produto escalar absoluto dos embeddings das sentenças do documento original e permutado

In [ ]:
print("Dprodabs Original :", DprodeabsOriginal)
print("Dprodabs Permutado:", DprodeabsPermutado)

if (DprodeabsOriginal < DprodeabsPermutado):
    print("Documento original tem maior similaridade com o produto absoluto entre as sentenças!")
else:
    print("Documento Permutado tem menor similaridade com o produto absoluto entre as sentenças!")

Dprodabs Original : 43.7073974609375
Dprodabs Permutado: 43.55047607421875
Documento Permutado tem menor similaridade com o produto absoluto entre as sentenças!


### Média entre os embeddings das sentenças

#### Calcula a média aritmética entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento original. 

In [ ]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaSavg = 0

# Percorre as sentenças do documento
for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSi=", mediaEmbeddingSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSj=", mediaEmbeddingSj.shape)
  
    # Média entre os embeddings de Si e Sj
    # Entrada: <768 ou 1024>  
    Savg = (mediaEmbeddingSi.add(mediaEmbeddingSj))/2.0  
    # Saída: <768 ou 1024>
    #print("Savg=", Savg.shape)  

    # Calcula a média dos embeddings do cálculo para realizar a soma
    somaSavg = somaSavg + torch.mean(Savg)

DavgOriginal = float(somaSavg)/float(n-1)
print("Davg Original:", DavgOriginal)

Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Davg Original: -0.002767913043498993


#### Calcula a média aritmética entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento permutado.

In [ ]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaSavg = 0

# Percorre as sentenças do documento
for i in range(np-1):
    # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSi=", mediaEmbeddingSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSj=", mediaEmbeddingSj.shape)
  
    # Média entre os embeddings de Si e Sj
    # Entrada: <768 ou 1024>  
    Savg = (mediaEmbeddingSi.add(mediaEmbeddingSj))/2.0  
    # Saída: <768 ou 1024>
    #print("Savg=", Savg.shape)  

    # Calcula a média dos embeddings do cálculo para realizar a soma
    somaSavg = somaSavg + torch.mean(Savg)

DavgPermutado = float(somaSavg)/float(np-1)
print("Davg permutado:", DavgPermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']
Quantidade de sentenças: 4
Davg permutado: -0.002713705413043499


#### Compara as médias dos embeddings das sentenças do documento original e permutado


In [ ]:
print("Davg Original :", DavgOriginal)
print("Davg Permutado:", DavgPermutado)

if (DavgOriginal < DavgPermutado):
    print("Documento original tem maior similaridade com a média entre as sentenças!")
else:
    print("Documento Permutado tem menor similaridade com a mpedia entre as sentenças!")

Davg Original : -0.002767913043498993
Davg Permutado: -0.002713705413043499
Documento original tem maior similaridade com a média entre as sentenças!


### Similaridade de cosseno entre os embeddings das sentenças

In [ ]:
# Import das bibliotecas.
from scipy.spatial.distance import cosine

def similaridadeCoseno(sentenca1, sentenca2):
  similaridade = 1 - cosine(sentenca1, sentenca2)
  return similaridade

#### Calcula a média aritmética da similaridade do coseno entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento original. 

In [ ]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaScos = 0

# Percorre as sentenças do documento
for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSi=", mediaEmbeddingSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSj=", mediaEmbeddingSj.shape)
  
    # Similaridade entre os embeddings Si e Sj
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Scos = similaridadeCoseno(mediaEmbeddingSi, mediaEmbeddingSj)
    # Saída: Um número real
    
    # Acumula a medida
    somaScos = somaScos + Scos

DcosOriginal = float(somaScos)/float(n-1)
print("Dcos Original:", DcosOriginal)  


Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Dcos Original: 0.6799910068511963


#### Calcula a média aritmética da similaridade do coseno entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento permutado. 

In [ ]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaScos = 0

# Percorre as sentenças do documento
for i in range(np-1):
    # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

   # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSi=", mediaEmbeddingSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSj=", mediaEmbeddingSj.shape)
  
   # Similaridade entre os embeddings Si e Sj
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Scos = similaridadeCoseno(mediaEmbeddingSi, mediaEmbeddingSj)
    # Saída: Um número real
    
    # Acumula a medida
    somaScos = somaScos + Scos

DcosPermutado = float(somaScos)/float(np-1)
print("Dcos Original:", DcosPermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']
Quantidade de sentenças: 4
Dcos Original: 0.696538249651591


#### Compara as médias da similaridade de cosseno dos embeddings das sentenças do documento original e permutado

Características das medidas:
- Documentos com sentenças iguais resulta uma medida igual a 1.
- Documentos com sentenças diferenntes resulta uma medida menor que 1.
- Documento com sentenças muito diferentes apresentam valores menores que 1.
- Documentos iguais resultam em medidas iguais. 
- É uma medida de similaridade.



In [ ]:
print("Dcos Original :", DcosOriginal)
print("Dcos Permutado:", DcosPermutado)

if (DcosOriginal > DcosPermutado):
    print("Documento original tem maior similaridade de cosseno entre as sentenças!")
else:
    print("Documento Permutado tem menor similaridade de cosseno entre as sentenças!")

Dcos Original : 0.6799910068511963
Dcos Permutado: 0.696538249651591
Documento Permutado tem menor similaridade de cosseno entre as sentenças!


### Distância euclidiana entre os embeddings das sentenças

Possui outros nomes como distância L2 ou norma L2.

In [ ]:
# Import das bibliotecas.
from scipy.spatial.distance import euclidean

def distanciaEuclidiana(sentenca1, sentenca2):
  distancia = euclidean(sentenca1, sentenca2)

  return distancia

#### Calcula a média aritmética da distância euclidiana entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento original. 

In [ ]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaSeuc = 0

# Percorre as sentenças do documento
for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSi=", mediaEmbeddingSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    # print("mediaEmbeddingSj=", mediaEmbeddingSj.shape)
  
    # Diferença entre os embeddings Si e Sj
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Seuc = distanciaEuclidiana(mediaEmbeddingSi, mediaEmbeddingSj)
    # Saída: Um número real
    
    # Acumula a medida
    somaSeuc = somaSeuc + Seuc

DeucOriginal = float(somaSeuc)/float(n-1)
print("Deuc Original:", DeucOriginal)  


Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Deuc Original: 6.340852737426758


#### Calcula a média aritmética da distância euclidiana entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento permutado. 

In [ ]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaSeuc = 0

# Percorre as sentenças do documento
for i in range(np-1):
    # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSi=", mediaEmbeddingSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSj=", mediaEmbeddingSj.shape)
  
    # Diferença entre os embeddings Si e Sj
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Seuc = distanciaEuclidiana(mediaEmbeddingSi, mediaEmbeddingSj)
    # Saída: Um número real
    
    # Acumula a medida
    somaSeuc = somaSeuc + Seuc

DeucPermutado = float(somaSeuc)/float(np-1)
print("Deuc Original:", DeucPermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']
Quantidade de sentenças: 4
Deuc Original: 6.154866059621175


#### Compara as médias da distância euclidiana dos embeddings das sentenças do documento original e permutado

Características das medidas:
- Documentos com sentenças iguais resulta uma medida igual a 0.
- Documentos com sentenças diferenntes resulta uma medida maior que 0.
- Documento com sentenças muito diferentes apresentam valores maiores que 0.
- Documentos iguais resultam em medidas iguais. 
- É uma medida de diferença.



In [ ]:
print("Deuc Original :", DeucOriginal)
print("Deuc Permutado:", DeucPermutado)

if (DeucOriginal > DeucPermutado):
    print("Documento original tem maior distância euclidiana entre as sentenças!")
else:
    print("Documento Permutado tem menor distância euclidiana entre as sentenças!")

Deuc Original : 6.340852737426758
Deuc Permutado: 6.154866059621175
Documento original tem maior distância euclidiana entre as sentenças!


### Distância Manhattan entre os embeddings das sentenças

Possui outros nomes como distância Cityblock, distância L1, norma L1 e métrica do táxi.

In [ ]:
# Import das bibliotecas.
from scipy.spatial.distance import cityblock

def distanciaEManhattan(sentenca1, sentenca2):
  distancia = cityblock(sentenca1, sentenca2)

  return distancia

#### Calcula a média aritmética da distância de manhattan entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento original. 

In [ ]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaSman = 0

# Percorre as sentenças do documento
for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSi=", mediaEmbeddingSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    # print("mediaEmbeddingSj=", mediaEmbeddingSj.shape)
  
    # Diferença entre os embeddings Si e Sj
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Sman = distanciaEManhattan(mediaEmbeddingSi, mediaEmbeddingSj)
    # Saída: Um número real
    
    # Acumula a medida
    somaSman = somaSman + Sman

DmanOriginal = float(somaSman)/float(n-1)
print("Dman Original:", DmanOriginal)  


Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Dman Original: 136.53579711914062


#### Calcula a média aritmética da distância de manhattan entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento permutado.

In [ ]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaSman = 0

# Percorre as sentenças do documento
for i in range(np-1):
    # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSi=", mediaEmbeddingSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSj=", mediaEmbeddingSj.shape)
  
    # Diferença entre os embeddings Si e Sj
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Sman = distanciaEManhattan(mediaEmbeddingSi, mediaEmbeddingSj)
    # Saída: Um número real
    
    # Acumula a medida
    somaSman = somaSman + Sman

DmanPermutado = float(somaSman)/float(np-1)
print("Deuc Original:", DmanPermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']
Quantidade de sentenças: 4
Deuc Original: 132.14489237467447


#### Compara as médias da distância de manhattan dos embeddings das sentenças do documento original e permutado

Características das medidas:
- Documentos com sentenças iguais resulta uma medida igual a 0.
- Documentos com sentenças diferenntes resulta uma medida maior que 0.
- Documento com sentenças muito diferentes apresentam valores maiores que 0.
- Documentos iguais resultam em medidas iguais. 
- É uma medida de diferença.



In [ ]:
print("Dman Original :", DmanOriginal)
print("Dman Permutado:", DmanPermutado)

if (DeucOriginal > DeucPermutado):
    print("Documento original tem maior distância de manhattan entre as sentenças!")
else:
    print("Documento Permutado tem menor distância de manhattan entre as sentenças!")

Dman Original : 136.53579711914062
Dman Permutado: 132.14489237467447
Documento original tem maior distância de manhattan entre as sentenças!


### Resumo

Resultado das medidas utilizando a última camada do BERT.

Base(MEAN):
- Dsub       :   0.00074918          -0.00064076
- Dubabs     :   0.17778097          0.17206367
- Dprod      :   43.70739746          43.55047607
- Dprodabs   :   43.70739746          43.55047607
- Dprode     :   43.70739746          43.55047607
- Dprodeabs  :   43.70739746          43.55047607
- Davg       :   -0.00276791          -0.00271371
- Dcos       :   0.67999101          0.69653825
- Deuc       :   6.34085274          6.15486606
- Dman       :   136.53579712          132.14489237


In [ ]:
print("Resultado das medidas utilizando a última camada do BERT")
print("Documento  :   Original            Permutado")
print('Dsub       :   {:.8f}          {:.8f}'.format(DsubOriginal,DsubPermutado))
print('Dubabs     :   {:.8f}          {:.8f}'.format(DsubabsOriginal,DsubabsPermutado))
print('Dprod      :   {:.8f}          {:.8f}'.format(DprodOriginal,DprodPermutado))
print('Dprodabs   :   {:.8f}          {:.8f}'.format(DprodabsOriginal,DprodabsPermutado))
print('Dprode     :   {:.8f}          {:.8f}'.format(DprodeOriginal,DprodePermutado))
print('Dprodeabs  :   {:.8f}          {:.8f}'.format(DprodeabsOriginal,DprodeabsPermutado))
print('Davg       :   {:.8f}          {:.8f}'.format(DavgOriginal,DavgPermutado))
print('Dcos       :   {:.8f}          {:.8f}'.format(DcosOriginal,DcosPermutado))
print('Deuc       :   {:.8f}          {:.8f}'.format(DeucOriginal,DeucPermutado))
print('Dman       :   {:.8f}          {:.8f}'.format(DmanOriginal,DmanPermutado))

Resultado das medidas utilizando a última camada do BERT
Documento  :   Original            Permutado
Dsub       :   0.00074918          -0.00064076
Dubabs     :   0.17778097          0.17206367
Dprod      :   43.70739746          43.55047607
Dprodabs   :   43.70739746          43.55047607
Dprode     :   43.70739746          43.55047607
Dprodeabs  :   43.70739746          43.55047607
Davg       :   -0.00276791          -0.00271371
Dcos       :   0.67999101          0.69653825
Deuc       :   6.34085274          6.15486606
Dman       :   136.53579712          132.14489237


## 7 - Exemplo sentenças de documento original e permutado utilizando embedding da última camada do BERT usando estratégia MAX

### Documento Original

In [ ]:
# Define um documento com 4 sentenças
documento_original = ["Bom Dia, professor.",
             "Qual o conteúdo da prova?",              
             "Vai cair tudo na prova?",
             "Aguardo uma resposta, João."]

# Concatena as sentenças do documento em uma string
stringDocumentoPermutado = ' '.join(documento_original)

# Adiciona os tokens especiais
documento_marcado_original = "[CLS] " + stringDocumentoPermutado + " [SEP]"

# Divide a sentença em tokens
documento_tokenizado_original = tokenizer.tokenize(documento_marcado_original)

# Mapeia os tokens em seus índices do vocabulário
documento_tokens_indexados_original = tokenizer.convert_tokens_to_ids(documento_tokenizado_original)

# Mostra os tokens com seus índices
i = 0
for tup in zip(documento_tokenizado_original, documento_tokens_indexados_original):
    print('{:>3} {:<12} {:>6,}'.format(i, tup[0], tup[1]))
    i = i + 1

  0 [CLS]           101
  1 Bom           8,399
  2 Dia           3,616
  3 ,               117
  4 professor     2,917
  5 .               119
  6 Qual         13,082
  7 o               146
  8 conteúdo      5,015
  9 da              180
 10 prova         2,310
 11 ?               136
 12 Vai          20,805
 13 cair          9,322
 14 tudo          2,745
 15 na              229
 16 prova         2,310
 17 ?               136
 18 Agu           8,125
 19 ##ardo        2,222
 20 uma             230
 21 resposta      4,299
 22 ,               117
 23 João          1,453
 24 .               119
 25 [SEP]           102


Máscara de atenção das palavras

In [ ]:
# Marca cada um dos tokens como pertencentes à sentença "1".
mascara_atencao_original = [1] * len(documento_tokenizado_original)

print (mascara_atencao_original)
print (len(mascara_atencao_original))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
26


Convertendo as listas em tensores

In [ ]:
# Importa a bibliteca
import torch

# Converte as entradas de listas para tensores do torch
tokens_tensores_original = torch.as_tensor([documento_tokens_indexados_original])
mascara_atencao_tensores_original = torch.as_tensor([mascara_atencao_original])

Gera os embeddings para o documento original. Guarda somente a última camada da rede em `outputs`.


In [ ]:
# Prediz os atributos dos estados ocultos para cada camada
with torch.no_grad():
    # output[0] contém last_hidden_states
    outputs = model(tokens_tensores_original, mascara_atencao_tensores_original)

Recupera a saída da última camada

In [ ]:
# Recupera a última e única camada da saída
last_hidden_states = outputs[0]

print ("O vetor da última camada oculta tem o formato:", last_hidden_states.size())

O vetor da última camada oculta tem o formato: torch.Size([1, 26, 768])


Vamos nos livrar da dimensão lotes "batches", pois não precisamos dela.

In [ ]:
# Remove a dimensão 1, o lote "batches".
#O método squeeze remove a primeira dimensão(0) pois possui tamanho 1
embeddingDocumentoOriginal = torch.squeeze(last_hidden_states, dim=0)

print ("O vetor de tokens de embedding do documento original tem o formato:", embeddingDocumentoOriginal.size())

O vetor de tokens de embedding do documento original tem o formato: torch.Size([26, 768])


Confirmando vetores dependentes do contexto


In [ ]:
for i, token_str in enumerate(documento_tokenizado_original):
  print (i, token_str)

0 [CLS]
1 Bom
2 Dia
3 ,
4 professor
5 .
6 Qual
7 o
8 conteúdo
9 da
10 prova
11 ?
12 Vai
13 cair
14 tudo
15 na
16 prova
17 ?
18 Agu
19 ##ardo
20 uma
21 resposta
22 ,
23 João
24 .
25 [SEP]


Exibe os embenddings das sentenças

In [ ]:
# Índice das sentenças a serem comparadas
sentenca1Original = documento_original[0]
sentenca2Original = documento_original[1]
sentenca3Original = documento_original[2]
sentenca4Original = documento_original[3]

embeddingSentenca1Original = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, sentenca1Original, tokenizer)
embeddingSentenca2Original = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, sentenca2Original, tokenizer)
embeddingSentenca3Original = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, sentenca3Original, tokenizer)
embeddingSentenca4Original = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, sentenca4Original, tokenizer)

print('Os primeiros 4 valores de cada sentença do documento original.')

print('\nSentença 1:', sentenca1Original,'-', str(embeddingSentenca1Original[:4]))
print('Soma embedding Sentença1:', sentenca1Original,'-', str(torch.sum(embeddingSentenca1Original[:4])))

print('\nSentença 2:', sentenca2Original,'-', str(embeddingSentenca2Original[:4]))
print('Soma embedding Sentença2:', sentenca2Original,'-', str(torch.sum(embeddingSentenca2Original[:4])))

print('\nSentença 3:', sentenca3Original,'-', str(embeddingSentenca3Original[:4]))
print('Soma embedding Sentença3:', sentenca3Original,'-', str(torch.sum(embeddingSentenca3Original[:4])))

print('\nSentença 4:', sentenca4Original,'-', str(embeddingSentenca4Original[:4]))
print('Soma embedding Sentença4:', sentenca4Original,'-', str(torch.sum(embeddingSentenca4Original[:4])))

Os primeiros 4 valores de cada sentença do documento original.

Sentença 1: Bom Dia, professor. - tensor([[-0.0680, -0.4615,  0.3552,  ..., -0.3943, -0.1818, -0.4821],
        [-0.1000, -0.0630,  0.0840,  ..., -0.6630,  0.1641, -0.8297],
        [-0.3165,  0.4208,  0.2178,  ..., -0.4981,  0.1935, -0.3366],
        [ 0.1248,  0.2383,  0.8987,  ..., -0.4940, -0.4578, -0.0353]])
Soma embedding Sentença1: Bom Dia, professor. - tensor(-13.8934)

Sentença 2: Qual o conteúdo da prova? - tensor([[-0.5894, -0.4310,  0.1449,  ...,  0.1601, -0.2918, -0.5303],
        [ 0.1349, -0.2476,  0.4605,  ..., -0.3036, -0.6972,  0.2135],
        [ 0.4359, -0.6972,  0.4066,  ...,  0.0177, -0.5852, -0.0615],
        [ 0.0544,  0.1606,  0.4150,  ..., -0.3822, -0.1052, -0.0296]])
Soma embedding Sentença2: Qual o conteúdo da prova? - tensor(-13.9466)

Sentença 3: Vai cair tudo na prova? - tensor([[-0.3652, -0.5015, -0.1626,  ...,  0.0161, -0.5967,  0.1675],
        [ 0.1421,  0.1797, -0.0014,  ..., -0.5730, -0.

Examinando os embeddings do documento original



In [ ]:
# Índice das sentenças a serem comparadas
sentenca1Original = documento_original[0]
sentenca2Original = documento_original[1]
sentenca3Original = documento_original[2]
sentenca4Original = documento_original[3]

print("Documento Original:", documento_original)

# Localiza os índices dos tokens da sentença no documento
sentenca1TokenizadaOriginal = tokenizer.tokenize(sentenca1Original)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_original,sentenca1TokenizadaOriginal)
embeddingSentenca1Original = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, sentenca1Original, tokenizer)
print('\nSentença 1 Original=\'', sentenca1Original, '\'')
print('    Sentença tokenizada:', sentenca1TokenizadaOriginal)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca1Original.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca1Original))

# Localiza os índices dos tokens da sentença no documento
sentenca2TokenizadaOriginal = tokenizer.tokenize(sentenca2Original)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_original,sentenca2TokenizadaOriginal)
embeddingSentenca2Original = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, sentenca2Original, tokenizer)
print('\nSentença 2 Original=\'', sentenca2Original, '\'')
print('    Sentença tokenizada:', sentenca2TokenizadaOriginal)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca2Original.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca2Original))

# Localiza os índices dos tokens da sentença no documento
sentenca3TokenizadaOriginal = tokenizer.tokenize(sentenca3Original)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_original,sentenca3TokenizadaOriginal)
embeddingSentenca3Original = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, sentenca3Original, tokenizer)
print('\nSentença 3 Original=\'', sentenca3Original, '\'')
print('    Sentença tokenizada:', sentenca3TokenizadaOriginal)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca3Original.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca3Original))

# Localiza os índices dos tokens da sentença no documento
sentenca4TokenizadaOriginal = tokenizer.tokenize(sentenca4Original)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_original,sentenca4TokenizadaOriginal)
embeddingSentenca4Original = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, sentenca4Original, tokenizer)
print('\nSentença 4 Original=\'', sentenca4Original, '\'')
print('    Sentença tokenizada:', sentenca4TokenizadaOriginal)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca4Original.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca4Original))


Documento Original: ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']

Sentença 1 Original=' Bom Dia, professor. '
    Sentença tokenizada: ['Bom', 'Dia', ',', 'professor', '.']
    => inicio em 1 e término em 5
    Formato modelo : torch.Size([5, 768])
    Soma embeddings:  -15.02

Sentença 2 Original=' Qual o conteúdo da prova? '
    Sentença tokenizada: ['Qual', 'o', 'conteúdo', 'da', 'prova', '?']
    => inicio em 6 e término em 11
    Formato modelo : torch.Size([6, 768])
    Soma embeddings:  -17.08

Sentença 3 Original=' Vai cair tudo na prova? '
    Sentença tokenizada: ['Vai', 'cair', 'tudo', 'na', 'prova', '?']
    => inicio em 12 e término em 17
    Formato modelo : torch.Size([6, 768])
    Soma embeddings:  -10.65

Sentença 4 Original=' Aguardo uma resposta, João. '
    Sentença tokenizada: ['Agu', '##ardo', 'uma', 'resposta', ',', 'João', '.']
    => inicio em 18 e término em 24
    Formato modelo : torch.Size([7

### Documento Permutado

In [ ]:
# Define um documento com a permutação das sentenças do documento original
documento_permutado = [documento_original[3],   # "Aguardo uma resposta, João.",
             documento_original[1],             # "Qual o conteúdo da prova?",              
             documento_original[0],             # "Vai cair tudo na prova?",
             documento_original[2]]             # "Bom Dia, professor."]     

# Use o documento permutado igual ao original para testar se as medidas estão corretas
#documento_permutado = documento_original

# Concatena as sentenças do documento em uma string
stringDocumentoPermutado = ' '.join(documento_permutado)

# Adiciona os tokens especiais
documento_marcado_permutado = "[CLS] " + stringDocumentoPermutado + " [SEP]"

# Divide a sentença em tokens
documento_tokenizado_permutado = tokenizer.tokenize(documento_marcado_permutado)

# Mapeia os tokens em seus índices do vocabulário
documento_tokens_indexados_permutado = tokenizer.convert_tokens_to_ids(documento_tokenizado_permutado)

# Mostra os tokens com seus índices
i = 0
for tup in zip(documento_tokenizado_permutado, documento_tokens_indexados_permutado):
    print('{:>3} {:<12} {:>6,}'.format(i, tup[0], tup[1]))
    i = i + 1

  0 [CLS]           101
  1 Agu           8,125
  2 ##ardo        2,222
  3 uma             230
  4 resposta      4,299
  5 ,               117
  6 João          1,453
  7 .               119
  8 Qual         13,082
  9 o               146
 10 conteúdo      5,015
 11 da              180
 12 prova         2,310
 13 ?               136
 14 Bom           8,399
 15 Dia           3,616
 16 ,               117
 17 professor     2,917
 18 .               119
 19 Vai          20,805
 20 cair          9,322
 21 tudo          2,745
 22 na              229
 23 prova         2,310
 24 ?               136
 25 [SEP]           102


In [ ]:
# Marca cada um dos tokens como pertencentes à sentença "1".
mascara_atencao_permutado = [1] * len(documento_tokenizado_permutado)

print (mascara_atencao_permutado)
print (len(mascara_atencao_permutado))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
26


Convertendo as listas em tensores

In [ ]:
# Importa a bibliteca
import torch

# Converte as entradas de listas para tensores do torch
tokens_tensores_permutado = torch.as_tensor([documento_tokens_indexados_permutado])
mascara_atencao_tensores_permutado = torch.as_tensor([mascara_atencao_permutado])

Gera os embeddings para o documento original. Guarda somente a última camada da rede em `outputs`.


In [ ]:
# Prediz os atributos dos estados ocultos para cada camada
with torch.no_grad():
    # output[0] contém last_hidden_states
    outputs = model(tokens_tensores_permutado, mascara_atencao_tensores_permutado)

Recupera a saída da última camada

In [ ]:
# Recupera a última e única camada da saída
last_hidden_states = outputs[0]

print ("O vetor da última camada oculta tem o formato:", last_hidden_states.size())

O vetor da última camada oculta tem o formato: torch.Size([1, 26, 768])


Vamos nos livrar da dimensão lotes "batches", pois não precisamos dela.

In [ ]:
# Remove a dimensão 1, o lote "batches".
#O método squeeze remove a primeira dimensão(0) pois possui tamanho 1
embeddingDocumentoPermutado = torch.squeeze(last_hidden_states, dim=0)

print ("O vetor de tokens de embedding do documento permutado tem o formato:", embeddingDocumentoPermutado.size())

O vetor de tokens de embedding do documento permutado tem o formato: torch.Size([26, 768])


Exibe os embenddings das sentenças

In [ ]:
# Índice das sentenças a serem comparadas
sentenca1Permutado = documento_permutado[0]
sentenca2Permutado = documento_permutado[1]
sentenca3Permutado = documento_permutado[2]
sentenca4Permutado = documento_permutado[3]

embeddingSentenca1Permutado = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoPermutado, stringDocumentoPermutado, sentenca1Permutado, tokenizer)
embeddingSentenca2Permutado = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoPermutado, stringDocumentoPermutado, sentenca2Permutado, tokenizer)
embeddingSentenca3Permutado = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoPermutado, stringDocumentoPermutado, sentenca3Permutado, tokenizer)
embeddingSentenca4Permutado = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoPermutado, stringDocumentoPermutado, sentenca4Permutado, tokenizer)

print('Os primeiros 4 valores de cada sentença do documento permutado.')

print('\nSentença 1:', sentenca1Permutado,'-', str(embeddingSentenca1Permutado[:4]))
print('Soma embedding Sentença1:', sentenca1Original,'-', str(torch.sum(embeddingSentenca1Original[:4])))

print('\nSentença 2:', sentenca2Permutado,'-', str(embeddingSentenca2Permutado[:4]))
print('Soma embedding Sentença2:', sentenca2Permutado,'-', str(torch.sum(embeddingSentenca2Permutado[:4])))

print('\nSentença 3:', sentenca3Permutado,'-', str(embeddingSentenca3Permutado[:4]))
print('Soma embedding Sentença3:', sentenca3Permutado,'-', str(torch.sum(embeddingSentenca3Original[:4])))

print('\nSentença 4:', sentenca4Permutado,'-', str(embeddingSentenca4Permutado[:4]))
print('Soma embedding Sentença4:', sentenca4Permutado,'-', str(torch.sum(embeddingSentenca4Permutado[:4])))

Os primeiros 4 valores de cada sentença do documento permutado.

Sentença 1: Aguardo uma resposta, João. - tensor([[-0.0052, -0.4706,  0.6481,  ..., -0.4811,  0.2881, -0.4057],
        [-0.1516, -0.2450,  0.2693,  ..., -0.5534, -0.3164, -0.3582],
        [-0.7358, -0.0988,  0.5145,  ..., -0.1814, -0.3193, -0.3159],
        [-0.1278,  0.3571,  0.5481,  ..., -0.0720, -0.0478, -0.4931]])
Soma embedding Sentença1: Bom Dia, professor. - tensor(-13.8934)

Sentença 2: Qual o conteúdo da prova? - tensor([[-0.4295, -0.3426,  0.1005,  ...,  0.1568, -0.3225, -0.3857],
        [ 0.1047, -0.1946,  0.4177,  ..., -0.3388, -0.7529,  0.1693],
        [ 0.4377, -0.5952,  0.5448,  ...,  0.0463, -0.5607, -0.1783],
        [ 0.0898,  0.1063,  0.4610,  ..., -0.3313, -0.1764,  0.0920]])
Soma embedding Sentença2: Qual o conteúdo da prova? - tensor(-13.8054)

Sentença 3: Bom Dia, professor. - tensor([[-0.0458, -0.4268,  0.2399,  ..., -0.4504, -0.2225, -0.6343],
        [ 0.0650, -0.1385,  0.0230,  ..., -0.7129

In [ ]:
# Índice das sentenças a serem comparadas
sentenca1Permutado = documento_permutado[0]
sentenca2Permutado = documento_permutado[1]
sentenca3Permutado = documento_permutado[2]
sentenca4Permutado = documento_permutado[3]

print("Documento Permutado:", documento_permutado)

# Localiza os índices dos tokens da sentença no documento
sentenca1TokenizadaPermutado = tokenizer.tokenize(sentenca1Permutado)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_permutado,sentenca1TokenizadaPermutado)
embeddingSentenca1Permutado = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoPermutado, stringDocumentoPermutado, sentenca1Permutado, tokenizer)
print('\nSentença 1 Permutada=\'', sentenca1Permutado, '\'')
print('    Sentença tokenizada:', sentenca1TokenizadaPermutado)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca1Permutado.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca1Permutado))

# Localiza os índices dos tokens da sentença no documento
sentenca2TokenizadaPermutado = tokenizer.tokenize(sentenca2Permutado)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_permutado,sentenca2TokenizadaPermutado)
embeddingSentenca2Permutado = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoPermutado, stringDocumentoPermutado, sentenca2Permutado, tokenizer)
print('\nSentença 2 Permutada=\'', sentenca2Permutado, '\'')
print('    Sentença tokenizada:', sentenca2TokenizadaPermutado)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca2Permutado.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca2Permutado))

# Localiza os índices dos tokens da sentença no documento
sentenca3TokenizadaPermutado = tokenizer.tokenize(sentenca3Permutado)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_permutado,sentenca3TokenizadaPermutado)
embeddingSentenca3Permutado = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoPermutado, stringDocumentoPermutado, sentenca3Permutado, tokenizer)
print('\nSentença 3 Permutada=\'', sentenca3Permutado, '\'')
print('    Sentença tokenizada:', sentenca3TokenizadaPermutado)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca3Permutado.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca3Permutado))

# Localiza os índices dos tokens da sentença no documento
sentenca4TokenizadaPermutado = tokenizer.tokenize(sentenca4Permutado)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_permutado,sentenca4TokenizadaPermutado)
embeddingSentenca4Permutado = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoPermutado, stringDocumentoPermutado, sentenca4Permutado, tokenizer)
print('\nSentença 4 Permutada=\'', sentenca4Permutado, '\'')
print('    Sentença tokenizada:', sentenca4TokenizadaPermutado)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca4Permutado.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca4Permutado))


Documento Permutado: ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']

Sentença 1 Permutada=' Aguardo uma resposta, João. '
    Sentença tokenizada: ['Agu', '##ardo', 'uma', 'resposta', ',', 'João', '.']
    => inicio em 1 e término em 7
    Formato modelo : torch.Size([7, 768])
    Soma embeddings:  -14.89

Sentença 2 Permutada=' Qual o conteúdo da prova? '
    Sentença tokenizada: ['Qual', 'o', 'conteúdo', 'da', 'prova', '?']
    => inicio em 8 e término em 13
    Formato modelo : torch.Size([6, 768])
    Soma embeddings:  -17.25

Sentença 3 Permutada=' Bom Dia, professor. '
    Sentença tokenizada: ['Bom', 'Dia', ',', 'professor', '.']
    => inicio em 14 e término em 18
    Formato modelo : torch.Size([5, 768])
    Soma embeddings:  -15.11

Sentença 4 Permutada=' Vai cair tudo na prova? '
    Sentença tokenizada: ['Vai', 'cair', 'tudo', 'na', 'prova', '?']
    => inicio em 19 e término em 24
    Formato modelo : torch.Si

### Examinando as sentenças

A mesma sentença apresenta embeddings com valores diferentes, pois se encontram em locais diferentes do documento. A soma de todos os embeddings demonstra isto.

In [ ]:
print('\nSentença 4 Original=\'', sentenca4Original, '\'')
print('    Sentença tokenizada:', sentenca4TokenizadaOriginal)
print('    Formato modelo :', embeddingSentenca4Original.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca4Original))
print('    Os 4 primeiros embeddings:', str(embeddingSentenca4Original[:4]))

print('\nSentença 1 Permutada=\'', sentenca1Permutado, '\'')
print('    Sentença tokenizada:', sentenca1TokenizadaPermutado)
print('    Formato modelo :', embeddingSentenca1Permutado.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca1Permutado))
print('    Os 4 primeiros embeddings:', str(embeddingSentenca1Permutado[:4]))


Sentença 4 Original=' Aguardo uma resposta, João. '
    Sentença tokenizada: ['Agu', '##ardo', 'uma', 'resposta', ',', 'João', '.']
    Formato modelo : torch.Size([7, 768])
    Soma embeddings:  -12.13
    Os 4 primeiros embeddings: tensor([[-0.0835, -0.4042,  0.4330,  ..., -0.3271,  0.2262, -0.4599],
        [-0.2782, -0.1630,  0.0997,  ..., -0.4231, -0.3089, -0.5329],
        [-0.5474, -0.0118,  0.3950,  ..., -0.0949, -0.3534, -0.3717],
        [ 0.0146,  0.2901,  0.3920,  ...,  0.1161,  0.0407, -0.6904]])

Sentença 1 Permutada=' Aguardo uma resposta, João. '
    Sentença tokenizada: ['Agu', '##ardo', 'uma', 'resposta', ',', 'João', '.']
    Formato modelo : torch.Size([7, 768])
    Soma embeddings:  -14.89
    Os 4 primeiros embeddings: tensor([[-0.0052, -0.4706,  0.6481,  ..., -0.4811,  0.2881, -0.4057],
        [-0.1516, -0.2450,  0.2693,  ..., -0.5534, -0.3164, -0.3582],
        [-0.7358, -0.0988,  0.5145,  ..., -0.1814, -0.3193, -0.3159],
        [-0.1278,  0.3571,  0.5481,  .

### Subtração entre os embeddings das sentenças

#### Calcula a média aritmética da subtração entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento original. 

In [ ]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaSsub = 0

# Percorre as sentenças do documento
for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Encontra os maiores embeddings os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSi, linha = torch.max(embeddingSi, dim=0)        
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSi:", len(maiorEmbeddingSi))
        
    # Encontra os maiores embeddings os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSj, linha = torch.max(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSj:", len(maiorEmbeddingSj))
       
    # Subtração entre os embeddings de Si e Sj
     # Entrada: (<768 ou 1024>) x (<768 ou 1024>) 
    Ssub = torch.sub(maiorEmbeddingSi, maiorEmbeddingSj)
    # Saída: <768 ou 1024>  
    #print("Ssub=", Ssub.shape)
    
    # Calcula a média dos embeddings do subtração para realizar a soma
    somaSsub = somaSsub + torch.mean(Ssub)

DsubOriginal = float(somaSsub)/float(len(documento_original)-1)
print("Ssub Original:", DsubOriginal)

Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Ssub Original: -0.027655382951100666


#### #### Calcula a média aritmética da subtração entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento permutado. 

In [ ]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaSsub = 0

# Percorre as sentenças do documento
for i in range(np-1):
    # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Encontra os maiores embeddings os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSi, linha = torch.max(embeddingSi, dim=0)        
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSi:", len(maiorEmbeddingSi))
        
    # Encontra os maiores embeddings os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSj, linha = torch.max(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSj:", len(maiorEmbeddingSj))
       
    # Subtração entre os embeddings de Si e Sj
     # Entrada: (<768 ou 1024>) x (<768 ou 1024>)  
    Ssub = torch.sub(maiorEmbeddingSi, maiorEmbeddingSj)
    # Saída: <768 ou 1024>  
    #print("Ssub=", Ssub.shape)
        
    # Calcula a média dos embeddings do subtração para realizar a soma
    somaSsub = somaSsub + torch.mean(Ssub)

DsubPermutado = float(somaSsub)/float(np-1)
print("Ssub permutado:", DsubPermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']
Quantidade de sentenças: 4
Ssub permutado: 0.023601315915584564


#### Compara as médias da subtração dos embeddings das sentenças do documento original e permutado.

Características das medidas:
- Documentos com sentenças iguais resulta uma medida igual a 0.
- Documentos com sentenças diferenntes resulta uma medida maior que 0.
- Documento com sentenças muito diferentes apresentam valores maiores que 0.
- Documentos iguais resultam em medidas iguais. 
- É uma medida de diferença.


In [ ]:
print("Dsub Original :", DsubOriginal)
print("Dsub Permutado:", DsubPermutado)

if (DsubOriginal <= DsubPermutado):
    print("Documento original tem menor diferença entre as sentenças!")
else:
    print("Documento Permutado tem menor diferença entre as sentenças!")

if (DsubOriginal > DsubPermutado):
    print("Documento original tem menor diferença entre as sentenças!")
else:
    print("Documento Permutado tem menor diferença entre as sentenças!")

Dsub Original : -0.027655382951100666
Dsub Permutado: 0.023601315915584564
Documento original tem menor diferença entre as sentenças!
Documento Permutado tem menor diferença entre as sentenças!


### Subtração absoluta entre os embeddings das sentenças

#### Calcula a média aritmética da subtração absoluta entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento original. 

In [ ]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaSsubabs = 0

# Percorre as sentenças do documento
for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Encontra os maiores embeddings os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSi, linha = torch.max(embeddingSi, dim=0)        
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSi:", len(maiorEmbeddingSi))
        
    # Encontra os maiores embeddings os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSj, linha = torch.max(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSj:", len(maiorEmbeddingSj))
       
    # Subtração entre os embeddings de Si e Sj
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Ssubabs = abs(torch.sub(maiorEmbeddingSi, maiorEmbeddingSj))
    # Saída: <768 ou 1024>  
    #print("Ssubabs=", Ssubabs.shape)
  
    # Calcula a média dos embeddings do subtração para realizar a soma
    somaSsubabs = somaSsubabs + torch.mean(Ssubabs)

DsubabsOriginal = float(somaSsubabs)/float(n-1)
print("Dsubabs Original:", DsubabsOriginal)

Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Dsubabs Original: 0.2602133552233378


#### Calcula a média aritmética da subtração absoluta entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento permutado. 

In [ ]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaSsubabs = 0

# Percorre as sentenças do documento
for i in range(np-1):
    # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Encontra os maiores embeddings os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSi, linha = torch.max(embeddingSi, dim=0)        
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSi:", len(maiorEmbeddingSi))
        
   # Encontra os maiores embeddings os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSj, linha = torch.max(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSj:", len(maiorEmbeddingSj))
       
    # Subtração entre os embeddings de Si e Sj
     # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Ssubabs = abs(torch.sub(maiorEmbeddingSi, maiorEmbeddingSj))
    # Saída: <768 ou 1024>  
    #print("Ssub=", Ssub.shape)

    # Calcula a média dos embeddings do subtração para realizar a soma
    somaSsubabs = somaSsubabs + torch.mean(Ssubabs)

DsubabsPermutado = float(somaSsubabs)/float(np-1)
print("Dsubabs permutado:", DsubabsPermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']
Quantidade de sentenças: 4
Dsubabs permutado: 0.2577057679494222


#### Compara as médias da subtração absoluta dos embeddings das sentenças do documento original e permutado

Características das medidas:
- Documentos com sentenças iguais resulta uma medida igual a 0.
- Documentos com sentenças diferenntes resulta uma medida maior que 0.
- Documento com sentenças muito diferentes apresentam valores maiores que 0.
- Documentos iguais resultam em medidas iguais. 
- É uma medida de diferença.

In [ ]:
print("Dsubabs Original :", DsubabsOriginal)
print("Dsubabs Permutado:", DsubabsPermutado)

if (DsubabsOriginal <= DsubabsPermutado):
    print("Documento original tem menor diferença absoluta entre as sentenças!")
else:
    print("Documento Permutado tem menor diferença absoluta entre as sentenças!")

Dsubabs Original : 0.2602133552233378
Dsubabs Permutado: 0.2577057679494222
Documento Permutado tem menor diferença absoluta entre as sentenças!


### Produto entre os embeddings das sentenças

#### Calcula a média aritmética do produto das matrizes dos tokens das sentenças do documento original. 

In [ ]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaSprod = 0

# Percorre as sentenças do documento
for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Para a multiplicação pode ocorrer número de colunas(qtde_tokens) de Si tem que ser igual ao número de linhas(embeddings) de Sj.
    # Permute realiza a troca da dimensão 1(qtde_embeddings) pela 0(qtde_tokesn) na sentença Sj para que seja possível realizar o produto. 
    # Ocorre a troca da qtde_tokens pela quantidade de embeddings.
    # Entrada: (<qtde_tokensSi> x <768 ou 1024>) x (<qtde_tokensSj> x <768 ou 1024>)
    # Permute: <qtde_tokensSi> x <768 ou 1024>) x (<768 ou 1024> x <qtde_tokensSj>)
    Sprod = torch.matmul(embeddingSi, embeddingSj.permute(1,0))    
    # Saída: <qtde_tokensSi> x  <qtde_tokensSj>
    #print("Sprod=", Sprod.shape)

    # Encontra os maiores embeddings do produto para realizar a soma
    somaSprod = somaSprod + torch.mean(Sprod)

DprodOriginal = float(somaSprod)/float(n-1)
print("Dprod Original:", DprodOriginal)
    


Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Dprod Original: 43.7073974609375


#### Calcula a média aritmética do produto das matrizes dos tokens das sentenças do documento permutado. 

In [ ]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaSprod = 0

# Percorre as sentenças do documento
for i in range(np-1):
    # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSi=", mediaEmbeddingSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSj=", mediaEmbeddingSj.shape)
  
    # Para a multiplicação pode ocorrer número de colunas(qtde_tokens) de Si tem que ser igual ao número de linhas(embeddings) de Sj.
    # Permute realiza a troca da dimensão 1(qtde_embeddings) pela 0(qtde_tokesn) na sentença Sj para que seja possível realizar o produto. 
    # Ocorre a troca da qtde_tokens pela quantidade de embeddings.
    # Entrada: (<qtde_tokensSi> x <768 ou 1024>) x (<qtde_tokensSj> x <768 ou 1024>)
    # Permute: <qtde_tokensSi> x <768 ou 1024>) x (<768 ou 1024> x <qtde_tokensSj>)
    Sprod = torch.matmul(embeddingSi, embeddingSj.permute(1,0))    
    # Saída: <qtde_tokensSi> x  <qtde_tokensSj>
    #print("Sprod=", Sprod.shape)
    
    # Encontra os maiores embeddings do produto para realizar a soma
    somaSprod = somaSprod + torch.mean(Sprod)

DprodPermutado = float(somaSprod)/float(np-1)
print("Dprod permutado:", DprodPermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']
Quantidade de sentenças: 4
Dprod permutado: 43.55047607421875


#### Compara as médias do produto dos embeddings das sentenças do documento original e permutado

Características das medidas:
- Dois documentos(vetores) que apontam em uma direção semelhante retornam um produto escalar positivo.
- Dois documentos perpendiculares retornam um produto escalar de zero.
- Dois documentos(vetores) que apontam em direções opostas retornam um produto escalar negativo.


In [ ]:
print("Dprod Original :", DprodOriginal)
print("Dprod Permutado:", DprodPermutado)

if (DprodOriginal > 0 and DprodPermutado > 0):
    print("As sentenças do documento original e as sentenças do documento permutado estão relacionadas!")
else:
  if (DprodOriginal < 0 and DprodPermutado > 0) or (DprodOriginal > 0 and DprodPermutado < 0):
    print("As sentenças do documento original e as sentenças do documento permutado não estão relacionadas!")
  else:  
    print("As sentenças do documento original e as sentenças do documento permutado não possuem relação!")              

Dprod Original : 43.7073974609375
Dprod Permutado: 43.55047607421875
As sentenças do documento original e as sentenças do documento permutado estão relacionadas!


### Produto Absoluto entre os embeddings das sentenças

#### Calcula a média aritmética do produto absoluto das matrizes dos tokens das sentenças do documento original. 

In [ ]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaSprodabs = 0

# Percorre as sentenças do documento
for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)
    
    # # Para a multiplicação pode ocorrer número de colunas(qtde_tokens) de Si tem que ser igual ao número de linhas(embeddings) de Sj.
    # Permute realiza a troca da dimensão 1(qtde_embeddings) pela 0(qtde_tokesn) na sentença Sj para que seja possível realizar o produto. 
    # Ocorre a troca da qtde_tokens pela quantidade de embeddings.
    # Entrada: (<qtde_tokensSi> x <768 ou 1024>) x (<qtde_tokensSj> x <768 ou 1024>)
    # Permute: <qtde_tokensSi> x <768 ou 1024>) x (<768 ou 1024> x <qtde_tokensSj>)
    Sprodabs = abs(torch.matmul(embeddingSi, embeddingSj.permute(1,0)))   
    # Saída: <qtde_tokensSi> x  <qtde_tokensSj>
    #print("Sprodabs=", Sprodabs.shape)
   
    # Encontra os maiores embeddings absolutos do produto para realizar a soma
    somaSprodabs = somaSprodabs + torch.mean(Sprodabs)

DprodabsOriginal = float(somaSprodabs)/float(n-1)
print("Dprodabs Original:", DprodabsOriginal)
    


Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Dprodabs Original: 43.7073974609375


#### Calcula a média aritmética do produto absoluto das matrizes dos tokens das sentenças do documento permutado. 

In [ ]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaSprodabs = 0

# Percorre as sentenças do documento
for i in range(np-1):
    # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)
 
     # Para a multiplicação pode ocorrer número de colunas(qtde_tokens) de Si tem que ser igual ao número de linhas(embeddings) de Sj.
    # Permute realiza a troca da dimensão 1(qtde_embeddings) pela 0(qtde_tokesn) na sentença Sj para que seja possível realizar o produto. 
    # Ocorre a troca da qtde_tokens pela quantidade de embeddings.
    # Entrada: (<qtde_tokensSi> x <768 ou 1024>) x (<qtde_tokensSj> x <768 ou 1024>)
    # Permute: <qtde_tokensSi> x <768 ou 1024>) x (<768 ou 1024> x <qtde_tokensSj>)
    Sprodabs = abs(torch.matmul(embeddingSi, embeddingSj.permute(1,0)))    
    # Saída: <qtde_tokensSi> x  <qtde_tokensSj>
    #print("Sprodabs=", Sprodabs.shape)

    # Encontra os maiores embeddings absolutos do produto para realizar a soma
    somaSprodabs = somaSprodabs + torch.mean(Sprodabs)

DprodabsPermutado = float(somaSprodabs)/float(np-1)
print("Dprodabs permutado:", DprodabsPermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']
Quantidade de sentenças: 4
Dprodabs permutado: 43.55047607421875


#### Compara as médias do produto absoluto dos embeddings das sentenças do documento original e permutado

Características das medidas:
- Dois documentos(vetores) que apontam em uma direção semelhante retornam um produto escalar positivo.
- Dois documentos perpendiculares retornam um produto escalar de zero.
- Dois documentos(vetores) que apontam em direções opostas retornam um produto escalar negativo.

In [ ]:
print("Dprodabs Original :", DprodabsOriginal)
print("Dprodabs Permutado:", DprodabsPermutado)

if (DprodabsOriginal > 0 and DprodabsPermutado > 0):
    print("As sentenças do documento original e as sentenças do documento permutado estão relacionadas!")
else:
  if (DprodabsOriginal < 0 and DprodabsPermutado > 0) or (DprodabsOriginal > 0 and DprodabsPermutado < 0):
    print("As sentenças do documento original e as sentenças do documento permutado não estão relacionadas!")
  else:  
    print("As sentenças do documento original e as sentenças do documento permutado não possuem relação!")          

Dprodabs Original : 43.7073974609375
Dprodabs Permutado: 43.55047607421875
As sentenças do documento original e as sentenças do documento permutado estão relacionadas!


### Produto Escalar entre os embeddings das sentenças

#### Calcula a média aritmética do produto escalar entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento orginal. 



In [ ]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaSprode = 0

# Percorre as sentenças do documento
for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

   # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

   # Encontra os maiores embeddings os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSi, linha = torch.max(embeddingSi, dim=0)        
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSi:", len(maiorEmbeddingSi))
        
    # Encontra os maiores embeddings os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSj, linha = torch.max(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSj:", len(maiorEmbeddingSj))

    # Produto escalar entre os embeddings de Si e Sj, pois Si e Sj possui uma única dimensão
    # https://pytorch.org/docs/master/generated/torch.matmul.html#torch.matmul
     # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Sprode = torch.matmul(maiorEmbeddingSi, maiorEmbeddingSj)
    # Saída: Um número real 
    #print("Sprode=", Sprode)
        
    somaSprode = somaSprode + Sprode

DprodeOriginal = float(somaSprode)/float(n-1)
print("Dprode Original:", DprodeOriginal)
    


Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Dprode Original: 200.03959147135416


#### Calcula a média aritmética do produto escalar entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento permutado. 

In [ ]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaSprode = 0

# Percorre as sentenças do documento
for i in range(np-1):
    # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Encontra os maiores embeddings os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSi, linha = torch.max(embeddingSi, dim=0)        
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSi:", len(maiorEmbeddingSi))
        
    # Encontra os maiores embeddings os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSj, linha = torch.max(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSj:", len(maiorEmbeddingSj))
  
    # Produto escalar entre os embeddings de Si e Sj, pois Si e Sj possui uma única dimensão
    # https://pytorch.org/docs/master/generated/torch.matmul.html#torch.matmul
     # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Sprode = torch.matmul(maiorEmbeddingSi, maiorEmbeddingSj)    
    # Saída: Um número real 
    #print("Sprode=", Sprode)
    
    somaSprode = somaSprode + Sprode

DprodePermutado = float(somaSprode)/float(np-1)
print("Dprode permutado:", DprodePermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']
Quantidade de sentenças: 4
Dprode permutado: 199.48846435546875


#### Compara as médias do produto escalar dos embeddings das sentenças do documento original e permutado

Dprod Original : 230.61181640625
Dprod Permutado: 227.75482177734375
Documento original tem maior similaridade com o produto entre as sentenças!

In [ ]:
print("Dprode Original :", DprodeOriginal)
print("Dprode Permutado:", DprodePermutado)

if (DprodeOriginal < DprodePermutado):
    print("Documento original tem maior similaridade com o produto entre as sentenças!")
else:
    print("Documento Permutado tem menor similaridade com o produto entre as sentenças!")

Dprode Original : 200.03959147135416
Dprode Permutado: 199.48846435546875
Documento Permutado tem menor similaridade com o produto entre as sentenças!


### Produto Escalar Absoluto entre os embeddings das sentenças

#### Calcula a média aritmética do produto escalar absoluto entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento original. 

In [ ]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaSprodeabs = 0

# Percorre as sentenças do documento
for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

     # Encontra os maiores embeddings os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSi, linha = torch.max(embeddingSi, dim=0)        
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSi:", len(maiorEmbeddingSi))
        
    # Encontra os maiores embeddings os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSj, linha = torch.max(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSj:", len(maiorEmbeddingSj))
  
    # Produto escalar entre os embeddings de Si e Sj, pois Si e Sj possui uma única dimensão
    # https://pytorch.org/docs/master/generated/torch.matmul.html#torch.matmul
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Sprodeabs = abs(torch.matmul(maiorEmbeddingSi, maiorEmbeddingSj))
    # Saída: Um número real 
    #print("Sprodeabs=", Sprodeabs)

    somaSprodeabs = somaSprodeabs + Sprodeabs

DprodeabsOriginal = float(somaSprodeabs)/float(n-1)
print("Dprodabs Original:", DprodeabsOriginal)

Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Dprodabs Original: 200.03959147135416


#### Calcula a média aritmética do produto escalar absoluto entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento permutado. 

In [ ]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaSprodeabs = 0

# Percorre as sentenças do documento
for i in range(np-1):
    # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)
    
     # Encontra os maiores embeddings os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSi, linha = torch.max(embeddingSi, dim=0)        
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSi:", len(maiorEmbeddingSi))
        
    # Encontra os maiores embeddings os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSj, linha = torch.max(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSj:", len(maiorEmbeddingSj))
    
    # Produto escalar entre os embeddings de Si e Sj, pois Si e Sj possui uma única dimensão
    # https://pytorch.org/docs/master/generated/torch.matmul.html#torch.matmul
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Sprodeabs = abs(torch.matmul(maiorEmbeddingSi, maiorEmbeddingSj))
    # Saída: Um número real 
    #print("Sprodeabs=", Sprodeabs)
    
    somaSprodeabs = somaSprodeabs + Sprodeabs

DprodeabsPermutado = float(somaSprodeabs)/float(np-1)
print("Dprodabs permutado:", DprodeabsPermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']
Quantidade de sentenças: 4
Dprodabs permutado: 199.48846435546875


#### Compara as médias do produto escalar absoluto dos embeddings das sentenças do documento original e permutado

In [ ]:
print("Dprodabs Original :", DprodeabsOriginal)
print("Dprodabs Permutado:", DprodeabsPermutado)

if (DprodeabsOriginal < DprodeabsPermutado):
    print("Documento original tem maior similaridade com o produto absoluto entre as sentenças!")
else:
    print("Documento Permutado tem menor similaridade com o produto absoluto entre as sentenças!")

Dprodabs Original : 200.03959147135416
Dprodabs Permutado: 199.48846435546875
Documento Permutado tem menor similaridade com o produto absoluto entre as sentenças!


### Média entre os embeddings das sentenças

#### Calcula a média aritmética entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento original. 

In [ ]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaSavg = 0

# Percorre as sentenças do documento
for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Encontra os maiores embeddings os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSi, linha = torch.max(embeddingSi, dim=0)        
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSi:", len(maiorEmbeddingSi))
        
    # Encontra os maiores embeddings os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSj, linha = torch.max(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSj:", len(maiorEmbeddingSj))
  
    # Média entre os embeddings de Si e Sj
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Savg = (maiorEmbeddingSi.add(maiorEmbeddingSj))/2.0  
    # Saída: <768 ou 1024>
    #print("Savg=", Savg.shape)  

    # Calcula a média dos embeddings do cálculo para realizar a soma
    somaSavg = somaSavg + torch.mean(Savg)

DavgOriginal = float(somaSavg)/float(n-1)
print("Davg Original:", DavgOriginal)

Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Davg Original: 0.43032602469126385


#### Calcula a média aritmética entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento permutado.

In [ ]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaSavg = 0

# Percorre as sentenças do documento
for i in range(np-1):
    # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Encontra os maiores embeddings os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSi, linha = torch.max(embeddingSi, dim=0)        
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSi:", len(maiorEmbeddingSi))
        
    # Encontra os maiores embeddings os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSj, linha = torch.max(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSj:", len(maiorEmbeddingSj))
  
    # Média entre os embeddings de Si e Sj
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)  
    Savg = (maiorEmbeddingSi.add(maiorEmbeddingSj))/2.0  
    # Saída: <768 ou 1024>
    #print("Savg=", Savg.shape)  

    # Calcula a média dos embeddings do cálculo para realizar a soma
    somaSavg = somaSavg + torch.mean(Savg)

DavgPermutado = float(somaSavg)/float(np-1)
print("Davg permutado:", DavgPermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']
Quantidade de sentenças: 4
Davg permutado: 0.4282990296681722


#### Compara as médias dos embeddings das sentenças do documento original e permutado


In [ ]:
print("Davg Original :", DavgOriginal)
print("Davg Permutado:", DavgPermutado)

if (DavgOriginal < DavgPermutado):
    print("Documento original tem maior similaridade com a média entre as sentenças!")
else:
    print("Documento Permutado tem menor similaridade com a mpedia entre as sentenças!")

Davg Original : 0.43032602469126385
Davg Permutado: 0.4282990296681722
Documento Permutado tem menor similaridade com a mpedia entre as sentenças!


### Similaridade de cosseno entre os embeddings das sentenças

In [ ]:
# Import das bibliotecas.
from scipy.spatial.distance import cosine

def similaridadeCoseno(sentenca1, sentenca2):
  similaridade = 1 - cosine(sentenca1, sentenca2)
  return similaridade

#### Calcula a média aritmética da similaridade do coseno entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento original. 

In [ ]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaScos = 0

# Percorre as sentenças do documento
for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Encontra os maiores embeddings os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSi, linha = torch.max(embeddingSi, dim=0)        
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSi:", len(maiorEmbeddingSi))
        
    # Encontra os maiores embeddings os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSj, linha = torch.max(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSj:", len(maiorEmbeddingSj))
  
    # Similaridade entre os embeddings Si e Sj
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Scos = similaridadeCoseno(maiorEmbeddingSi, maiorEmbeddingSj)
    # Saída: Um número real
    
    # Acumula a medida
    somaScos = somaScos + Scos

DcosOriginal = float(somaScos)/float(n-1)
print("Dcos Original:", DcosOriginal)  


Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Dcos Original: 0.819700022538503


#### Calcula a média aritmética da similaridade do coseno entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento permutado. 

In [ ]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaScos = 0

# Percorre as sentenças do documento
for i in range(np-1):
    # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Encontra os maiores embeddings os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSi, linha = torch.max(embeddingSi, dim=0)        
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSi:", len(maiorEmbeddingSi))
        
    # Encontra os maiores embeddings os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSj, linha = torch.max(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSj:", len(maiorEmbeddingSj))
  
    # Similaridade entre os embeddings Si e Sj
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Scos = similaridadeCoseno(maiorEmbeddingSi, maiorEmbeddingSj)
    # Saída: Um número real
    
    # Acumula a medida
    somaScos = somaScos + Scos

DcosPermutado = float(somaScos)/float(np-1)
print("Dcos Original:", DcosPermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']
Quantidade de sentenças: 4
Dcos Original: 0.8229710857073466


#### Compara as médias da similaridade de cosseno dos embeddings das sentenças do documento original e permutado

Características das medidas:
- Documentos com sentenças iguais resulta uma medida igual a 1.
- Documentos com sentenças diferenntes resulta uma medida menor que 1.
- Documento com sentenças muito diferentes apresentam valores menores que 1.
- Documentos iguais resultam em medidas iguais. 
- É uma medida de similaridade.



In [ ]:
print("Dcos Original :", DcosOriginal)
print("Dcos Permutado:", DcosPermutado)

if (DcosOriginal > DcosPermutado):
    print("Documento original tem maior similaridade de cosseno entre as sentenças!")
else:
    print("Documento Permutado tem menor similaridade de cosseno entre as sentenças!")

Dcos Original : 0.819700022538503
Dcos Permutado: 0.8229710857073466
Documento Permutado tem menor similaridade de cosseno entre as sentenças!


### Distância euclidiana entre os embeddings das sentenças

Possui outros nomes como distância L2 ou norma L2.

In [ ]:
# Import das bibliotecas.
from scipy.spatial.distance import euclidean

def distanciaEuclidiana(sentenca1, sentenca2):
  distancia = euclidean(sentenca1, sentenca2)

  return distancia

#### Calcula a média aritmética da distância euclidiana entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento original. 

In [ ]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaSeuc = 0

# Percorre as sentenças do documento
for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Encontra os maiores embeddings os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSi, linha = torch.max(embeddingSi, dim=0)        
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSi:", len(maiorEmbeddingSi))
        
    # Encontra os maiores embeddings os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSj, linha = torch.max(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSj:", len(maiorEmbeddingSj))
  
    # Diferença entre os embeddings Si e Sj
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Seuc = distanciaEuclidiana(maiorEmbeddingSi, maiorEmbeddingSj)
    # Saída: Um número real
    
    # Acumula a medida
    somaSeuc = somaSeuc + Seuc

DeucOriginal = float(somaSeuc)/float(n-1)
print("Deuc Original:", DeucOriginal)  


Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Deuc Original: 9.424604415893555


#### Calcula a média aritmética da distância euclidiana entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento permutado. 

In [ ]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaSeuc = 0

# Percorre as sentenças do documento
for i in range(np-1):
    # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Encontra os maiores embeddings os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSi, linha = torch.max(embeddingSi, dim=0)        
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSi:", len(maiorEmbeddingSi))
        
    # Encontra os maiores embeddings os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSj, linha = torch.max(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSj:", len(maiorEmbeddingSj))
  
    # Diferença entre os embeddings Si e Sj
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Seuc = distanciaEuclidiana(maiorEmbeddingSi, maiorEmbeddingSj)
    # Saída: Um número real
    
    # Acumula a medida
    somaSeuc = somaSeuc + Seuc

DeucPermutado = float(somaSeuc)/float(np-1)
print("Deuc Original:", DeucPermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']
Quantidade de sentenças: 4
Deuc Original: 9.29481569925944


#### Compara as médias da distância euclidiana dos embeddings das sentenças do documento original e permutado

Características das medidas:
- Documentos com sentenças iguais resulta uma medida igual a 0.
- Documentos com sentenças diferenntes resulta uma medida maior que 0.
- Documento com sentenças muito diferentes apresentam valores maiores que 0.
- Documentos iguais resultam em medidas iguais. 
- É uma medida de diferença.



In [ ]:
print("Deuc Original :", DeucOriginal)
print("Deuc Permutado:", DeucPermutado)

if (DeucOriginal > DeucPermutado):
    print("Documento original tem maior distância euclidiana entre as sentenças!")
else:
    print("Documento Permutado tem menor distância euclidiana entre as sentenças!")

Deuc Original : 9.424604415893555
Deuc Permutado: 9.29481569925944
Documento original tem maior distância euclidiana entre as sentenças!


### Distância Manhattan entre os embeddings das sentenças

Possui outros nomes como distância Cityblock, distância L1, norma L1 e métrica do táxi.

In [ ]:
# Import das bibliotecas.
from scipy.spatial.distance import cityblock

def distanciaEManhattan(sentenca1, sentenca2):
  distancia = cityblock(sentenca1, sentenca2)

  return distancia

#### Calcula a média aritmética da distância de manhattan entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento original. 

In [ ]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaSman = 0

# Percorre as sentenças do documento
for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Encontra os maiores embeddings os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSi, linha = torch.max(embeddingSi, dim=0)        
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSi:", len(maiorEmbeddingSi))
        
    # Encontra os maiores embeddings os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSj, linha = torch.max(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSj:", len(maiorEmbeddingSj)) 
  
    # Diferença entre os embeddings Si e Sj
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Sman = distanciaEManhattan(maiorEmbeddingSi, maiorEmbeddingSj)
    # Saída: Um número real
    
    # Acumula a medida
    somaSman = somaSman + Sman

DmanOriginal = float(somaSman)/float(n-1)
print("Dman Original:", DmanOriginal)

Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Dman Original: 199.8438466389974


#### Calcula a média aritmética da distância de manhattan entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento permutado.

In [ ]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaSman = 0

# Percorre as sentenças do documento
for i in range(np-1):
    # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Encontra os maiores embeddings os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSi, linha = torch.max(embeddingSi, dim=0)        
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSi:", len(maiorEmbeddingSi))
        
    # Encontra os maiores embeddings os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSj, linha = torch.max(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSj:", len(maiorEmbeddingSj)) 
  
    # Diferença entre os embeddings Si e Sj
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Sman = distanciaEManhattan(maiorEmbeddingSi, maiorEmbeddingSj)
    # Saída: Um número real
    
    # Acumula a medida
    somaSman = somaSman + Sman

DmanPermutado = float(somaSman)/float(np-1)
print("Deuc Original:", DmanPermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']
Quantidade de sentenças: 4
Deuc Original: 197.91803995768228


#### Compara as médias da distância de manhattan dos embeddings das sentenças do documento original e permutado

Características das medidas:
- Documentos com sentenças iguais resulta uma medida igual a 0.
- Documentos com sentenças diferenntes resulta uma medida maior que 0.
- Documento com sentenças muito diferentes apresentam valores maiores que 0.
- Documentos iguais resultam em medidas iguais. 
- É uma medida de diferença.



In [ ]:
print("Dman Original :", DmanOriginal)
print("Dman Permutado:", DmanPermutado)

if (DeucOriginal > DeucPermutado):
    print("Documento original tem maior distância de manhattan entre as sentenças!")
else:
    print("Documento Permutado tem menor distância de manhattan entre as sentenças!")

Dman Original : 199.8438466389974
Dman Permutado: 197.91803995768228
Documento original tem maior distância de manhattan entre as sentenças!


### Resumo

Resultado das medidas utilizando a última camada do BERT.

Base(MEAN):
- Dsub       :   0.00074918          -0.00064076
- Dubabs     :   0.17778097          0.17206367
- Dprod      :   43.70739746          43.55047607
- Dprodabs   :   43.70739746          43.55047607
- Dprode     :   43.70739746          43.55047607
- Dprodeabs  :   43.70739746          43.55047607
- Davg       :   -0.00276791          -0.00271371
- Dcos       :   0.67999101          0.69653825
- Deuc       :   6.34085274          6.15486606
- Dman       :   136.53579712          132.14489237

Base(MAX):
- Dsub       :   -0.02765538          0.02360132
- Dubabs     :   0.26021336          0.25770577
- Dprod      :   43.70739746          43.55047607
- Dprodabs   :   43.70739746          43.55047607
- Dprode     :   200.03959147          199.48846436
- Dprodeabs  :   200.03959147          199.48846436
- Davg       :   0.43032602          0.42829903
- Dcos       :   0.81970002          0.82297109
- Deuc       :   9.42460442          9.29481570
- Dman       :   199.84384664          197.91803996

In [ ]:
print("Resultado das medidas utilizando a última camada do BERT")
print("Documento  :   Original            Permutado")
print('Dsub       :   {:.8f}          {:.8f}'.format(DsubOriginal,DsubPermutado))
print('Dubabs     :   {:.8f}          {:.8f}'.format(DsubabsOriginal,DsubabsPermutado))
print('Dprod      :   {:.8f}          {:.8f}'.format(DprodOriginal,DprodPermutado))
print('Dprodabs   :   {:.8f}          {:.8f}'.format(DprodabsOriginal,DprodabsPermutado))
print('Dprode     :   {:.8f}          {:.8f}'.format(DprodeOriginal,DprodePermutado))
print('Dprodeabs  :   {:.8f}          {:.8f}'.format(DprodeabsOriginal,DprodeabsPermutado))
print('Davg       :   {:.8f}          {:.8f}'.format(DavgOriginal,DavgPermutado))
print('Dcos       :   {:.8f}          {:.8f}'.format(DcosOriginal,DcosPermutado))
print('Deuc       :   {:.8f}          {:.8f}'.format(DeucOriginal,DeucPermutado))
print('Dman       :   {:.8f}          {:.8f}'.format(DmanOriginal,DmanPermutado))

Resultado das medidas utilizando a última camada do BERT
Documento  :   Original            Permutado
Dsub       :   -0.02765538          0.02360132
Dubabs     :   0.26021336          0.25770577
Dprod      :   43.70739746          43.55047607
Dprodabs   :   43.70739746          43.55047607
Dprode     :   200.03959147          199.48846436
Dprodeabs  :   200.03959147          199.48846436
Davg       :   0.43032602          0.42829903
Dcos       :   0.81970002          0.82297109
Deuc       :   9.42460442          9.29481570
Dman       :   199.84384664          197.91803996


## 8 - Exemplo sentenças de documento original e permutado utilizando embedding a concatenação das 4 últimas camadas do BERT usando estratégia a MEAN

Como estamos utilizando os embeddings concatenado das 4 últimas camadas onde ocorre 768 entenda-se 3072 que é o resultado de 768 por 4 que é a dimensão do MCL BERT de tamanho base. E onde ocorre 1024 entenda-se 4096 que é o resultado de 1024 por 4 que é a dimensão do MCL BERT de tamanho large.

### Documento Original

In [ ]:
# Define um documento com 4 sentenças
documento_original = ["Bom Dia, professor.",
             "Qual o conteúdo da prova?",              
             "Vai cair tudo na prova?",
             "Aguardo uma resposta, João."]

# Concatena as sentenças do documento em uma string
stringDocumentoPermutado = ' '.join(documento_original)

# Adiciona os tokens especiais
documento_marcado_original = "[CLS] " + stringDocumentoPermutado + " [SEP]"

# Divide a sentença em tokens
documento_tokenizado_original = tokenizer.tokenize(documento_marcado_original)

# Mapeia os tokens em seus índices do vocabulário
documento_tokens_indexados_original = tokenizer.convert_tokens_to_ids(documento_tokenizado_original)

# Mostra os tokens com seus índices
i = 0
for tup in zip(documento_tokenizado_original, documento_tokens_indexados_original):
    print('{:>3} {:<12} {:>6,}'.format(i, tup[0], tup[1]))
    i = i + 1

  0 [CLS]           101
  1 Bom           8,399
  2 Dia           3,616
  3 ,               117
  4 professor     2,917
  5 .               119
  6 Qual         13,082
  7 o               146
  8 conteúdo      5,015
  9 da              180
 10 prova         2,310
 11 ?               136
 12 Vai          20,805
 13 cair          9,322
 14 tudo          2,745
 15 na              229
 16 prova         2,310
 17 ?               136
 18 Agu           8,125
 19 ##ardo        2,222
 20 uma             230
 21 resposta      4,299
 22 ,               117
 23 João          1,453
 24 .               119
 25 [SEP]           102


Máscara de atenção das palavras

In [ ]:
# Marca cada um dos tokens como pertencentes à sentença "1".
mascara_atencao_original = [1] * len(documento_tokenizado_original)

print (mascara_atencao_original)
print (len(mascara_atencao_original))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
26


Convertendo as listas em tensores

In [ ]:
# Importa a bibliteca
import torch

# Converte as entradas de listas para tensores do torch
tokens_tensores_original = torch.as_tensor([documento_tokens_indexados_original])
mascara_atencao_tensores_original = torch.as_tensor([mascara_atencao_original])

Gera os embeddings para o documento original. Guarda todas as camadas da rede em `outputs`.


In [ ]:
# Prediz os atributos dos estados ocultos para cada camada
with torch.no_grad():
    # output[0] contém last_hidden_states
    outputs = model(tokens_tensores_original, mascara_atencao_tensores_original)

Recupera a saída e concatena as 4 últimas camada do BERT

In [ ]:
# Cria uma lista com os tensores a serem concatenados
# Entrada: List das camadas(13 ou 25) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
# Lista com os tensores a serem concatenados
listaConcat = []
# Percorre os 4 últimos
for i in [-1,-2,-3,-4]:
    # Concatena da lista
    listaConcat.append(outputs[2][i])
    # Saída: Entrada: List das camadas(4) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
     #print("listaConcat=",len(listaConcat))

# Realiza a concatenação dos embeddings de todos as camadas
# Saída: Entrada: List das camadas(4) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
concat4_hidden_states = torch.cat(listaConcat, dim=-1)
# Saída: Entrada: (<1(lote)> x <qtde_tokens> <3072 ou 4096>)  

print ("O vetor da  concatenação das 4 últimas camadas oculta tem o formato:", concat4_hidden_states.size())

O vetor da  concatenação das 4 últimas camadas oculta tem o formato: torch.Size([1, 26, 3072])


Vamos nos livrar da dimensão lotes "batches", pois não precisamos dela.

In [ ]:
# Remove a dimensão 1, o lote "batches".
#O método squeeze remove a primeira dimensão(0) pois possui tamanho 1
embeddingDocumentoOriginal = torch.squeeze(concat4_hidden_states, dim=0)

print ("O vetor de tokens de embedding do documento original tem o formato:", embeddingDocumentoOriginal.size())

O vetor de tokens de embedding do documento original tem o formato: torch.Size([26, 3072])


Confirmando vetores dependentes do contexto


In [ ]:
for i, token_str in enumerate(documento_tokenizado_original):
  print (i, token_str)

0 [CLS]
1 Bom
2 Dia
3 ,
4 professor
5 .
6 Qual
7 o
8 conteúdo
9 da
10 prova
11 ?
12 Vai
13 cair
14 tudo
15 na
16 prova
17 ?
18 Agu
19 ##ardo
20 uma
21 resposta
22 ,
23 João
24 .
25 [SEP]


Exibe os embenddings das sentenças

In [ ]:
# Índice das sentenças a serem comparadas
sentenca1Original = documento_original[0]
sentenca2Original = documento_original[1]
sentenca3Original = documento_original[2]
sentenca4Original = documento_original[3]

embeddingSentenca1Original = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, sentenca1Original, tokenizer)
embeddingSentenca2Original = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, sentenca2Original, tokenizer)
embeddingSentenca3Original = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, sentenca3Original, tokenizer)
embeddingSentenca4Original = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, sentenca4Original, tokenizer)

print('Os primeiros 4 valores de cada sentença do documento original.')

print('\nSentença 1:', sentenca1Original,'-', str(embeddingSentenca1Original[:4]))
print('Soma embedding Sentença1:', sentenca1Original,'-', str(torch.sum(embeddingSentenca1Original[:4])))

print('\nSentença 2:', sentenca2Original,'-', str(embeddingSentenca2Original[:4]))
print('Soma embedding Sentença2:', sentenca2Original,'-', str(torch.sum(embeddingSentenca2Original[:4])))

print('\nSentença 3:', sentenca3Original,'-', str(embeddingSentenca3Original[:4]))
print('Soma embedding Sentença3:', sentenca3Original,'-', str(torch.sum(embeddingSentenca3Original[:4])))

print('\nSentença 4:', sentenca4Original,'-', str(embeddingSentenca4Original[:4]))
print('Soma embedding Sentença4:', sentenca4Original,'-', str(torch.sum(embeddingSentenca4Original[:4])))

Os primeiros 4 valores de cada sentença do documento original.

Sentença 1: Bom Dia, professor. - tensor([[-0.0680, -0.4615,  0.3552,  ...,  0.1657,  0.3478,  0.4525],
        [-0.1000, -0.0630,  0.0840,  ..., -0.3926,  0.7971,  0.3287],
        [-0.3165,  0.4208,  0.2178,  ..., -0.3834,  0.9520, -0.2181],
        [ 0.1248,  0.2383,  0.8987,  ..., -0.1989,  0.1001,  0.1140]])
Soma embedding Sentença1: Bom Dia, professor. - tensor(-327.0100)

Sentença 2: Qual o conteúdo da prova? - tensor([[-0.5894, -0.4310,  0.1449,  ...,  0.2399, -0.8495, -0.1672],
        [ 0.1349, -0.2476,  0.4605,  ..., -0.1353, -0.1674,  0.4057],
        [ 0.4359, -0.6972,  0.4066,  ...,  0.5851, -0.0441, -0.0027],
        [ 0.0544,  0.1606,  0.4150,  ..., -0.8060, -0.3698,  0.0466]])
Soma embedding Sentença2: Qual o conteúdo da prova? - tensor(-315.7253)

Sentença 3: Vai cair tudo na prova? - tensor([[-0.3652, -0.5015, -0.1626,  ..., -0.5431, -0.5805,  0.4617],
        [ 0.1421,  0.1797, -0.0014,  ..., -0.7004, -

Examinando os embeddings do documento original



In [ ]:
# Índice das sentenças a serem comparadas
sentenca1Original = documento_original[0]
sentenca2Original = documento_original[1]
sentenca3Original = documento_original[2]
sentenca4Original = documento_original[3]

print("Documento Original:", documento_original)

# Localiza os índices dos tokens da sentença no documento
sentenca1TokenizadaOriginal = tokenizer.tokenize(sentenca1Original)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_original,sentenca1TokenizadaOriginal)
embeddingSentenca1Original = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, sentenca1Original, tokenizer)
print('\nSentença 1 Original=\'', sentenca1Original, '\'')
print('    Sentença tokenizada:', sentenca1TokenizadaOriginal)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca1Original.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca1Original))

# Localiza os índices dos tokens da sentença no documento
sentenca2TokenizadaOriginal = tokenizer.tokenize(sentenca2Original)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_original,sentenca2TokenizadaOriginal)
embeddingSentenca2Original = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, sentenca2Original, tokenizer)
print('\nSentença 2 Original=\'', sentenca2Original, '\'')
print('    Sentença tokenizada:', sentenca2TokenizadaOriginal)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca2Original.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca2Original))

# Localiza os índices dos tokens da sentença no documento
sentenca3TokenizadaOriginal = tokenizer.tokenize(sentenca3Original)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_original,sentenca3TokenizadaOriginal)
embeddingSentenca3Original = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, sentenca3Original, tokenizer)
print('\nSentença 3 Original=\'', sentenca3Original, '\'')
print('    Sentença tokenizada:', sentenca3TokenizadaOriginal)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca3Original.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca3Original))

# Localiza os índices dos tokens da sentença no documento
sentenca4TokenizadaOriginal = tokenizer.tokenize(sentenca4Original)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_original,sentenca4TokenizadaOriginal)
embeddingSentenca4Original = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, sentenca4Original, tokenizer)
print('\nSentença 4 Original=\'', sentenca4Original, '\'')
print('    Sentença tokenizada:', sentenca4TokenizadaOriginal)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca4Original.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca4Original))


Documento Original: ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']

Sentença 1 Original=' Bom Dia, professor. '
    Sentença tokenizada: ['Bom', 'Dia', ',', 'professor', '.']
    => inicio em 1 e término em 5
    Formato modelo : torch.Size([5, 3072])
    Soma embeddings:  -398.53

Sentença 2 Original=' Qual o conteúdo da prova? '
    Sentença tokenizada: ['Qual', 'o', 'conteúdo', 'da', 'prova', '?']
    => inicio em 6 e término em 11
    Formato modelo : torch.Size([6, 3072])
    Soma embeddings:  -467.44

Sentença 3 Original=' Vai cair tudo na prova? '
    Sentença tokenizada: ['Vai', 'cair', 'tudo', 'na', 'prova', '?']
    => inicio em 12 e término em 17
    Formato modelo : torch.Size([6, 3072])
    Soma embeddings:  -466.27

Sentença 4 Original=' Aguardo uma resposta, João. '
    Sentença tokenizada: ['Agu', '##ardo', 'uma', 'resposta', ',', 'João', '.']
    => inicio em 18 e término em 24
    Formato modelo : torch.S

### Documento Permutado

In [ ]:
# Define um documento com a permutação das sentenças do documento original
documento_permutado = [documento_original[3],   # "Aguardo uma resposta, João.",
             documento_original[1],             # "Qual o conteúdo da prova?",              
             documento_original[0],             # "Vai cair tudo na prova?",
             documento_original[2]]             # "Bom Dia, professor."]     

# Use o documento permutado igual ao original para testar se as medidas estão corretas
#documento_permutado = documento_original

# Concatena as sentenças do documento em uma string
stringDocumentoPermutado = ' '.join(documento_permutado)

# Adiciona os tokens especiais
documento_marcado_permutado = "[CLS] " + stringDocumentoPermutado + " [SEP]"

# Divide a sentença em tokens
documento_tokenizado_permutado = tokenizer.tokenize(documento_marcado_permutado)

# Mapeia os tokens em seus índices do vocabulário
documento_tokens_indexados_permutado = tokenizer.convert_tokens_to_ids(documento_tokenizado_permutado)

# Mostra os tokens com seus índices
i = 0
for tup in zip(documento_tokenizado_permutado, documento_tokens_indexados_permutado):
    print('{:>3} {:<12} {:>6,}'.format(i, tup[0], tup[1]))
    i = i + 1

  0 [CLS]           101
  1 Agu           8,125
  2 ##ardo        2,222
  3 uma             230
  4 resposta      4,299
  5 ,               117
  6 João          1,453
  7 .               119
  8 Qual         13,082
  9 o               146
 10 conteúdo      5,015
 11 da              180
 12 prova         2,310
 13 ?               136
 14 Bom           8,399
 15 Dia           3,616
 16 ,               117
 17 professor     2,917
 18 .               119
 19 Vai          20,805
 20 cair          9,322
 21 tudo          2,745
 22 na              229
 23 prova         2,310
 24 ?               136
 25 [SEP]           102


Máscara de atenção das palavras

In [ ]:
# Marca cada um dos tokens como pertencentes à sentença "1".
mascara_atencao_permutado = [1] * len(documento_tokenizado_permutado)

print (mascara_atencao_permutado)
print (len(mascara_atencao_permutado))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
26


Convertendo as listas em tensores

In [ ]:
# Importa a bibliteca
import torch

# Converte as entradas de listas para tensores do torch
tokens_tensores_permutado = torch.as_tensor([documento_tokens_indexados_permutado])
mascara_atencao_tensores_permutado = torch.as_tensor([mascara_atencao_permutado])

Gera os embeddings para o documento original. Guarda todas as camadas da rede em `outputs`.


In [ ]:
# Prediz os atributos dos estados ocultos para cada camada
with torch.no_grad():
    # output[0] contém last_hidden_states
    outputs = model(tokens_tensores_permutado, mascara_atencao_tensores_permutado)

Recupera a saída e concatena as 4 últimas camada do BERT

In [ ]:
# Cria uma lista com os tensores a serem concatenados
# Entrada: List das camadas(13 ou 25) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
# Lista com os tensores a serem concatenados
listaConcat = []
# Percorre os 4 últimos
for i in [-1,-2,-3,-4]:
    # Concatena da lista
    listaConcat.append(outputs[2][i])
    # Saída: Entrada: List das camadas(4) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
     #print("listaConcat=",len(listaConcat))

# Realiza a concatenação dos embeddings de todos as camadas
# Saída: Entrada: List das camadas(4) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
concat4_hidden_states = torch.cat(listaConcat, dim=-1)
# Saída: Entrada: (<1(lote)> x <qtde_tokens> <3072 ou 4096>)  

Vamos nos livrar da dimensão lotes "batches", pois não precisamos dela.

In [ ]:
# Remove a dimensão 1, o lote "batches".
#O método squeeze remove a primeira dimensão(0) pois possui tamanho 1
embeddingDocumentoPermutado = torch.squeeze(concat4_hidden_states, dim=0)

print ("O vetor de tokens de embedding do documento permutado tem o formato:", embeddingDocumentoPermutado.size())

O vetor de tokens de embedding do documento permutado tem o formato: torch.Size([26, 3072])


Exibe os embenddings das sentenças

In [ ]:
# Índice das sentenças a serem comparadas
sentenca1Permutado = documento_permutado[0]
sentenca2Permutado = documento_permutado[1]
sentenca3Permutado = documento_permutado[2]
sentenca4Permutado = documento_permutado[3]

embeddingSentenca1Permutado = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoPermutado, stringDocumentoPermutado, sentenca1Permutado, tokenizer)
embeddingSentenca2Permutado = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoPermutado, stringDocumentoPermutado, sentenca2Permutado, tokenizer)
embeddingSentenca3Permutado = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoPermutado, stringDocumentoPermutado, sentenca3Permutado, tokenizer)
embeddingSentenca4Permutado = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoPermutado, stringDocumentoPermutado, sentenca4Permutado, tokenizer)

print('Os primeiros 4 valores de cada sentença do documento permutado.')

print('\nSentença 1:', sentenca1Permutado,'-', str(embeddingSentenca1Permutado[:4]))
print('Soma embedding Sentença1:', sentenca1Original,'-', str(torch.sum(embeddingSentenca1Original[:4])))

print('\nSentença 2:', sentenca2Permutado,'-', str(embeddingSentenca2Permutado[:4]))
print('Soma embedding Sentença2:', sentenca2Permutado,'-', str(torch.sum(embeddingSentenca2Permutado[:4])))

print('\nSentença 3:', sentenca3Permutado,'-', str(embeddingSentenca3Permutado[:4]))
print('Soma embedding Sentença3:', sentenca3Permutado,'-', str(torch.sum(embeddingSentenca3Original[:4])))

print('\nSentença 4:', sentenca4Permutado,'-', str(embeddingSentenca4Permutado[:4]))
print('Soma embedding Sentença4:', sentenca4Permutado,'-', str(torch.sum(embeddingSentenca4Permutado[:4])))

Os primeiros 4 valores de cada sentença do documento permutado.

Sentença 1: Aguardo uma resposta, João. - tensor([[-0.0052, -0.4706,  0.6481,  ..., -0.1775,  0.2231,  0.8036],
        [-0.1516, -0.2450,  0.2693,  ..., -0.3783,  0.1529,  0.5125],
        [-0.7358, -0.0988,  0.5145,  ...,  0.0164, -0.1749,  0.2557],
        [-0.1278,  0.3571,  0.5481,  ..., -0.2708,  0.1942, -0.0740]])
Soma embedding Sentença1: Bom Dia, professor. - tensor(-327.0100)

Sentença 2: Qual o conteúdo da prova? - tensor([[-0.4295, -0.3426,  0.1005,  ...,  0.2608, -0.8301, -0.1044],
        [ 0.1047, -0.1946,  0.4177,  ..., -0.1734, -0.2517,  0.4004],
        [ 0.4377, -0.5952,  0.5448,  ...,  0.4539, -0.1388, -0.0339],
        [ 0.0898,  0.1063,  0.4610,  ..., -0.5308, -0.4884,  0.2119]])
Soma embedding Sentença2: Qual o conteúdo da prova? - tensor(-314.9913)

Sentença 3: Bom Dia, professor. - tensor([[-0.0458, -0.4268,  0.2399,  ...,  0.4282,  0.3110,  0.0241],
        [ 0.0650, -0.1385,  0.0230,  ..., -0.36

In [ ]:
# Índice das sentenças a serem comparadas
sentenca1Permutado = documento_permutado[0]
sentenca2Permutado = documento_permutado[1]
sentenca3Permutado = documento_permutado[2]
sentenca4Permutado = documento_permutado[3]

print("Documento Permutado:", documento_permutado)

# Localiza os índices dos tokens da sentença no documento
sentenca1TokenizadaPermutado = tokenizer.tokenize(sentenca1Permutado)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_permutado,sentenca1TokenizadaPermutado)
embeddingSentenca1Permutado = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoPermutado, stringDocumentoPermutado, sentenca1Permutado, tokenizer)
print('\nSentença 1 Permutada=\'', sentenca1Permutado, '\'')
print('    Sentença tokenizada:', sentenca1TokenizadaPermutado)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca1Permutado.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca1Permutado))

# Localiza os índices dos tokens da sentença no documento
sentenca2TokenizadaPermutado = tokenizer.tokenize(sentenca2Permutado)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_permutado,sentenca2TokenizadaPermutado)
embeddingSentenca2Permutado = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoPermutado, stringDocumentoPermutado, sentenca2Permutado, tokenizer)
print('\nSentença 2 Permutada=\'', sentenca2Permutado, '\'')
print('    Sentença tokenizada:', sentenca2TokenizadaPermutado)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca2Permutado.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca2Permutado))

# Localiza os índices dos tokens da sentença no documento
sentenca3TokenizadaPermutado = tokenizer.tokenize(sentenca3Permutado)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_permutado,sentenca3TokenizadaPermutado)
embeddingSentenca3Permutado = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoPermutado, stringDocumentoPermutado, sentenca3Permutado, tokenizer)
print('\nSentença 3 Permutada=\'', sentenca3Permutado, '\'')
print('    Sentença tokenizada:', sentenca3TokenizadaPermutado)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca3Permutado.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca3Permutado))

# Localiza os índices dos tokens da sentença no documento
sentenca4TokenizadaPermutado = tokenizer.tokenize(sentenca4Permutado)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_permutado,sentenca4TokenizadaPermutado)
embeddingSentenca4Permutado = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoPermutado, stringDocumentoPermutado, sentenca4Permutado, tokenizer)
print('\nSentença 4 Permutada=\'', sentenca4Permutado, '\'')
print('    Sentença tokenizada:', sentenca4TokenizadaPermutado)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca4Permutado.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca4Permutado))


Documento Permutado: ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']

Sentença 1 Permutada=' Aguardo uma resposta, João. '
    Sentença tokenizada: ['Agu', '##ardo', 'uma', 'resposta', ',', 'João', '.']
    => inicio em 1 e término em 7
    Formato modelo : torch.Size([7, 3072])
    Soma embeddings:  -532.56

Sentença 2 Permutada=' Qual o conteúdo da prova? '
    Sentença tokenizada: ['Qual', 'o', 'conteúdo', 'da', 'prova', '?']
    => inicio em 8 e término em 13
    Formato modelo : torch.Size([6, 3072])
    Soma embeddings:  -467.65

Sentença 3 Permutada=' Bom Dia, professor. '
    Sentença tokenizada: ['Bom', 'Dia', ',', 'professor', '.']
    => inicio em 14 e término em 18
    Formato modelo : torch.Size([5, 3072])
    Soma embeddings:  -402.97

Sentença 4 Permutada=' Vai cair tudo na prova? '
    Sentença tokenizada: ['Vai', 'cair', 'tudo', 'na', 'prova', '?']
    => inicio em 19 e término em 24
    Formato modelo : to

### Examinando as sentenças

A mesma sentença apresenta embeddings com valores diferentes, pois se encontram em locais diferentes do documento. A soma de todos os embeddings demonstra isto.

In [ ]:
print('\nSentença 4 Original=\'', sentenca4Original, '\'')
print('    Sentença tokenizada:', sentenca4TokenizadaOriginal)
print('    Formato modelo :', embeddingSentenca4Original.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca4Original))
print('    Os 4 primeiros embeddings:', str(embeddingSentenca4Original[:4]))

print('\nSentença 1 Permutada=\'', sentenca1Permutado, '\'')
print('    Sentença tokenizada:', sentenca1TokenizadaPermutado)
print('    Formato modelo :', embeddingSentenca1Permutado.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca1Permutado))
print('    Os 4 primeiros embeddings:', str(embeddingSentenca1Permutado[:4]))


Sentença 4 Original=' Aguardo uma resposta, João. '
    Sentença tokenizada: ['Agu', '##ardo', 'uma', 'resposta', ',', 'João', '.']
    Formato modelo : torch.Size([7, 3072])
    Soma embeddings:  -525.26
    Os 4 primeiros embeddings: tensor([[-0.0835, -0.4042,  0.4330,  ...,  0.2881,  0.1448,  0.7778],
        [-0.2782, -0.1630,  0.0997,  ..., -0.0102,  0.1851,  0.4587],
        [-0.5474, -0.0118,  0.3950,  ...,  0.1932, -0.0735,  0.2649],
        [ 0.0146,  0.2901,  0.3920,  ..., -0.0370,  0.3236, -0.3671]])

Sentença 1 Permutada=' Aguardo uma resposta, João. '
    Sentença tokenizada: ['Agu', '##ardo', 'uma', 'resposta', ',', 'João', '.']
    Formato modelo : torch.Size([7, 3072])
    Soma embeddings:  -532.56
    Os 4 primeiros embeddings: tensor([[-0.0052, -0.4706,  0.6481,  ..., -0.1775,  0.2231,  0.8036],
        [-0.1516, -0.2450,  0.2693,  ..., -0.3783,  0.1529,  0.5125],
        [-0.7358, -0.0988,  0.5145,  ...,  0.0164, -0.1749,  0.2557],
        [-0.1278,  0.3571,  0.5481

### Subtração entre os embeddings das sentenças

#### Calcula a média aritmética da subtração entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento original. 



In [ ]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaSsub = 0

# Percorre as sentenças do documento
for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSi=", mediaEmbeddingSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSj=", mediaEmbeddingSj.shape)
  
    # Subtração entre os embeddings de Si e Sj
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Ssub = torch.sub(mediaEmbeddingSi, mediaEmbeddingSj)
    # Saída: <768 ou 1024>  
    #print("Ssub=", Ssub.shape)

    # Calcula a média dos embeddings do subtração para realizar a soma
    somaSsub = somaSsub + torch.mean(Ssub)

DsubOriginal = float(somaSsub)/float(len(documento_original)-1)
print("Ssub Original:", DsubOriginal)
    


Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Ssub Original: 0.0002815294816779594


#### Calcula a média aritmética da subtração entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento permutado. 

In [ ]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaSsub = 0

# Percorre as sentenças do documento
for i in range(np-1):
    # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSi=", mediaEmbeddingSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSj=", mediaEmbeddingSj.shape)
  
    # Subtração entre os embeddings de Si e Sj
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Ssub = torch.sub(mediaEmbeddingSi, mediaEmbeddingSj)
    # Saída: <768 ou 1024>  
    #print("Ssub=", Ssub.shape)

    # Calcula a média dos embeddings do subtração para realizar a soma
    somaSsub = somaSsub + torch.mean(Ssub)

DsubPermutado = float(somaSsub)/float(np-1)
print("Ssub permutado:", DsubPermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']
Quantidade de sentenças: 4
Ssub permutado: -0.0005001813018073639


#### Compara as médias da subtração dos embeddings das sentenças do documento original e permutado.

Características das medidas:
- Documentos com sentenças iguais resulta uma medida igual a 0.
- Documentos com sentenças diferenntes resulta uma medida maior que 0.
- Documento com sentenças muito diferentes apresentam valores maiores que 0.
- Documentos iguais resultam em medidas iguais. 
- É uma medida de diferença.


In [ ]:
print("Dsub Original :", DsubOriginal)
print("Dsub Permutado:", DsubPermutado)

if (DsubOriginal < DsubPermutado):
    print("Documento original tem menor subtração entre as sentenças!")
else:
    print("Documento Permutado tem menor subtração entre as sentenças!")

Dsub Original : 0.0002815294816779594
Dsub Permutado: -0.0005001813018073639
Documento Permutado tem menor subtração entre as sentenças!


### Subtração absoluta entre os embeddings das sentenças

#### Calcula a média aritmética da subtração absoluta entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento original. 

In [ ]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaSsubabs = 0

# Percorre as sentenças do documento
for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSi=", mediaEmbeddingSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSj=", mediaEmbeddingSj.shape)
  
    # Subtração entre os embeddings de Si e Sj
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Ssubabs = abs(torch.sub(mediaEmbeddingSi, mediaEmbeddingSj))
    # Saída: <768 ou 1024>  
    #print("Ssubabs=", Ssubabs.shape)

    # Calcula a média dos embeddings do subtração para realizar a soma
    somaSsubabs = somaSsubabs + torch.mean(Ssubabs)

DsubabsOriginal = float(somaSsubabs)/float(n-1)
print("Dsubabs Original:", DsubabsOriginal)
    


Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Dsubabs Original: 0.26571635405222577


#### Calcula a média aritmética da subtração absoluta entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento permutado. 

In [ ]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaSsubabs = 0

# Percorre as sentenças do documento
for i in range(np-1):
    # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSi=", mediaEmbeddingSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSj=", mediaEmbeddingSj.shape)
  
    # Subtração entre os embeddings de Si e Sj
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Ssubabs = abs(torch.sub(mediaEmbeddingSi, mediaEmbeddingSj))
    # Saída: <768 ou 1024>  
    #print("Ssubabs=", Ssubabs.shape)

    # Calcula a média dos embeddings do subtração para realizar a soma
    somaSsubabs = somaSsubabs + torch.mean(Ssubabs)

DsubabsPermutado = float(somaSsubabs)/float(np-1)
print("Dsubabs permutado:", DsubabsPermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']
Quantidade de sentenças: 4
Dsubabs permutado: 0.24592039982477823


#### Compara as médias da subtração absoluta dos embeddings das sentenças do documento original e permutado

Características das medidas:
- Documentos com sentenças iguais resulta uma medida igual a 0.
- Documentos com sentenças diferenntes resulta uma medida maior que 0.
- Documento com sentenças muito diferentes apresentam valores maiores que 0.
- Documentos iguais resultam em medidas iguais. 
- É uma medida de diferença.

In [ ]:
print("Dsubabs Original :", DsubabsOriginal)
print("Dsubabs Permutado:", DsubabsPermutado)

if (DsubabsOriginal < DsubabsPermutado):
    print("Documento original tem menor subtração absoluta entre as sentenças!")
else:
    print("Documento Permutado tem menor subtração absoluta entre as sentenças!")

Dsubabs Original : 0.26571635405222577
Dsubabs Permutado: 0.24592039982477823
Documento Permutado tem menor subtração absoluta entre as sentenças!


### Produto entre os embeddings das sentenças

#### Calcula a média aritmética do produto das matrizes dos tokens das sentenças do documento original. 

In [ ]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaSprod = 0

# Percorre as sentenças do documento
for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Para a multiplicação pode ocorrer número de colunas(qtde_tokens) de Si tem que ser igual ao número de linhas(embeddings) de Sj.
    # Permute realiza a troca da dimensão 1(qtde_embeddings) pela 0(qtde_tokesn) na sentença Sj para que seja possível realizar o produto. 
    # Ocorre a troca da qtde_tokens pela quantidade de embeddings.
    # Entrada: (<qtde_tokensSi> x <768 ou 1024>) x (<qtde_tokensSj> x <768 ou 1024>)
    # Permute: <qtde_tokensSi> x <768 ou 1024>) x (<768 ou 1024> x <qtde_tokensSj>)
    Sprod = torch.matmul(embeddingSi, embeddingSj.permute(1,0))    
    # Saída: <qtde_tokensSi> x  <qtde_tokensSj    #print("Sprod=", Sprod.shape)
    
    # Calcula a média dos embeddings do produto para realizar a soma
    somaSprod = somaSprod + torch.mean(Sprod)

DprodOriginal = float(somaSprod)/float(n-1)
print("Dprod Original:", DprodOriginal)
    


Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Dprod Original: 932.7980143229166


#### Calcula a média aritmética do produto das matrizes dos tokens das sentenças do documento permutado. 

In [ ]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaSprod = 0

# Percorre as sentenças do documento
for i in range(np-1):
    # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)
 
    # Produto entre os embeddings de Si e Sj    
    # Para a multiplicação pode ocorrer número de colunas(qtde_tokens) de Si tem que ser igual ao número de linhas(embeddings) de Sj.
    # Permute realiza a troca da dimensão 1(qtde_embeddings) pela 0(qtde_tokesn) na sentença Sj para que seja possível realizar o produto. 
    # Ocorre a troca da qtde_tokens pela quantidade de embeddings.
    # Entrada: (<qtde_tokensSi> x <768 ou 1024>) x (<qtde_tokensSj> x <768 ou 1024>)
    # Permute: <qtde_tokensSi> x <768 ou 1024>) x (<768 ou 1024> x <qtde_tokensSj>)
    Sprod = torch.matmul(embeddingSi, embeddingSj.permute(1,0))    
    # Saída: <qtde_tokensSi> x  <qtde_tokensSj>
    #print("Sprod=", Sprod.shape)
        
    # Calcula a média dos embeddings do produto para realizar a soma
    somaSprod = somaSprod + torch.mean(Sprod)

DprodPermutado = float(somaSprod)/float(np-1)
print("Dprod permutado:", DprodPermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']
Quantidade de sentenças: 4
Dprod permutado: 942.978515625


#### Compara as médias do produto dos embeddings das sentenças do documento original e permutado



In [ ]:
print("Dprod Original :", DprodOriginal)
print("Dprod Permutado:", DprodPermutado)

if (DprodOriginal < DprodPermutado):
    print("Documento original tem maior similaridade com o produto entre as sentenças!")
else:
    print("Documento Permutado tem menor similaridade com o produto entre as sentenças!")

Dprod Original : 932.7980143229166
Dprod Permutado: 942.978515625
Documento original tem maior similaridade com o produto entre as sentenças!


### Produto Absoluto entre os embeddings das sentenças

#### Calcula a média aritmética do produto das matrizes entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento original. 

In [ ]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaSprodabs = 0

# Percorre as sentenças do documento
for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)
    
    # Produto entre os embeddings de Si e Sj
    # Entrada: <qtde_tokens> x <768 ou 1024>
    # Para a multiplicação pode ocorrer número de colunas(qtde_tokens) de Si tem que ser igual ao número de linhas(embeddings) de Sj.
    # Permute realiza a troca da dimensão 1(qtde_embeddings) pela 0(qtde_tokesn) na sentença Sj para que seja possível realizar o produto. 
    # Ocorre a troca da qtde_tokens pela quantidade de embeddings.
    # Entrada: (<qtde_tokensSi> x <768 ou 1024>) x (<qtde_tokensSj> x <768 ou 1024>)
    # Permute: <qtde_tokensSi> x <768 ou 1024>) x (<768 ou 1024> x <qtde_tokensSj>)
    Sprodabs = abs(torch.matmul(embeddingSi, embeddingSj.permute(1,0)))   
    # Saída: <qtde_tokensSi> x  <qtde_tokensSj>
    #print("Sprodabs=", Sprodabs.shape)

    # Calcula a média dos embeddings do produto para realizar a soma
    somaSprodabs = somaSprodabs + torch.mean(Sprodabs)

DprodabsOriginal = float(somaSprodabs)/float(n-1)
print("Dprodabs Original:", DprodabsOriginal)

Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Dprodabs Original: 932.7980143229166


#### Calcula a média aritmética do produto das matrizes entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento permutado. 

In [ ]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaSprodabs = 0

# Percorre as sentenças do documento
for i in range(np-1):
    # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)
 
    # Produto entre os embeddings de Si e Sj
    # Entrada: <qtde_tokens> x <768 ou 1024>
    # Para a multiplicação pode ocorrer número de colunas(qtde_tokens) de Si tem que ser igual ao número de linhas(embeddings) de Sj.
    # Permute realiza a troca da dimensão 1(qtde_embeddings) pela 0(qtde_tokesn) na sentença Sj para que seja possível realizar o produto. 
    # Ocorre a troca da qtde_tokens pela quantidade de embeddings.
    # Entrada: (<qtde_tokensSi> x <768 ou 1024>) x (<qtde_tokensSj> x <768 ou 1024>)
    # Permute: <qtde_tokensSi> x <768 ou 1024>) x (<768 ou 1024> x <qtde_tokensSj>)
    Sprodabs = abs(torch.matmul(embeddingSi, embeddingSj.permute(1,0)))    
    # Saída: <qtde_tokensSi> x  <qtde_tokensSj>
    #print("Sprodabs=", Sprodabs.shape)

    # Calcula a média dos embeddings do produto para realizar a soma
    somaSprodabs = somaSprodabs + torch.mean(Sprodabs)

DprodabsPermutado = float(somaSprodabs)/float(np-1)
print("Dprodabs permutado:", DprodabsPermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']
Quantidade de sentenças: 4
Dprodabs permutado: 942.978515625


#### Compara as médias do produto absoluto dos embeddings das sentenças do documento original e permutado


In [ ]:
print("Dprodabs Original :", DprodabsOriginal)
print("Dprodabs Permutado:", DprodabsPermutado)

if (DprodabsOriginal < DprodabsPermutado):
    print("Documento original tem maior similaridade com o produto absoluto entre as sentenças!")
else:
    print("Documento Permutado tem menor similaridade com o produto absoluto entre as sentenças!")

Dprodabs Original : 932.7980143229166
Dprodabs Permutado: 942.978515625
Documento original tem maior similaridade com o produto absoluto entre as sentenças!


### Produto Escalar entre os embeddings das sentenças

#### Calcula a média aritmética do produto escalar entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento orginal. 



In [ ]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaSprode = 0

# Percorre as sentenças do documento
for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSi=", mediaEmbeddingSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSj=", mediaEmbeddingSj.shape)

    # Produto escalar entre os embeddings de Si e Sj, pois Si e Sj possui uma única dimensão
    # https://pytorch.org/docs/master/generated/torch.matmul.html#torch.matmul
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Sprode = torch.matmul(mediaEmbeddingSi, mediaEmbeddingSj)
    # Saída: Um número real 
    #print("Sprode=", Sprode.shape)
        
    somaSprode = somaSprode + Sprode

DprodeOriginal = float(somaSprode)/float(n-1)
print("Dprode Original:", DprodeOriginal)

Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Dprode Original: 932.798095703125


#### Calcula a média aritmética do produto escalar entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento permutado. 

In [ ]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaSprode = 0

# Percorre as sentenças do documento
for i in range(np-1):
    # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSi=", mediaEmbeddingSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSj=", mediaEmbeddingSj.shape)
  
    # Produto escalar entre os embeddings de Si e Sj, pois Si e Sj possui uma única dimensão
    # https://pytorch.org/docs/master/generated/torch.matmul.html#torch.matmul
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Sprode = torch.matmul(mediaEmbeddingSi, mediaEmbeddingSj)    
    # Saída: Um número real 
    #print("Sprode=", Sprode)
        
    somaSprode = somaSprode + Sprode

DprodePermutado = float(somaSprode)/float(np-1)
print("Dprod permutado:", DprodePermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']
Quantidade de sentenças: 4
Dprod permutado: 942.9785970052084


#### Compara as médias do produto escalar dos embeddings das sentenças do documento original e permutado

Dprod Original : 230.61181640625
Dprod Permutado: 227.75482177734375
Documento original tem maior similaridade com o produto entre as sentenças!

In [ ]:
print("Dprode Original :", DprodeOriginal)
print("Dprode Permutado:", DprodePermutado)

if (DprodeOriginal < DprodePermutado):
    print("Documento original tem maior similaridade com o produto entre as sentenças!")
else:
    print("Documento Permutado tem menor similaridade com o produto entre as sentenças!")

Dprode Original : 932.798095703125
Dprode Permutado: 942.9785970052084
Documento original tem maior similaridade com o produto entre as sentenças!


### Produto Escalar Absoluto entre os embeddings das sentenças

#### Calcula a média aritmética do produto escalar absoluto entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento original. 

In [ ]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaSprodeabs = 0

# Percorre as sentenças do documento
for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSi=", mediaEmbeddingSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSj=", mediaEmbeddingSj.shape)
  
    # Produto escalar entre os embeddings de Si e Sj, pois Si e Sj possui uma única dimensão
    # https://pytorch.org/docs/master/generated/torch.matmul.html#torch.matmul
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Sprodeabs = torch.matmul(mediaEmbeddingSi, mediaEmbeddingSj)
    # Saída: Um número real 
    #print("Sprodeabs=", Sprodeabs.shape)

    somaSprodeabs = somaSprodeabs + Sprodeabs

DprodeabsOriginal = float(somaSprodeabs)/float(n-1)
print("Dprodabs Original:", DprodeabsOriginal)

Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Dprodabs Original: 932.798095703125


#### Calcula a média aritmética do produto escalar absoluto entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento permutado. 

In [ ]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaSprodeabs = 0

# Percorre as sentenças do documento
for i in range(np-1):
    # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)
    
    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSi=", mediaEmbeddingSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSj=", mediaEmbeddingSj.shape)
    
    # Produto escalar entre os embeddings de Si e Sj, pois Si e Sj possui uma única dimensão
    # https://pytorch.org/docs/master/generated/torch.matmul.html#torch.matmul
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Sprodeabs = torch.matmul(mediaEmbeddingSi, mediaEmbeddingSj)
    # Saída: Um número real 
    #print("Sprodeabs=", Sprodeabs.shape)
    
    somaSprodeabs = somaSprodeabs + Sprodeabs

DprodeabsPermutado = float(somaSprodeabs)/float(np-1)
print("Dprodabs permutado:", DprodeabsPermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']
Quantidade de sentenças: 4
Dprodabs permutado: 942.9785970052084


#### Compara as médias do produto escalar absoluto dos embeddings das sentenças do documento original e permutado

In [ ]:
print("Dprodabs Original :", DprodeabsOriginal)
print("Dprodabs Permutado:", DprodeabsPermutado)

if (DprodeabsOriginal < DprodeabsPermutado):
    print("Documento original tem maior similaridade com o produto absoluto entre as sentenças!")
else:
    print("Documento Permutado tem menor similaridade com o produto absoluto entre as sentenças!")

Dprodabs Original : 932.798095703125
Dprodabs Permutado: 942.9785970052084
Documento original tem maior similaridade com o produto absoluto entre as sentenças!


### Média entre os embeddings das sentenças

#### Calcula a média aritmética entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento original. 

In [ ]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaSavg = 0

# Percorre as sentenças do documento
for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSi=", mediaEmbeddingSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSj=", mediaEmbeddingSj.shape)
  
    # Média entre os embeddings de Si e Sj
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Savg = (mediaEmbeddingSi.add(mediaEmbeddingSj))/2.0  
    # Saída: <768 ou 1024>
    #print("Savg=", Savg.shape)  

    # Calcula a média dos embeddings do cálculo para realizar a soma
    somaSavg = somaSavg + torch.mean(Savg)

DavgOriginal = float(somaSavg)/float(n-1)
print("Davg Original:", DavgOriginal)

Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Davg Original: -0.02508241931597392


#### Calcula a média aritmética entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento permutado.

In [ ]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaSavg = 0

# Percorre as sentenças do documento
for i in range(np-1):
    # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSi=", mediaEmbeddingSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSj=", mediaEmbeddingSj.shape)
  
    # Média entre os embeddings de Si e Sj
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)  
    Savg = (mediaEmbeddingSi.add(mediaEmbeddingSj))/2.0  
    # Saída: <768 ou 1024>
    #print("Savg=", Savg.shape)  

    # Calcula a média dos embeddings do cálculo para realizar a soma
    somaSavg = somaSavg + torch.mean(Savg)

DavgPermutado = float(somaSavg)/float(np-1)
print("Davg permutado:", DavgPermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']
Quantidade de sentenças: 4
Davg permutado: -0.025191746652126312


#### Compara as médias dos embeddings das sentenças do documento original e permutado


In [ ]:
print("Davg Original :", DavgOriginal)
print("Davg Permutado:", DavgPermutado)

if (DavgOriginal < DavgPermutado):
    print("Documento original tem maior similaridade com a média entre as sentenças!")
else:
    print("Documento Permutado tem menor similaridade com a mpedia entre as sentenças!")

Davg Original : -0.02508241931597392
Davg Permutado: -0.025191746652126312
Documento Permutado tem menor similaridade com a mpedia entre as sentenças!


### Similaridade de cosseno entre os embeddings das sentenças

In [ ]:
# Import das bibliotecas.
from scipy.spatial.distance import cosine

def similaridadeCoseno(sentenca1, sentenca2):
  similaridade = 1 - cosine(sentenca1, sentenca2)
  return similaridade

#### Calcula a média aritmética da similaridade do coseno entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento original. 

In [ ]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaScos = 0

# Percorre as sentenças do documento
for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSi=", mediaEmbeddingSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSj=", mediaEmbeddingSj.shape)
  
    # Similaridade entre os embeddings Si e Sj
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>) 
    Scos = similaridadeCoseno(mediaEmbeddingSi, mediaEmbeddingSj)
    # Saída: Um número real
    
    # Acumula a medida
    somaScos = somaScos + Scos

DcosOriginal = float(somaScos)/float(n-1)
print("Dcos Original:", DcosOriginal)  


Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Dcos Original: 0.8301623066266378


#### Calcula a média aritmética da similaridade do coseno entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento permutado. 

In [ ]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaScos = 0

# Percorre as sentenças do documento
for i in range(np-1):
    # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSi=", mediaEmbeddingSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSj=", mediaEmbeddingSj.shape)
  
    # Similaridade entre os embeddings Si e Sj
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Scos = similaridadeCoseno(mediaEmbeddingSi, mediaEmbeddingSj)
    # Saída: Um número real
    
    # Acumula a medida
    somaScos = somaScos + Scos

DcosPermutado = float(somaScos)/float(np-1)
print("Dcos Original:", DcosPermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']
Quantidade de sentenças: 4
Dcos Original: 0.8546958367029825


#### Compara as médias da similaridade de cosseno dos embeddings das sentenças do documento original e permutado

Características das medidas:
- Documentos com sentenças iguais resulta uma medida igual a 1.
- Documentos com sentenças diferenntes resulta uma medida menor que 1.
- Documento com sentenças muito diferentes apresentam valores menores que 1.
- Documentos iguais resultam em medidas iguais. 
- É uma medida de similaridade.



In [ ]:
print("Dcos Original :", DcosOriginal)
print("Dcos Permutado:", DcosPermutado)

if (DcosOriginal > DcosPermutado):
    print("Documento original tem maior similaridade de cosseno entre as sentenças!")
else:
    print("Documento Permutado tem menor similaridade de cosseno entre as sentenças!")

Dcos Original : 0.8301623066266378
Dcos Permutado: 0.8546958367029825
Documento Permutado tem menor similaridade de cosseno entre as sentenças!


### Distância euclidiana entre os embeddings das sentenças

Possui outros nomes como distância L2 ou norma L2.

In [ ]:
# Import das bibliotecas.
from scipy.spatial.distance import euclidean

def distanciaEuclidiana(sentenca1, sentenca2):
  distancia = euclidean(sentenca1, sentenca2)

  return distancia

#### Calcula a média aritmética da distância euclidiana entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento original. 

In [ ]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaSeuc = 0

# Percorre as sentenças do documento
for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSi=", mediaEmbeddingSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    # print("mediaEmbeddingSj=", mediaEmbeddingSj.shape)
  
    # Distância euclidiana entre os embeddings Si e Sj
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)  
    Seuc = distanciaEuclidiana(mediaEmbeddingSi, mediaEmbeddingSj)
    # Saída: Um número real
    
    # Acumula a medida
    somaSeuc = somaSeuc + Seuc

DeucOriginal = float(somaSeuc)/float(n-1)
print("Deuc Original:", DeucOriginal)  


Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Deuc Original: 19.280046463012695


#### Calcula a média aritmética da distância euclidiana entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento permutado. 

In [ ]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaSeuc = 0

# Percorre as sentenças do documento
for i in range(np-1):
    # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSi=", mediaEmbeddingSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSj=", mediaEmbeddingSj.shape)
  
    # Distância euclidiana entre os embeddings Si e Sj
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>) 
    Seuc = distanciaEuclidiana(mediaEmbeddingSi, mediaEmbeddingSj)
    # Saída: Um número real
    
    # Acumula a medida
    somaSeuc = somaSeuc + Seuc

DeucPermutado = float(somaSeuc)/float(np-1)
print("Deuc Original:", DeucPermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']
Quantidade de sentenças: 4
Deuc Original: 17.829952239990234


#### Compara as médias da distância euclidiana dos embeddings das sentenças do documento original e permutado

Características das medidas:
- Documentos com sentenças iguais resulta uma medida igual a 0.
- Documentos com sentenças diferenntes resulta uma medida maior que 0.
- Documento com sentenças muito diferentes apresentam valores maiores que 0.
- Documentos iguais resultam em medidas iguais. 
- É uma medida de diferença.



In [ ]:
print("Deuc Original :", DeucOriginal)
print("Deuc Permutado:", DeucPermutado)

if (DeucOriginal > DeucPermutado):
    print("Documento original tem maior distância euclidiana entre as sentenças!")
else:
    print("Documento Permutado tem menor distância euclidiana entre as sentenças!")

Deuc Original : 19.280046463012695
Deuc Permutado: 17.829952239990234
Documento original tem maior distância euclidiana entre as sentenças!


### Distância Manhattan entre os embeddings das sentenças

Possui outros nomes como distância Cityblock, distância L1, norma L1 e métrica do táxi.

In [ ]:
# Import das bibliotecas.
from scipy.spatial.distance import cityblock

def distanciaEManhattan(sentenca1, sentenca2):
  distancia = cityblock(sentenca1, sentenca2)

  return distancia

#### Calcula a média aritmética da distância de manhattan entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento original. 

In [ ]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaSman = 0

# Percorre as sentenças do documento
for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSi=", mediaEmbeddingSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    # print("mediaEmbeddingSj=", mediaEmbeddingSj.shape)
  
    # Distância de manhattan entre os embeddings Si e Sj
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>) 
    Sman = distanciaEManhattan(mediaEmbeddingSi, mediaEmbeddingSj)
    # Saída: Um número real
    
    # Acumula a medida
    somaSman = somaSman + Sman

DmanOriginal = float(somaSman)/float(n-1)
print("Dman Original:", DmanOriginal)  


Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Dman Original: 816.2806193033854


#### Calcula a média aritmética da distância de manhattan entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento permutado.

In [ ]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaSman = 0

# Percorre as sentenças do documento
for i in range(np-1):
    # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSi=", mediaEmbeddingSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSj=", mediaEmbeddingSj.shape)
  
    # Distância de manhattan entre os embeddings Si e Sj
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Sman = distanciaEManhattan(mediaEmbeddingSi, mediaEmbeddingSj)
    # Saída: Um número real
    
    # Acumula a medida
    somaSman = somaSman + Sman

DmanPermutado = float(somaSman)/float(np-1)
print("Deuc Original:", DmanPermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']
Quantidade de sentenças: 4
Deuc Original: 755.4674682617188


#### Compara as médias da distância de manhattan dos embeddings das sentenças do documento original e permutado

Características das medidas:
- Documentos com sentenças iguais resulta uma medida igual a 0.
- Documentos com sentenças diferenntes resulta uma medida maior que 0.
- Documento com sentenças muito diferentes apresentam valores maiores que 0.
- Documentos iguais resultam em medidas iguais. 
- É uma medida de diferença.



In [ ]:
print("Dman Original :", DmanOriginal)
print("Dman Permutado:", DmanPermutado)

if (DmanOriginal > DmanPermutado):
    print("Documento original tem maior distância de manhattan entre as sentenças!")
else:
    print("Documento Permutado tem menor distância de manhattan entre as sentenças!")

Dman Original : 816.2806193033854
Dman Permutado: 755.4674682617188
Documento original tem maior distância de manhattan entre as sentenças!


### Resumo

Resultado das medidas utilizando as quatro últimas camadas do BERT.

Base(MEAN):
- Dsub       :   0.00028153          -0.00050018
- Dubabs     :   0.26571635          0.24592040
- Dprod      :   932.79801432          942.97851562
- Dprodabs   :   932.79801432          942.97851562
- Dprode     :   932.79809570          942.97859701
- Dprodeabs  :   932.79809570          942.97859701
- Davg       :   -0.02508242          -0.02519175
- Dcos       :   0.83016231          0.85469584
- Deuc       :   19.28004646          17.82995224
- Dman       :   816.28061930          755.46746826
----------------------
Resultado das medidas utilizando a última camada do BERT.

Base(MEAN):
- Dsub       :   0.00074918          -0.00064076
- Dubabs     :   0.17778097          0.17206367
- Dprod      :   43.70739746          43.55047607
- Dprodabs   :   43.70739746          43.55047607
- Dprode     :   43.70739746          43.55047607
- Dprodeabs  :   43.70739746          43.55047607
- Davg       :   -0.00276791          -0.00271371
- Dcos       :   0.67999101          0.69653825
- Deuc       :   6.34085274          6.15486606
- Dman       :   136.53579712          132.14489237

Base(MAX):
- Dsub       :   -0.02765538          0.02360132
- Dubabs     :   0.26021336          0.25770577
- Dprod      :   43.70739746          43.55047607
- Dprodabs   :   43.70739746          43.55047607
- Dprode     :   200.03959147          199.48846436
- Dprodeabs  :   200.03959147          199.48846436
- Davg       :   0.43032602          0.42829903
- Dcos       :   0.81970002          0.82297109
- Deuc       :   9.42460442          9.29481570
- Dman       :   199.84384664          197.91803996

In [ ]:
print("Resultado das medidas utilizando as quatro últimas camada do BERT")
print("Documento  :   Original            Permutado")
print('Dsub       :   {:.8f}          {:.8f}'.format(DsubOriginal,DsubPermutado))
print('Dubabs     :   {:.8f}          {:.8f}'.format(DsubabsOriginal,DsubabsPermutado))
print('Dprod      :   {:.8f}          {:.8f}'.format(DprodOriginal,DprodPermutado))
print('Dprodabs   :   {:.8f}          {:.8f}'.format(DprodabsOriginal,DprodabsPermutado))
print('Dprode     :   {:.8f}          {:.8f}'.format(DprodeOriginal,DprodePermutado))
print('Dprodeabs  :   {:.8f}          {:.8f}'.format(DprodeabsOriginal,DprodeabsPermutado))
print('Davg       :   {:.8f}          {:.8f}'.format(DavgOriginal,DavgPermutado))
print('Dcos       :   {:.8f}          {:.8f}'.format(DcosOriginal,DcosPermutado))
print('Deuc       :   {:.8f}          {:.8f}'.format(DeucOriginal,DeucPermutado))
print('Dman       :   {:.8f}          {:.8f}'.format(DmanOriginal,DmanPermutado))

Resultado das medidas utilizando as quatro últimas camada do BERT
Documento  :   Original            Permutado
Dsub       :   0.00028153          -0.00050018
Dubabs     :   0.26571635          0.24592040
Dprod      :   932.79801432          942.97851562
Dprodabs   :   932.79801432          942.97851562
Dprode     :   932.79809570          942.97859701
Dprodeabs  :   932.79809570          942.97859701
Davg       :   -0.02508242          -0.02519175
Dcos       :   0.83016231          0.85469584
Deuc       :   19.28004646          17.82995224
Dman       :   816.28061930          755.46746826


## 9 - Exemplo sentenças de documento original e permutado utilizando embedding da concatenação das 4 últimas camadas do BERT usando estratégia MAX

Como estamos utilizando os embeddings concatenado das 4 últimas camadas onde ocorre 768 entenda-se 3072 que é o resultado de 768 por 4 que é a dimensão do MCL BERT de tamanho base. E onde ocorre 1024 entenda-se 4096 que é o resultado de 1024 por 4 que é a dimensão do MCL BERT de tamanho large.

### Documento Original

In [ ]:
# Define um documento com 4 sentenças
documento_original = ["Bom Dia, professor.",
             "Qual o conteúdo da prova?",              
             "Vai cair tudo na prova?",
             "Aguardo uma resposta, João."]

# Concatena as sentenças do documento em uma string
stringDocumentoPermutado = ' '.join(documento_original)

# Adiciona os tokens especiais
documento_marcado_original = "[CLS] " + stringDocumentoPermutado + " [SEP]"

# Divide a sentença em tokens
documento_tokenizado_original = tokenizer.tokenize(documento_marcado_original)

# Mapeia os tokens em seus índices do vocabulário
documento_tokens_indexados_original = tokenizer.convert_tokens_to_ids(documento_tokenizado_original)

# Mostra os tokens com seus índices
i = 0
for tup in zip(documento_tokenizado_original, documento_tokens_indexados_original):
    print('{:>3} {:<12} {:>6,}'.format(i, tup[0], tup[1]))
    i = i + 1

  0 [CLS]           101
  1 Bom           8,399
  2 Dia           3,616
  3 ,               117
  4 professor     2,917
  5 .               119
  6 Qual         13,082
  7 o               146
  8 conteúdo      5,015
  9 da              180
 10 prova         2,310
 11 ?               136
 12 Vai          20,805
 13 cair          9,322
 14 tudo          2,745
 15 na              229
 16 prova         2,310
 17 ?               136
 18 Agu           8,125
 19 ##ardo        2,222
 20 uma             230
 21 resposta      4,299
 22 ,               117
 23 João          1,453
 24 .               119
 25 [SEP]           102


Máscara de atenção das palavras

In [ ]:
# Marca cada um dos tokens como pertencentes à sentença "1".
mascara_atencao_original = [1] * len(documento_tokenizado_original)

print (mascara_atencao_original)
print (len(mascara_atencao_original))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
26


Convertendo as listas em tensores

In [ ]:
# Importa a bibliteca
import torch

# Converte as entradas de listas para tensores do torch
tokens_tensores_original = torch.as_tensor([documento_tokens_indexados_original])
mascara_atencao_tensores_original = torch.as_tensor([mascara_atencao_original])

Gera os embeddings para o documento original. Guarda todas as camadas da rede em `outputs`.


In [ ]:
# Prediz os atributos dos estados ocultos para cada camada
with torch.no_grad():
    # output[0] contém last_hidden_states
    outputs = model(tokens_tensores_original, mascara_atencao_tensores_original)

In [ ]:
# Cria uma lista com os tensores a serem concatenados
# Entrada: List das camadas(13 ou 25) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
# Lista com os tensores a serem concatenados
listaConcat = []
# Percorre os 4 últimos
for i in [-1,-2,-3,-4]:
    # Concatena da lista
    listaConcat.append(outputs[2][i])
    # Saída: Entrada: List das camadas(4) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
     #print("listaConcat=",len(listaConcat))

# Realiza a concatenação dos embeddings de todos as camadas
# Saída: Entrada: List das camadas(4) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
concat4_hidden_states = torch.cat(listaConcat, dim=-1)
# Saída: Entrada: (<1(lote)> x <qtde_tokens> <3072 ou 4096>)  

print ("O vetor da  concatenação das 4 últimas camadas oculta tem o formato:", concat4_hidden_states.size())

O vetor da  concatenação das 4 últimas camadas oculta tem o formato: torch.Size([1, 26, 3072])


Vamos nos livrar da dimensão lotes "batches", pois não precisamos dela.

In [ ]:
# Remove a dimensão 1, o lote "batches".
#O método squeeze remove a primeira dimensão(0) pois possui tamanho 1
embeddingDocumentoOriginal = torch.squeeze(concat4_hidden_states, dim=0)

print ("O vetor de tokens de embedding do documento original tem o formato:", embeddingDocumentoOriginal.size())

O vetor de tokens de embedding do documento original tem o formato: torch.Size([26, 3072])


Confirmando vetores dependentes do contexto


In [ ]:
for i, token_str in enumerate(documento_tokenizado_original):
  print (i, token_str)

0 [CLS]
1 Bom
2 Dia
3 ,
4 professor
5 .
6 Qual
7 o
8 conteúdo
9 da
10 prova
11 ?
12 Vai
13 cair
14 tudo
15 na
16 prova
17 ?
18 Agu
19 ##ardo
20 uma
21 resposta
22 ,
23 João
24 .
25 [SEP]


Exibe os embenddings das sentenças

In [ ]:
# Índice das sentenças a serem comparadas
sentenca1Original = documento_original[0]
sentenca2Original = documento_original[1]
sentenca3Original = documento_original[2]
sentenca4Original = documento_original[3]

embeddingSentenca1Original = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, sentenca1Original, tokenizer)
embeddingSentenca2Original = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, sentenca2Original, tokenizer)
embeddingSentenca3Original = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, sentenca3Original, tokenizer)
embeddingSentenca4Original = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, sentenca4Original, tokenizer)

print('Os primeiros 4 valores de cada sentença do documento original.')

print('\nSentença 1:', sentenca1Original,'-', str(embeddingSentenca1Original[:4]))
print('Soma embedding Sentença1:', sentenca1Original,'-', str(torch.sum(embeddingSentenca1Original[:4])))

print('\nSentença 2:', sentenca2Original,'-', str(embeddingSentenca2Original[:4]))
print('Soma embedding Sentença2:', sentenca2Original,'-', str(torch.sum(embeddingSentenca2Original[:4])))

print('\nSentença 3:', sentenca3Original,'-', str(embeddingSentenca3Original[:4]))
print('Soma embedding Sentença3:', sentenca3Original,'-', str(torch.sum(embeddingSentenca3Original[:4])))

print('\nSentença 4:', sentenca4Original,'-', str(embeddingSentenca4Original[:4]))
print('Soma embedding Sentença4:', sentenca4Original,'-', str(torch.sum(embeddingSentenca4Original[:4])))

Os primeiros 4 valores de cada sentença do documento original.

Sentença 1: Bom Dia, professor. - tensor([[-0.0680, -0.4615,  0.3552,  ...,  0.1657,  0.3478,  0.4525],
        [-0.1000, -0.0630,  0.0840,  ..., -0.3926,  0.7971,  0.3287],
        [-0.3165,  0.4208,  0.2178,  ..., -0.3834,  0.9520, -0.2181],
        [ 0.1248,  0.2383,  0.8987,  ..., -0.1989,  0.1001,  0.1140]])
Soma embedding Sentença1: Bom Dia, professor. - tensor(-327.0100)

Sentença 2: Qual o conteúdo da prova? - tensor([[-0.5894, -0.4310,  0.1449,  ...,  0.2399, -0.8495, -0.1672],
        [ 0.1349, -0.2476,  0.4605,  ..., -0.1353, -0.1674,  0.4057],
        [ 0.4359, -0.6972,  0.4066,  ...,  0.5851, -0.0441, -0.0027],
        [ 0.0544,  0.1606,  0.4150,  ..., -0.8060, -0.3698,  0.0466]])
Soma embedding Sentença2: Qual o conteúdo da prova? - tensor(-315.7253)

Sentença 3: Vai cair tudo na prova? - tensor([[-0.3652, -0.5015, -0.1626,  ..., -0.5431, -0.5805,  0.4617],
        [ 0.1421,  0.1797, -0.0014,  ..., -0.7004, -

Examinando os embeddings do documento original



In [ ]:
# Índice das sentenças a serem comparadas
sentenca1Original = documento_original[0]
sentenca2Original = documento_original[1]
sentenca3Original = documento_original[2]
sentenca4Original = documento_original[3]

print("Documento Original:", documento_original)

# Localiza os índices dos tokens da sentença no documento
sentenca1TokenizadaOriginal = tokenizer.tokenize(sentenca1Original)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_original,sentenca1TokenizadaOriginal)
embeddingSentenca1Original = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, sentenca1Original, tokenizer)
print('\nSentença 1 Original=\'', sentenca1Original, '\'')
print('    Sentença tokenizada:', sentenca1TokenizadaOriginal)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca1Original.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca1Original))

# Localiza os índices dos tokens da sentença no documento
sentenca2TokenizadaOriginal = tokenizer.tokenize(sentenca2Original)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_original,sentenca2TokenizadaOriginal)
embeddingSentenca2Original = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, sentenca2Original, tokenizer)
print('\nSentença 2 Original=\'', sentenca2Original, '\'')
print('    Sentença tokenizada:', sentenca2TokenizadaOriginal)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca2Original.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca2Original))

# Localiza os índices dos tokens da sentença no documento
sentenca3TokenizadaOriginal = tokenizer.tokenize(sentenca3Original)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_original,sentenca3TokenizadaOriginal)
embeddingSentenca3Original = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, sentenca3Original, tokenizer)
print('\nSentença 3 Original=\'', sentenca3Original, '\'')
print('    Sentença tokenizada:', sentenca3TokenizadaOriginal)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca3Original.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca3Original))

# Localiza os índices dos tokens da sentença no documento
sentenca4TokenizadaOriginal = tokenizer.tokenize(sentenca4Original)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_original,sentenca4TokenizadaOriginal)
embeddingSentenca4Original = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, sentenca4Original, tokenizer)
print('\nSentença 4 Original=\'', sentenca4Original, '\'')
print('    Sentença tokenizada:', sentenca4TokenizadaOriginal)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca4Original.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca4Original))


Documento Original: ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']

Sentença 1 Original=' Bom Dia, professor. '
    Sentença tokenizada: ['Bom', 'Dia', ',', 'professor', '.']
    => inicio em 1 e término em 5
    Formato modelo : torch.Size([5, 3072])
    Soma embeddings:  -398.53

Sentença 2 Original=' Qual o conteúdo da prova? '
    Sentença tokenizada: ['Qual', 'o', 'conteúdo', 'da', 'prova', '?']
    => inicio em 6 e término em 11
    Formato modelo : torch.Size([6, 3072])
    Soma embeddings:  -467.44

Sentença 3 Original=' Vai cair tudo na prova? '
    Sentença tokenizada: ['Vai', 'cair', 'tudo', 'na', 'prova', '?']
    => inicio em 12 e término em 17
    Formato modelo : torch.Size([6, 3072])
    Soma embeddings:  -466.27

Sentença 4 Original=' Aguardo uma resposta, João. '
    Sentença tokenizada: ['Agu', '##ardo', 'uma', 'resposta', ',', 'João', '.']
    => inicio em 18 e término em 24
    Formato modelo : torch.S

### Documento Permutado

In [ ]:
# Define um documento com a permutação das sentenças do documento original
documento_permutado = [documento_original[3],   # "Aguardo uma resposta, João.",
             documento_original[1],             # "Qual o conteúdo da prova?",              
             documento_original[0],             # "Vai cair tudo na prova?",
             documento_original[2]]             # "Bom Dia, professor."]     

# Use o documento permutado igual ao original para testar se as medidas estão corretas
#documento_permutado = documento_original

# Concatena as sentenças do documento em uma string
stringDocumentoPermutado = ' '.join(documento_permutado)

# Adiciona os tokens especiais
documento_marcado_permutado = "[CLS] " + stringDocumentoPermutado + " [SEP]"

# Divide a sentença em tokens
documento_tokenizado_permutado = tokenizer.tokenize(documento_marcado_permutado)

# Mapeia os tokens em seus índices do vocabulário
documento_tokens_indexados_permutado = tokenizer.convert_tokens_to_ids(documento_tokenizado_permutado)

# Mostra os tokens com seus índices
i = 0
for tup in zip(documento_tokenizado_permutado, documento_tokens_indexados_permutado):
    print('{:>3} {:<12} {:>6,}'.format(i, tup[0], tup[1]))
    i = i + 1

  0 [CLS]           101
  1 Agu           8,125
  2 ##ardo        2,222
  3 uma             230
  4 resposta      4,299
  5 ,               117
  6 João          1,453
  7 .               119
  8 Qual         13,082
  9 o               146
 10 conteúdo      5,015
 11 da              180
 12 prova         2,310
 13 ?               136
 14 Bom           8,399
 15 Dia           3,616
 16 ,               117
 17 professor     2,917
 18 .               119
 19 Vai          20,805
 20 cair          9,322
 21 tudo          2,745
 22 na              229
 23 prova         2,310
 24 ?               136
 25 [SEP]           102


Máscara de atenção das palavras

In [ ]:
# Marca cada um dos tokens como pertencentes à sentença "1".
mascara_atencao_permutado = [1] * len(documento_tokenizado_permutado)

print (mascara_atencao_permutado)
print (len(mascara_atencao_permutado))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
26


Convertendo as listas em tensores

In [ ]:
# Importa a bibliteca
import torch

# Converte as entradas de listas para tensores do torch
tokens_tensores_permutado = torch.as_tensor([documento_tokens_indexados_permutado])
mascara_atencao_tensores_permutado = torch.as_tensor([mascara_atencao_permutado])

Gera os embeddings para o documento original. Guarda todas as camadas da rede em `outputs`.


In [ ]:
# Prediz os atributos dos estados ocultos para cada camada
with torch.no_grad():
    # output[0] contém last_hidden_states
    outputs = model(tokens_tensores_permutado, mascara_atencao_tensores_permutado)

Recupera a saída e concatena as 4 últimas camada do BERT

In [ ]:
# Cria uma lista com os tensores a serem concatenados
# Entrada: List das camadas(13 ou 25) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
# Lista com os tensores a serem concatenados
listaConcat = []
# Percorre os 4 últimos
for i in [-1,-2,-3,-4]:
    # Concatena da lista
    listaConcat.append(outputs[2][i])
    # Saída: Entrada: List das camadas(4) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
     #print("listaConcat=",len(listaConcat))

# Realiza a concatenação dos embeddings de todos as camadas
# Saída: Entrada: List das camadas(4) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
concat4_hidden_states = torch.cat(listaConcat, dim=-1)
# Saída: Entrada: (<1(lote)> x <qtde_tokens> <3072 ou 4096>)  

Vamos nos livrar da dimensão lotes "batches", pois não precisamos dela.

In [ ]:
# Remove a dimensão 1, o lote "batches".
#O método squeeze remove a primeira dimensão(0) pois possui tamanho 1
embeddingDocumentoPermutado = torch.squeeze(concat4_hidden_states, dim=0)

print ("O vetor de tokens de embedding do documento permutado tem o formato:", embeddingDocumentoPermutado.size())

O vetor de tokens de embedding do documento permutado tem o formato: torch.Size([26, 3072])


Exibe os embenddings das sentenças

In [ ]:
# Índice das sentenças a serem comparadas
sentenca1Permutado = documento_permutado[0]
sentenca2Permutado = documento_permutado[1]
sentenca3Permutado = documento_permutado[2]
sentenca4Permutado = documento_permutado[3]

embeddingSentenca1Permutado = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoPermutado, stringDocumentoPermutado, sentenca1Permutado, tokenizer)
embeddingSentenca2Permutado = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoPermutado, stringDocumentoPermutado, sentenca2Permutado, tokenizer)
embeddingSentenca3Permutado = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoPermutado, stringDocumentoPermutado, sentenca3Permutado, tokenizer)
embeddingSentenca4Permutado = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoPermutado, stringDocumentoPermutado, sentenca4Permutado, tokenizer)

print('Os primeiros 4 valores de cada sentença do documento permutado.')

print('\nSentença 1:', sentenca1Permutado,'-', str(embeddingSentenca1Permutado[:4]))
print('Soma embedding Sentença1:', sentenca1Original,'-', str(torch.sum(embeddingSentenca1Original[:4])))

print('\nSentença 2:', sentenca2Permutado,'-', str(embeddingSentenca2Permutado[:4]))
print('Soma embedding Sentença2:', sentenca2Permutado,'-', str(torch.sum(embeddingSentenca2Permutado[:4])))

print('\nSentença 3:', sentenca3Permutado,'-', str(embeddingSentenca3Permutado[:4]))
print('Soma embedding Sentença3:', sentenca3Permutado,'-', str(torch.sum(embeddingSentenca3Original[:4])))

print('\nSentença 4:', sentenca4Permutado,'-', str(embeddingSentenca4Permutado[:4]))
print('Soma embedding Sentença4:', sentenca4Permutado,'-', str(torch.sum(embeddingSentenca4Permutado[:4])))

Os primeiros 4 valores de cada sentença do documento permutado.

Sentença 1: Aguardo uma resposta, João. - tensor([[-0.0052, -0.4706,  0.6481,  ..., -0.1775,  0.2231,  0.8036],
        [-0.1516, -0.2450,  0.2693,  ..., -0.3783,  0.1529,  0.5125],
        [-0.7358, -0.0988,  0.5145,  ...,  0.0164, -0.1749,  0.2557],
        [-0.1278,  0.3571,  0.5481,  ..., -0.2708,  0.1942, -0.0740]])
Soma embedding Sentença1: Bom Dia, professor. - tensor(-327.0100)

Sentença 2: Qual o conteúdo da prova? - tensor([[-0.4295, -0.3426,  0.1005,  ...,  0.2608, -0.8301, -0.1044],
        [ 0.1047, -0.1946,  0.4177,  ..., -0.1734, -0.2517,  0.4004],
        [ 0.4377, -0.5952,  0.5448,  ...,  0.4539, -0.1388, -0.0339],
        [ 0.0898,  0.1063,  0.4610,  ..., -0.5308, -0.4884,  0.2119]])
Soma embedding Sentença2: Qual o conteúdo da prova? - tensor(-314.9913)

Sentença 3: Bom Dia, professor. - tensor([[-0.0458, -0.4268,  0.2399,  ...,  0.4282,  0.3110,  0.0241],
        [ 0.0650, -0.1385,  0.0230,  ..., -0.36

In [ ]:
# Índice das sentenças a serem comparadas
sentenca1Permutado = documento_permutado[0]
sentenca2Permutado = documento_permutado[1]
sentenca3Permutado = documento_permutado[2]
sentenca4Permutado = documento_permutado[3]

print("Documento Permutado:", documento_permutado)

# Localiza os índices dos tokens da sentença no documento
sentenca1TokenizadaPermutado = tokenizer.tokenize(sentenca1Permutado)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_permutado,sentenca1TokenizadaPermutado)
embeddingSentenca1Permutado = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoPermutado, stringDocumentoPermutado, sentenca1Permutado, tokenizer)
print('\nSentença 1 Permutada=\'', sentenca1Permutado, '\'')
print('    Sentença tokenizada:', sentenca1TokenizadaPermutado)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca1Permutado.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca1Permutado))

# Localiza os índices dos tokens da sentença no documento
sentenca2TokenizadaPermutado = tokenizer.tokenize(sentenca2Permutado)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_permutado,sentenca2TokenizadaPermutado)
embeddingSentenca2Permutado = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoPermutado, stringDocumentoPermutado, sentenca2Permutado, tokenizer)
print('\nSentença 2 Permutada=\'', sentenca2Permutado, '\'')
print('    Sentença tokenizada:', sentenca2TokenizadaPermutado)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca2Permutado.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca2Permutado))

# Localiza os índices dos tokens da sentença no documento
sentenca3TokenizadaPermutado = tokenizer.tokenize(sentenca3Permutado)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_permutado,sentenca3TokenizadaPermutado)
embeddingSentenca3Permutado = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoPermutado, stringDocumentoPermutado, sentenca3Permutado, tokenizer)
print('\nSentença 3 Permutada=\'', sentenca3Permutado, '\'')
print('    Sentença tokenizada:', sentenca3TokenizadaPermutado)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca3Permutado.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca3Permutado))

# Localiza os índices dos tokens da sentença no documento
sentenca4TokenizadaPermutado = tokenizer.tokenize(sentenca4Permutado)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_permutado,sentenca4TokenizadaPermutado)
embeddingSentenca4Permutado = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoPermutado, stringDocumentoPermutado, sentenca4Permutado, tokenizer)
print('\nSentença 4 Permutada=\'', sentenca4Permutado, '\'')
print('    Sentença tokenizada:', sentenca4TokenizadaPermutado)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingSentenca4Permutado.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca4Permutado))


Documento Permutado: ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']

Sentença 1 Permutada=' Aguardo uma resposta, João. '
    Sentença tokenizada: ['Agu', '##ardo', 'uma', 'resposta', ',', 'João', '.']
    => inicio em 1 e término em 7
    Formato modelo : torch.Size([7, 3072])
    Soma embeddings:  -532.56

Sentença 2 Permutada=' Qual o conteúdo da prova? '
    Sentença tokenizada: ['Qual', 'o', 'conteúdo', 'da', 'prova', '?']
    => inicio em 8 e término em 13
    Formato modelo : torch.Size([6, 3072])
    Soma embeddings:  -467.65

Sentença 3 Permutada=' Bom Dia, professor. '
    Sentença tokenizada: ['Bom', 'Dia', ',', 'professor', '.']
    => inicio em 14 e término em 18
    Formato modelo : torch.Size([5, 3072])
    Soma embeddings:  -402.97

Sentença 4 Permutada=' Vai cair tudo na prova? '
    Sentença tokenizada: ['Vai', 'cair', 'tudo', 'na', 'prova', '?']
    => inicio em 19 e término em 24
    Formato modelo : to

### Examinando as sentenças

A mesma sentença apresenta embeddings com valores diferentes, pois se encontram em locais diferentes do documento. A soma de todos os embeddings demonstra isto.

In [ ]:
print('\nSentença 4 Original=\'', sentenca4Original, '\'')
print('    Sentença tokenizada:', sentenca4TokenizadaOriginal)
print('    Formato modelo :', embeddingSentenca4Original.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca4Original))
print('    Os 4 primeiros embeddings:', str(embeddingSentenca4Original[:4]))

print('\nSentença 1 Permutada=\'', sentenca1Permutado, '\'')
print('    Sentença tokenizada:', sentenca1TokenizadaPermutado)
print('    Formato modelo :', embeddingSentenca1Permutado.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingSentenca1Permutado))
print('    Os 4 primeiros embeddings:', str(embeddingSentenca1Permutado[:4]))


Sentença 4 Original=' Aguardo uma resposta, João. '
    Sentença tokenizada: ['Agu', '##ardo', 'uma', 'resposta', ',', 'João', '.']
    Formato modelo : torch.Size([7, 3072])
    Soma embeddings:  -525.26
    Os 4 primeiros embeddings: tensor([[-0.0835, -0.4042,  0.4330,  ...,  0.2881,  0.1448,  0.7778],
        [-0.2782, -0.1630,  0.0997,  ..., -0.0102,  0.1851,  0.4587],
        [-0.5474, -0.0118,  0.3950,  ...,  0.1932, -0.0735,  0.2649],
        [ 0.0146,  0.2901,  0.3920,  ..., -0.0370,  0.3236, -0.3671]])

Sentença 1 Permutada=' Aguardo uma resposta, João. '
    Sentença tokenizada: ['Agu', '##ardo', 'uma', 'resposta', ',', 'João', '.']
    Formato modelo : torch.Size([7, 3072])
    Soma embeddings:  -532.56
    Os 4 primeiros embeddings: tensor([[-0.0052, -0.4706,  0.6481,  ..., -0.1775,  0.2231,  0.8036],
        [-0.1516, -0.2450,  0.2693,  ..., -0.3783,  0.1529,  0.5125],
        [-0.7358, -0.0988,  0.5145,  ...,  0.0164, -0.1749,  0.2557],
        [-0.1278,  0.3571,  0.5481

### Subtração entre os embeddings das sentenças

#### Calcula a média aritmética da subtração entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento original. 

In [ ]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaSsub = 0

# Percorre as sentenças do documento
for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Encontra os maiores embeddings os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSi, linha = torch.max(embeddingSi, dim=0)        
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSi:", len(maiorEmbeddingSi))
        
    # Encontra os maiores embeddings os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSj, linha = torch.max(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSj:", len(maiorEmbeddingSj))
       
    # Subtração entre os embeddings de Si e Sj
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Ssub = torch.sub(maiorEmbeddingSi, maiorEmbeddingSj)
    # Saída: <768 ou 1024>  
    #print("Ssub=", Ssub.shape)
    
    # Calcula a média dos embeddings do subtração para realizar a soma
    somaSsub = somaSsub + torch.mean(Ssub)

DsubOriginal = float(somaSsub)/float(len(documento_original)-1)
print("Ssub Original:", DsubOriginal)

Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Ssub Original: -0.023183703422546387


#### #### Calcula a média aritmética da subtração entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento permutado. 

In [ ]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaSsub = 0

# Percorre as sentenças do documento
for i in range(np-1):
    # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Encontra os maiores embeddings os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSi, linha = torch.max(embeddingSi, dim=0)        
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSi:", len(maiorEmbeddingSi))
        
    # Encontra os maiores embeddings os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSj, linha = torch.max(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSj:", len(maiorEmbeddingSj))
       
    # Subtração entre os embeddings de Si e Sj
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Ssub = torch.sub(maiorEmbeddingSi, maiorEmbeddingSj)
    # Saída: <768 ou 1024>  
    #print("Ssub=", Ssub.shape)
        
    # Calcula a média dos embeddings do subtração para realizar a soma
    somaSsub = somaSsub + torch.mean(Ssub)

DsubPermutado = float(somaSsub)/float(np-1)
print("Ssub permutado:", DsubPermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']
Quantidade de sentenças: 4
Ssub permutado: 0.03090549260377884


#### Compara as médias da subtração dos embeddings das sentenças do documento original e permutado.

Características das medidas:
- Documentos com sentenças iguais resulta uma medida igual a 0.
- Documentos com sentenças diferenntes resulta uma medida maior que 0.
- Documento com sentenças muito diferentes apresentam valores maiores que 0.
- Documentos iguais resultam em medidas iguais. 
- É uma medida de diferença.


In [ ]:
print("Dsub Original :", DsubOriginal)
print("Dsub Permutado:", DsubPermutado)

if (DsubOriginal <= DsubPermutado):
    print("Documento original tem menor diferença entre as sentenças!")
else:
    print("Documento Permutado tem menor diferença entre as sentenças!")

if (DsubOriginal > DsubPermutado):
    print("Documento original tem menor diferença entre as sentenças!")
else:
    print("Documento Permutado tem menor diferença entre as sentenças!")

Dsub Original : -0.023183703422546387
Dsub Permutado: 0.03090549260377884
Documento original tem menor diferença entre as sentenças!
Documento Permutado tem menor diferença entre as sentenças!


### Subtração absoluta entre os embeddings das sentenças

#### Calcula a média aritmética da subtração absoluta entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento original. 

In [ ]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaSsubabs = 0

# Percorre as sentenças do documento
for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Encontra os maiores embeddings os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSi, linha = torch.max(embeddingSi, dim=0)        
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSi:", len(maiorEmbeddingSi))
        
    # Encontra os maiores embeddings os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSj, linha = torch.max(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSj:", len(maiorEmbeddingSj))
       
    # Subtração entre os embeddings de Si e Sj
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>) 
    Ssubabs = abs(torch.sub(maiorEmbeddingSi, maiorEmbeddingSj))
    # Saída: <768 ou 1024>  
    #print("Ssubabs=", Ssubabs.shape)
  
    # Calcula a média dos embeddings do subtração para realizar a soma
    somaSsubabs = somaSsubabs + torch.mean(Ssubabs)

DsubabsOriginal = float(somaSsubabs)/float(n-1)
print("Dsubabs Original:", DsubabsOriginal)

Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Dsubabs Original: 0.3687093257904053


#### Calcula a média aritmética da subtração absoluta entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento permutado. 

In [ ]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaSsubabs = 0

# Percorre as sentenças do documento
for i in range(np-1):
    # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Encontra os maiores embeddings os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSi, linha = torch.max(embeddingSi, dim=0)        
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSi:", len(maiorEmbeddingSi))
        
    # Encontra os maiores embeddings os tokens de Sj, removendo a primeira dimensão.
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    maiorEmbeddingSj, linha = torch.max(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSj:", len(maiorEmbeddingSj))
       
    # Subtração entre os embeddings de Si e Sj
    # Entrada: <768 ou 1024>  
    Ssubabs = abs(torch.sub(maiorEmbeddingSi, maiorEmbeddingSj))
    # Saída: <768 ou 1024>  
    #print("Ssub=", Ssub.shape)

    # Calcula a média dos embeddings do subtração para realizar a soma
    somaSsubabs = somaSsubabs + torch.mean(Ssubabs)

DsubabsPermutado = float(somaSsubabs)/float(np-1)
print("Dsubabs permutado:", DsubabsPermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']
Quantidade de sentenças: 4
Dsubabs permutado: 0.35224469502766925


#### Compara as médias da subtração absoluta dos embeddings das sentenças do documento original e permutado

Características das medidas:
- Documentos com sentenças iguais resulta uma medida igual a 0.
- Documentos com sentenças diferenntes resulta uma medida maior que 0.
- Documento com sentenças muito diferentes apresentam valores maiores que 0.
- Documentos iguais resultam em medidas iguais. 
- É uma medida de diferença.

In [ ]:
print("Dsubabs Original :", DsubabsOriginal)
print("Dsubabs Permutado:", DsubabsPermutado)

if (DsubabsOriginal <= DsubabsPermutado):
    print("Documento original tem menor diferença absoluta entre as sentenças!")
else:
    print("Documento Permutado tem menor diferença absoluta entre as sentenças!")

Dsubabs Original : 0.3687093257904053
Dsubabs Permutado: 0.35224469502766925
Documento Permutado tem menor diferença absoluta entre as sentenças!


### Produto entre os embeddings das sentenças

#### Calcula a média aritmética do produto das matrizes dos tokens das sentenças do documento original. 

In [ ]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaSprod = 0

# Percorre as sentenças do documento
for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

   # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Para a multiplicação pode ocorrer número de colunas(qtde_tokens) de Si tem que ser igual ao número de linhas(embeddings) de Sj.
    # Permute realiza a troca da dimensão 1(qtde_embeddings) pela 0(qtde_tokesn) na sentença Sj para que seja possível realizar o produto. 
    # Ocorre a troca da qtde_tokens pela quantidade de embeddings.
    # Entrada: (<qtde_tokensSi> x <768 ou 1024>) x (<qtde_tokensSj> x <768 ou 1024>)
    # Permute: <qtde_tokensSi> x <768 ou 1024>) x (<768 ou 1024> x <qtde_tokensSj>)
    Sprod = torch.matmul(embeddingSi, embeddingSj.permute(1,0))    
    # Saída: <qtde_tokensSi> x  <qtde_tokensSj>
    #print("Sprod=", Sprod.shape)

    # Encontra os maiores embeddings do produto para realizar a soma
    somaSprod = somaSprod + torch.mean(Sprod)

DprodOriginal = float(somaSprod)/float(n-1)
print("Dprod Original:", DprodOriginal)
    


Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Dprod Original: 932.7980143229166


#### Calcula a média aritmética do produto das matrizes dos tokens das sentenças do documento permutado. 

In [ ]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaSprod = 0

# Percorre as sentenças do documento
for i in range(np-1):
    # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSi = torch.mean(embeddingSi, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSi=", mediaEmbeddingSi.shape)
  
    # Calcula a média dos embeddings para os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    mediaEmbeddingSj = torch.mean(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("mediaEmbeddingSj=", mediaEmbeddingSj.shape)
  
    # Para a multiplicação pode ocorrer número de colunas(qtde_tokens) de Si tem que ser igual ao número de linhas(embeddings) de Sj.
    # Permute realiza a troca da dimensão 1(qtde_embeddings) pela 0(qtde_tokesn) na sentença Sj para que seja possível realizar o produto. 
    # Ocorre a troca da qtde_tokens pela quantidade de embeddings.
    # Entrada: (<qtde_tokensSi> x <768 ou 1024>) x (<qtde_tokensSj> x <768 ou 1024>)
    # Permute: <qtde_tokensSi> x <768 ou 1024>) x (<768 ou 1024> x <qtde_tokensSj>)
    Sprod = torch.matmul(embeddingSi, embeddingSj.permute(1,0))    
    # Saída: <qtde_tokensSi> x  <qtde_tokensSj>
    #print("Sprod=", Sprod.shape)
    
    # Encontra os maiores embeddings do produto para realizar a soma
    somaSprod = somaSprod + torch.mean(Sprod)

DprodPermutado = float(somaSprod)/float(np-1)
print("Dprod permutado:", DprodPermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']
Quantidade de sentenças: 4
Dprod permutado: 942.978515625


#### Compara as médias do produto dos embeddings das sentenças do documento original e permutado

Características das medidas:
- Dois documentos(vetores) que apontam em uma direção semelhante retornam um produto escalar positivo.
- Dois documentos perpendiculares retornam um produto escalar de zero.
- Dois documentos(vetores) que apontam em direções opostas retornam um produto escalar negativo.


In [ ]:
print("Dprod Original :", DprodOriginal)
print("Dprod Permutado:", DprodPermutado)

if (DprodOriginal > 0 and DprodPermutado > 0):
    print("As sentenças do documento original e as sentenças do documento permutado estão relacionadas!")
else:
  if (DprodOriginal < 0 and DprodPermutado > 0) or (DprodOriginal > 0 and DprodPermutado < 0):
    print("As sentenças do documento original e as sentenças do documento permutado não estão relacionadas!")
  else:  
    print("As sentenças do documento original e as sentenças do documento permutado não possuem relação!")              

Dprod Original : 932.7980143229166
Dprod Permutado: 942.978515625
As sentenças do documento original e as sentenças do documento permutado estão relacionadas!


### Produto Absoluto entre os embeddings das sentenças

#### Calcula a média aritmética do produto absoluto das matrizes dos tokens das sentenças do documento original. 

In [ ]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaSprodabs = 0

# Percorre as sentenças do documento
for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)
    
    # # Para a multiplicação pode ocorrer número de colunas(qtde_tokens) de Si tem que ser igual ao número de linhas(embeddings) de Sj.
    # Permute realiza a troca da dimensão 1(qtde_embeddings) pela 0(qtde_tokesn) na sentença Sj para que seja possível realizar o produto. 
    # Ocorre a troca da qtde_tokens pela quantidade de embeddings.
    # Entrada: (<qtde_tokensSi> x <768 ou 1024>) x (<qtde_tokensSj> x <768 ou 1024>)
    # Permute: <qtde_tokensSi> x <768 ou 1024>) x (<768 ou 1024> x <qtde_tokensSj>)
    Sprodabs = abs(torch.matmul(embeddingSi, embeddingSj.permute(1,0)))   
    # Saída: <qtde_tokensSi> x  <qtde_tokensSj>
    #print("Sprodabs=", Sprodabs.shape)
   
    # Encontra os maiores embeddings absolutos do produto para realizar a soma
    somaSprodabs = somaSprodabs + torch.mean(Sprodabs)

DprodabsOriginal = float(somaSprodabs)/float(n-1)
print("Dprodabs Original:", DprodabsOriginal)
    


Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Dprodabs Original: 932.7980143229166


#### Calcula a média aritmética do produto absoluto das matrizes dos tokens das sentenças do documento permutado. 

In [ ]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaSprodabs = 0

# Percorre as sentenças do documento
for i in range(np-1):
    # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)
 
     # Para a multiplicação pode ocorrer número de colunas(qtde_tokens) de Si tem que ser igual ao número de linhas(embeddings) de Sj.
    # Permute realiza a troca da dimensão 1(qtde_embeddings) pela 0(qtde_tokesn) na sentença Sj para que seja possível realizar o produto. 
    # Ocorre a troca da qtde_tokens pela quantidade de embeddings.
    # Entrada: (<qtde_tokensSi> x <768 ou 1024>) x (<qtde_tokensSj> x <768 ou 1024>)
    # Permute: <qtde_tokensSi> x <768 ou 1024>) x (<768 ou 1024> x <qtde_tokensSj>)
    Sprodabs = abs(torch.matmul(embeddingSi, embeddingSj.permute(1,0)))    
    # Saída: <qtde_tokensSi> x  <qtde_tokensSj>
    #print("Sprodabs=", Sprodabs.shape)

    # Encontra os maiores embeddings absolutos do produto para realizar a soma
    somaSprodabs = somaSprodabs + torch.mean(Sprodabs)

DprodabsPermutado = float(somaSprodabs)/float(np-1)
print("Dprodabs permutado:", DprodabsPermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']
Quantidade de sentenças: 4
Dprodabs permutado: 942.978515625


#### Compara as médias do produto absoluto dos embeddings das sentenças do documento original e permutado

Características das medidas:
- Dois documentos(vetores) que apontam em uma direção semelhante retornam um produto escalar positivo.
- Dois documentos perpendiculares retornam um produto escalar de zero.
- Dois documentos(vetores) que apontam em direções opostas retornam um produto escalar negativo.

In [ ]:
print("Dprodabs Original :", DprodabsOriginal)
print("Dprodabs Permutado:", DprodabsPermutado)

if (DprodabsOriginal > 0 and DprodabsPermutado > 0):
    print("As sentenças do documento original e as sentenças do documento permutado estão relacionadas!")
else:
  if (DprodabsOriginal < 0 and DprodabsPermutado > 0) or (DprodabsOriginal > 0 and DprodabsPermutado < 0):
    print("As sentenças do documento original e as sentenças do documento permutado não estão relacionadas!")
  else:  
    print("As sentenças do documento original e as sentenças do documento permutado não possuem relação!")          

Dprodabs Original : 932.7980143229166
Dprodabs Permutado: 942.978515625
As sentenças do documento original e as sentenças do documento permutado estão relacionadas!


### Produto Escalar entre os embeddings das sentenças

#### Calcula a média aritmética do produto escalar entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento orginal. 



In [ ]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaSprode = 0

# Percorre as sentenças do documento
for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

   # Encontra os maiores embeddings os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSi, linha = torch.max(embeddingSi, dim=0)        
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSi:", len(maiorEmbeddingSi))
        
    # Encontra os maiores embeddings os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSj, linha = torch.max(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSj:", len(maiorEmbeddingSj))

    # Produto escalar entre os embeddings de Si e Sj, pois Si e Sj possui uma única dimensão
    # https://pytorch.org/docs/master/generated/torch.matmul.html#torch.matmul
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Sprode = torch.matmul(maiorEmbeddingSi, maiorEmbeddingSj)
    # Saída: Um número real 
    #print("Sprode=", Sprode)
        
    somaSprode = somaSprode + Sprode

DprodeOriginal = float(somaSprode)/float(n-1)
print("Dprode Original:", DprodeOriginal)
    


Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Dprode Original: 1986.7194010416667


#### Calcula a média aritmética do produto escalar entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento permutado. 

In [ ]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaSprode = 0

# Percorre as sentenças do documento
for i in range(np-1):
    # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Encontra os maiores embeddings os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSi, linha = torch.max(embeddingSi, dim=0)        
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSi:", len(maiorEmbeddingSi))
        
    # Encontra os maiores embeddings os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSj, linha = torch.max(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSj:", len(maiorEmbeddingSj))
  
    # Produto escalar entre os embeddings de Si e Sj, pois Si e Sj possui uma única dimensão
    # https://pytorch.org/docs/master/generated/torch.matmul.html#torch.matmul
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Sprode = torch.matmul(maiorEmbeddingSi, maiorEmbeddingSj)    
    # Saída: Um número real 
    #print("Sprode=", Sprode)
    
    somaSprode = somaSprode + Sprode

DprodePermutado = float(somaSprode)/float(np-1)
print("Dprode permutado:", DprodePermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']
Quantidade de sentenças: 4
Dprode permutado: 2025.1311848958333


#### Compara as médias do produto escalar dos embeddings das sentenças do documento original e permutado

Dprod Original : 230.61181640625
Dprod Permutado: 227.75482177734375
Documento original tem maior similaridade com o produto entre as sentenças!

In [ ]:
print("Dprode Original :", DprodeOriginal)
print("Dprode Permutado:", DprodePermutado)

if (DprodeOriginal < DprodePermutado):
    print("Documento original tem maior similaridade com o produto entre as sentenças!")
else:
    print("Documento Permutado tem menor similaridade com o produto entre as sentenças!")

Dprode Original : 1986.7194010416667
Dprode Permutado: 2025.1311848958333
Documento original tem maior similaridade com o produto entre as sentenças!


### Produto Escalar Absoluto entre os embeddings das sentenças

#### Calcula a média aritmética do produto escalar absoluto entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento original. 

In [ ]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaSprodeabs = 0

# Percorre as sentenças do documento
for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

     # Encontra os maiores embeddings os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSi, linha = torch.max(embeddingSi, dim=0)        
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSi:", len(maiorEmbeddingSi))
        
    # Encontra os maiores embeddings os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSj, linha = torch.max(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSj:", len(maiorEmbeddingSj))
  
    # Produto escalar entre os embeddings de Si e Sj, pois Si e Sj possui uma única dimensão
    # https://pytorch.org/docs/master/generated/torch.matmul.html#torch.matmul
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Sprodeabs = abs(torch.matmul(maiorEmbeddingSi, maiorEmbeddingSj))
    # Saída: Um número real 
    #print("Sprodeabs=", Sprodeabs)

    somaSprodeabs = somaSprodeabs + Sprodeabs

DprodeabsOriginal = float(somaSprodeabs)/float(n-1)
print("Dprodabs Original:", DprodeabsOriginal)

Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Dprodabs Original: 1986.7194010416667


#### Calcula a média aritmética do produto escalar absoluto entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento permutado. 

In [ ]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaSprodeabs = 0

# Percorre as sentenças do documento
for i in range(np-1):
    # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)
    
     # Encontra os maiores embeddings os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSi, linha = torch.max(embeddingSi, dim=0)        
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSi:", len(maiorEmbeddingSi))
        
    # Encontra os maiores embeddings os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSj, linha = torch.max(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSj:", len(maiorEmbeddingSj))
    
    # Produto escalar entre os embeddings de Si e Sj, pois Si e Sj possui uma única dimensão
    # https://pytorch.org/docs/master/generated/torch.matmul.html#torch.matmul
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Sprodeabs = abs(torch.matmul(maiorEmbeddingSi, maiorEmbeddingSj))
    # Saída: Um número real 
    #print("Sprodeabs=", Sprodeabs)
    
    somaSprodeabs = somaSprodeabs + Sprodeabs

DprodeabsPermutado = float(somaSprodeabs)/float(np-1)
print("Dprodabs permutado:", DprodeabsPermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']
Quantidade de sentenças: 4
Dprodabs permutado: 2025.1311848958333


#### Compara as médias do produto escalar absoluto dos embeddings das sentenças do documento original e permutado

In [ ]:
print("Dprodabs Original :", DprodeabsOriginal)
print("Dprodabs Permutado:", DprodeabsPermutado)

if (DprodeabsOriginal < DprodeabsPermutado):
    print("Documento original tem maior similaridade com o produto absoluto entre as sentenças!")
else:
    print("Documento Permutado tem menor similaridade com o produto absoluto entre as sentenças!")

Dprodabs Original : 1986.7194010416667
Dprodabs Permutado: 2025.1311848958333
Documento original tem maior similaridade com o produto absoluto entre as sentenças!


### Média entre os embeddings das sentenças

#### Calcula a média aritmética entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento original. 

In [ ]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaSavg = 0

# Percorre as sentenças do documento
for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Encontra os maiores embeddings os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSi, linha = torch.max(embeddingSi, dim=0)        
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSi:", len(maiorEmbeddingSi))
        
    # Encontra os maiores embeddings os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSj, linha = torch.max(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSj:", len(maiorEmbeddingSj))
  
    # Média entre os embeddings de Si e Sj
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Savg = (maiorEmbeddingSi.add(maiorEmbeddingSj))/2.0  
    # Saída: <768 ou 1024>
    #print("Savg=", Savg.shape)  

    # Calcula a média dos embeddings do cálculo para realizar a soma
    somaSavg = somaSavg + torch.mean(Savg)

DavgOriginal = float(somaSavg)/float(n-1)
print("Davg Original:", DavgOriginal)

Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Davg Original: 0.5656667153040568


#### Calcula a média aritmética entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento permutado.

In [ ]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaSavg = 0

# Percorre as sentenças do documento
for i in range(np-1):
    # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Encontra os maiores embeddings os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSi, linha = torch.max(embeddingSi, dim=0)        
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSi:", len(maiorEmbeddingSi))
        
    # Encontra os maiores embeddings os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSj, linha = torch.max(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSj:", len(maiorEmbeddingSj))
  
    # Média entre os embeddings de Si e Sj
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>) 
    Savg = (maiorEmbeddingSi.add(maiorEmbeddingSj))/2.0  
    # Saída: <768 ou 1024>
    #print("Savg=", Savg.shape)  

    # Calcula a média dos embeddings do cálculo para realizar a soma
    somaSavg = somaSavg + torch.mean(Savg)

DavgPermutado = float(somaSavg)/float(np-1)
print("Davg permutado:", DavgPermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']
Quantidade de sentenças: 4
Davg permutado: 0.5695276260375977


#### Compara as médias dos embeddings das sentenças do documento original e permutado


In [ ]:
print("Davg Original :", DavgOriginal)
print("Davg Permutado:", DavgPermutado)

if (DavgOriginal < DavgPermutado):
    print("Documento original tem maior similaridade com a média entre as sentenças!")
else:
    print("Documento Permutado tem menor similaridade com a mpedia entre as sentenças!")

Davg Original : 0.5656667153040568
Davg Permutado: 0.5695276260375977
Documento original tem maior similaridade com a média entre as sentenças!


### Similaridade de cosseno entre os embeddings das sentenças

In [ ]:
# Import das bibliotecas.
from scipy.spatial.distance import cosine

def similaridadeCoseno(sentenca1, sentenca2):
  similaridade = 1 - cosine(sentenca1, sentenca2)
  return similaridade

#### Calcula a média aritmética da similaridade do coseno entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento original. 

In [ ]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaScos = 0

# Percorre as sentenças do documento
for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Encontra os maiores embeddings os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSi, linha = torch.max(embeddingSi, dim=0)        
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSi:", len(maiorEmbeddingSi))
        
    # Encontra os maiores embeddings os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSj, linha = torch.max(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSj:", len(maiorEmbeddingSj))
  
    # Similaridade entre os embeddings Si e Sj
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Scos = similaridadeCoseno(maiorEmbeddingSi, maiorEmbeddingSj)
    # Saída: Um número real
    
    # Acumula a medida
    somaScos = somaScos + Scos

DcosOriginal = float(somaScos)/float(n-1)
print("Dcos Original:", DcosOriginal)  


Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Dcos Original: 0.846483051776886


#### Calcula a média aritmética da similaridade do coseno entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento permutado. 

In [ ]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaScos = 0

# Percorre as sentenças do documento
for i in range(np-1):
    # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Encontra os maiores embeddings os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSi, linha = torch.max(embeddingSi, dim=0)        
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSi:", len(maiorEmbeddingSi))
        
    # Encontra os maiores embeddings os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSj, linha = torch.max(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSj:", len(maiorEmbeddingSj))
  
    # Similaridade entre os embeddings Si e Sj
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Scos = similaridadeCoseno(maiorEmbeddingSi, maiorEmbeddingSj)
    # Saída: Um número real
    
    # Acumula a medida
    somaScos = somaScos + Scos

DcosPermutado = float(somaScos)/float(np-1)
print("Dcos Original:", DcosPermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']
Quantidade de sentenças: 4
Dcos Original: 0.8591363628705343


#### Compara as médias da similaridade de cosseno dos embeddings das sentenças do documento original e permutado

Características das medidas:
- Documentos com sentenças iguais resulta uma medida igual a 1.
- Documentos com sentenças diferenntes resulta uma medida menor que 1.
- Documento com sentenças muito diferentes apresentam valores menores que 1.
- Documentos iguais resultam em medidas iguais. 
- É uma medida de similaridade.



In [ ]:
print("Dcos Original :", DcosOriginal)
print("Dcos Permutado:", DcosPermutado)

if (DcosOriginal > DcosPermutado):
    print("Documento original tem maior similaridade de cosseno entre as sentenças!")
else:
    print("Documento Permutado tem menor similaridade de cosseno entre as sentenças!")

Dcos Original : 0.846483051776886
Dcos Permutado: 0.8591363628705343
Documento Permutado tem menor similaridade de cosseno entre as sentenças!


### Distância euclidiana entre os embeddings das sentenças

Possui outros nomes como distância L2 ou norma L2.

In [ ]:
# Import das bibliotecas.
from scipy.spatial.distance import euclidean

def distanciaEuclidiana(sentenca1, sentenca2):
  distancia = euclidean(sentenca1, sentenca2)

  return distancia

#### Calcula a média aritmética da distância euclidiana entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento original. 

In [ ]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaSeuc = 0

# Percorre as sentenças do documento
for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Encontra os maiores embeddings os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSi, linha = torch.max(embeddingSi, dim=0)        
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSi:", len(maiorEmbeddingSi))
        
    # Encontra os maiores embeddings os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSj, linha = torch.max(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSj:", len(maiorEmbeddingSj))
  
    # Diferença entre os embeddings Si e Sj
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Seuc = distanciaEuclidiana(maiorEmbeddingSi, maiorEmbeddingSj)
    # Saída: Um número real
    
    # Acumula a medida
    somaSeuc = somaSeuc + Seuc

DeucOriginal = float(somaSeuc)/float(n-1)
print("Deuc Original:", DeucOriginal)  


Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Deuc Original: 26.750072479248047


#### Calcula a média aritmética da distância euclidiana entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento permutado. 

In [ ]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaSeuc = 0

# Percorre as sentenças do documento
for i in range(np-1):
    # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Encontra os maiores embeddings os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSi, linha = torch.max(embeddingSi, dim=0)        
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSi:", len(maiorEmbeddingSi))
        
    # Encontra os maiores embeddings os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSj, linha = torch.max(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSj:", len(maiorEmbeddingSj))
  
    # Diferença entre os embeddings Si e Sj
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Seuc = distanciaEuclidiana(maiorEmbeddingSi, maiorEmbeddingSj)
    # Saída: Um número real
    
    # Acumula a medida
    somaSeuc = somaSeuc + Seuc

DeucPermutado = float(somaSeuc)/float(np-1)
print("Deuc Original:", DeucPermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']
Quantidade de sentenças: 4
Deuc Original: 25.777802149454754


#### Compara as médias da distância euclidiana dos embeddings das sentenças do documento original e permutado

Características das medidas:
- Documentos com sentenças iguais resulta uma medida igual a 0.
- Documentos com sentenças diferenntes resulta uma medida maior que 0.
- Documento com sentenças muito diferentes apresentam valores maiores que 0.
- Documentos iguais resultam em medidas iguais. 
- É uma medida de diferença.



In [ ]:
print("Deuc Original :", DeucOriginal)
print("Deuc Permutado:", DeucPermutado)

if (DeucOriginal > DeucPermutado):
    print("Documento original tem maior distância euclidiana entre as sentenças!")
else:
    print("Documento Permutado tem menor distância euclidiana entre as sentenças!")

Deuc Original : 26.750072479248047
Deuc Permutado: 25.777802149454754
Documento original tem maior distância euclidiana entre as sentenças!


### Distância Manhattan entre os embeddings das sentenças

Possui outros nomes como distância Cityblock, distância L1, norma L1 e métrica do táxi.

In [ ]:
# Import das bibliotecas.
from scipy.spatial.distance import cityblock

def distanciaEManhattan(sentenca1, sentenca2):
  distancia = cityblock(sentenca1, sentenca2)

  return distancia

#### Calcula a média aritmética da distância de manhattan entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento original. 

In [ ]:
print("Documento Original  :", str(documento_original))
print("Quantidade de sentenças:",len(documento_original))

# Quantidade de sentenças no documento
n = len(documento_original)

somaSman = 0

# Percorre as sentenças do documento
for i in range(n-1):
    # Seleciona as sentenças do documento  
    Si = documento_original[i]
    Sj = documento_original[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Encontra os maiores embeddings os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSi, linha = torch.max(embeddingSi, dim=0)        
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSi:", len(maiorEmbeddingSi))
        
    # Encontra os maiores embeddings os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSj, linha = torch.max(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSj:", len(maiorEmbeddingSj)) 
  
    # Diferença entre os embeddings Si e Sj
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Sman = distanciaEManhattan(maiorEmbeddingSi, maiorEmbeddingSj)
    # Saída: Um número real
    
    # Acumula a medida
    somaSman = somaSman + Sman

DmanOriginal = float(somaSman)/float(n-1)
print("Dman Original:", DmanOriginal)

Documento Original  : ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']
Quantidade de sentenças: 4
Dman Original: 1132.6749877929688


#### Calcula a média aritmética da distância de manhattan entre os embeddings das sentenças utilizando a média aritmética dos tokens do documento permutado.

In [ ]:
print("Documento Permutado :", str(documento_permutado))
print("Quantidade de sentenças:", len(documento_permutado))

# Quantidade de sentenças no documento
np = len(documento_permutado)

somaSman = 0

# Percorre as sentenças do documento
for i in range(np-1):
    # Seleciona as sentenças do documento  
    Si = documento_permutado[i]
    Sj = documento_permutado[i+1]

    # Recupera os embeddings das sentenças no embeddings do documento original    
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSi, tokenizador
    embeddingSi = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Si, tokenizer)
    # Saída: <qtde_tokens_Si> x <768 ou 1024>
    #print("embeddingSi=", embeddingSi.shape)
    # Entrada: <qtde_tokens_documento> x <768 ou 1024>, texto documento,  SentencaSj, tokenizador
    embeddingSj = getEmbeddingSentencaDocumentoEmbedding(embeddingDocumentoOriginal, stringDocumentoPermutado, Sj, tokenizer)
    # Saída: <qtde_tokens_Sj> x <768 ou 1024>
    #print("embeddingSj=", embeddingSj.shape)

    # Encontra os maiores embeddings os tokens de Si, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSi, linha = torch.max(embeddingSi, dim=0)        
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSi:", len(maiorEmbeddingSi))
        
    # Encontra os maiores embeddings os tokens de Sj, removendo a primeira dimensão.
    # Entrada: <qtde_tokens> x <768 ou 1024>  
    maiorEmbeddingSj, linha = torch.max(embeddingSj, dim=0)    
    # Saída: <768 ou 1024>
    #print("maiorEmbeddingSj:", len(maiorEmbeddingSj)) 
  
    # Diferença entre os embeddings Si e Sj
    # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
    Sman = distanciaEManhattan(maiorEmbeddingSi, maiorEmbeddingSj)
    # Saída: Um número real
    
    # Acumula a medida
    somaSman = somaSman + Sman

DmanPermutado = float(somaSman)/float(np-1)
print("Deuc Original:", DmanPermutado)

Documento Permutado : ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']
Quantidade de sentenças: 4
Deuc Original: 1082.0957641601562


#### Compara as médias da distância de manhattan dos embeddings das sentenças do documento original e permutado

Características das medidas:
- Documentos com sentenças iguais resulta uma medida igual a 0.
- Documentos com sentenças diferenntes resulta uma medida maior que 0.
- Documento com sentenças muito diferentes apresentam valores maiores que 0.
- Documentos iguais resultam em medidas iguais. 
- É uma medida de diferença.



In [ ]:
print("Dman Original :", DmanOriginal)
print("Dman Permutado:", DmanPermutado)

if (DeucOriginal > DeucPermutado):
    print("Documento original tem maior distância de manhattan entre as sentenças!")
else:
    print("Documento Permutado tem menor distância de manhattan entre as sentenças!")

Dman Original : 1132.6749877929688
Dman Permutado: 1082.0957641601562
Documento original tem maior distância de manhattan entre as sentenças!


### Resumo

Resultado das medidas utilizando as quatro últimas camadas do BERT.

Base(MEAN):
- Dsub       :   0.00028153          -0.00050018
- Dubabs     :   0.26571635          0.24592040
- Dprod      :   932.79801432          942.97851562
- Dprodabs   :   932.79801432          942.97851562
- Dprode     :   932.79809570          942.97859701
- Dprodeabs  :   932.79809570          942.97859701
- Davg       :   -0.02508242          -0.02519175
- Dcos       :   0.83016231          0.85469584
- Deuc       :   19.28004646          17.82995224
- Dman       :   816.28061930          755.46746826

Base(MAX):
- Dsub       :   -0.02318370          0.03090549
- Dubabs     :   0.36870933          0.35224470
- Dprod      :   932.79801432          942.97851562
- Dprodabs   :   932.79801432          942.97851562
- Dprode     :   1986.71940104          2025.13118490
- Dprodeabs  :   1986.71940104          2025.13118490
- Davg       :   0.56566672          0.56952763
- Dcos       :   0.84648305          0.85913636
- Deuc       :   26.75007248          25.77780215
- Dman       :   1132.67498779          1082.09576416

----------------------
Resultado das medidas utilizando a última camada do BERT.

Base(MEAN):
- Dsub       :   0.00074918          -0.00064076
- Dubabs     :   0.17778097          0.17206367
- Dprod      :   43.70739746          43.55047607
- Dprodabs   :   43.70739746          43.55047607
- Dprode     :   43.70739746          43.55047607
- Dprodeabs  :   43.70739746          43.55047607
- Davg       :   -0.00276791          -0.00271371
- Dcos       :   0.67999101          0.69653825
- Deuc       :   6.34085274          6.15486606
- Dman       :   136.53579712          132.14489237

Base(MAX):
- Dsub       :   -0.02765538          0.02360132
- Dubabs     :   0.26021336          0.25770577
- Dprod      :   43.70739746          43.55047607
- Dprodabs   :   43.70739746          43.55047607
- Dprode     :   200.03959147          199.48846436
- Dprodeabs  :   200.03959147          199.48846436
- Davg       :   0.43032602          0.42829903
- Dcos       :   0.81970002          0.82297109
- Deuc       :   9.42460442          9.29481570
- Dman       :   199.84384664          197.91803996

In [ ]:
print("Resultado das medidas utilizando a última camada do BERT")
print("Documento  :   Original            Permutado")
print('Dsub       :   {:.8f}          {:.8f}'.format(DsubOriginal,DsubPermutado))
print('Dubabs     :   {:.8f}          {:.8f}'.format(DsubabsOriginal,DsubabsPermutado))
print('Dprod      :   {:.8f}          {:.8f}'.format(DprodOriginal,DprodPermutado))
print('Dprodabs   :   {:.8f}          {:.8f}'.format(DprodabsOriginal,DprodabsPermutado))
print('Dprode     :   {:.8f}          {:.8f}'.format(DprodeOriginal,DprodePermutado))
print('Dprodeabs  :   {:.8f}          {:.8f}'.format(DprodeabsOriginal,DprodeabsPermutado))
print('Davg       :   {:.8f}          {:.8f}'.format(DavgOriginal,DavgPermutado))
print('Dcos       :   {:.8f}          {:.8f}'.format(DcosOriginal,DcosPermutado))
print('Deuc       :   {:.8f}          {:.8f}'.format(DeucOriginal,DeucPermutado))
print('Dman       :   {:.8f}          {:.8f}'.format(DmanOriginal,DmanPermutado))

Resultado das medidas utilizando a última camada do BERT
Documento  :   Original            Permutado
Dsub       :   -0.02318370          0.03090549
Dubabs     :   0.36870933          0.35224470
Dprod      :   932.79801432          942.97851562
Dprodabs   :   932.79801432          942.97851562
Dprode     :   1986.71940104          2025.13118490
Dprodeabs  :   1986.71940104          2025.13118490
Davg       :   0.56566672          0.56952763
Dcos       :   0.84648305          0.85913636
Deuc       :   26.75007248          25.77780215
Dman       :   1132.67498779          1082.09576416
